In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Install Dependencies

In [2]:
%%capture
# !pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install nltk spacy
!python -m spacy download en_core_web_sm

In [3]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip uninstall torch-sparse  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 55.0 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 57.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.6 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-sqxm0ybp
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-sqxm0ybp
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 025ef209ee57d56cb9e736fe0d0ed076e13c5ad4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=10

#Download Data

<h3><font color=lightgreen>ConceptNet Full KnowledgeGraph (including none english data)</font></h3>

In [ ]:
# download ConceptNet
%mkdir -p data/
%mkdir -p data/cpnet/
!wget -nc -P data/cpnet/ https://s3.amazonaws.com/conceptnet/downloads/2018/edges/conceptnet-assertions-5.6.0.csv.gz

In [ ]:
%cd data/cpnet/
!yes n | gzip -d conceptnet-assertions-5.6.0.csv.gz

In [ ]:
# download ConceptNet entity embedding
!wget https://csr.s3-us-west-1.amazonaws.com/tzw.ent.npy
%cd ../../

<h3><font color=Hotpink>CommensenseQA datset</font></h3>

In [ ]:
# download CommensenseQA dataset
%mkdir -p data/csqa/
!wget -nc -P data/csqa/ https://s3.amazonaws.com/commensenseqa/train_rand_split.jsonl
!wget -nc -P data/csqa/ https://s3.amazonaws.com/commensenseqa/dev_rand_split.jsonl
!wget -nc -P data/csqa/ https://s3.amazonaws.com/commensenseqa/test_rand_split_no_answers.jsonl

# create output folders
%mkdir -p data/csqa/grounded/
%mkdir -p data/csqa/graph/
%mkdir -p data/csqa/statement/

<font color=yellow>OBQA datset</font>

In [ ]:
# # download OpenBookQA dataset
# !wget -nc -P data/obqa/ https://s3-us-west-2.amazonaws.com/ai2-website/data/OpenBookQA-V1-Sep2018.zip
# !yes n | unzip data/obqa/OpenBookQA-V1-Sep2018.zip -d data/obqa/

# # create output folders
# %mkdir -p data/obqa/fairseq/official/
# %mkdir -p data/obqa/grounded/
# %mkdir -p data/obqa/graph/
# %mkdir -p data/obqa/statement/


#Download Preprocessed data (ConceptNet and CommonsenseQA and OBQA)

In [ ]:
!mv data data_old

!wget https://nlp.stanford.edu/projects/myasu/QAGNN/data_preprocessed_release.zip
!unzip data_preprocessed_release.zip
!mv data_preprocessed_release  data

mv: cannot stat 'data': No such file or directory
--2023-12-12 14:59:58--  https://nlp.stanford.edu/projects/myasu/QAGNN/data_preprocessed_release.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4226838278 (3.9G) [application/zip]
Saving to: ‘data_preprocessed_release.zip’

data_preprocessed_r 100%[===================>]   3.94G  40.1MB/s    in 2m 11s  

2023-12-12 15:02:09 (30.8 MB/s) - ‘data_preprocessed_release.zip’ saved [4226838278/4226838278]

Archive:  data_preprocessed_release.zip
   creating: data_preprocessed_release/
   creating: data_preprocessed_release/cpnet/
  inflating: data_preprocessed_release/cpnet/conceptnet.en.unpruned.graph  
  inflating: data_preprocessed_release/cpnet/conceptnet.en.csv  
  inflating: data_preprocessed_release/cpnet/matcher_patterns.json  
  inflating: data_preprocessed_release/cpnet/conceptne

#Preprocess the data

###from utils.convert_csqa import convert_to_entailment

In [ ]:
"""
Script to convert the retrieved HITS into an entailment dataset
USAGE:
 python convert_csqa.py input_file output_file

JSONL format of files
 1. input_file:
 {
   "id": "d3b479933e716fb388dfb297e881054c",
   "question": {
      "stem": "If a lantern is not for sale, where is it likely to be?"
      "choices": [{"label": "A", "text": "antique shop"}, {"label": "B", "text": "house"}, {"label": "C", "text": "dark place"}]
    },
     "answerKey":"B"
  }

 2. output_file:
   {
   "id": "d3b479933e716fb388dfb297e881054c",
   "question": {
      "stem": "If a lantern is not for sale, where is it likely to be?"
      "choices": [{"label": "A", "text": "antique shop"}, {"label": "B", "text": "house"}, {"label": "C", "text": "dark place"}]
    },
    "answerKey":"B",

    "statements":[
        {label:true, stem: "If a lantern is not for sale, it likely to be at house"},
        {label:false, stem: "If a lantern is not for sale, it likely to be at antique shop"},
        {label:false, stem: "If a lantern is not for sale, it likely to be at dark place"}
        ]
  }
"""

import json
import re
import sys
from tqdm import tqdm

__all__ = ['convert_to_entailment']

# String used to indicate a blank
BLANK_STR = "___"


def convert_to_entailment(qa_file: str, output_file: str, ans_pos: bool=False):
    print(f'converting {qa_file} to entailment dataset...')
    nrow = sum(1 for _ in open(qa_file, 'r'))
    with open(output_file, 'w') as output_handle, open(qa_file, 'r') as qa_handle:
        # print("Writing to {} from {}".format(output_file, qa_file))
        for line in tqdm(qa_handle, total=nrow):
            json_line = json.loads(line)
            output_dict = convert_qajson_to_entailment(json_line, ans_pos)
            output_handle.write(json.dumps(output_dict))
            output_handle.write("\n")
    print(f'converted statements saved to {output_file}')
    print()


# Convert the QA file json to output dictionary containing premise and hypothesis
def convert_qajson_to_entailment(qa_json: dict, ans_pos: bool):
    question_text = qa_json["question"]["stem"]
    choices = qa_json["question"]["choices"]
    for choice in choices:
        choice_text = choice["text"]
        pos = None
        if not ans_pos:
            statement = create_hypothesis(get_fitb_from_question(question_text), choice_text, ans_pos)
        else:
            statement, pos = create_hypothesis(get_fitb_from_question(question_text), choice_text, ans_pos)
        create_output_dict(qa_json, statement,  choice["label"] == qa_json.get("answerKey", "A"), ans_pos, pos)

    return qa_json


# Get a Fill-In-The-Blank (FITB) statement from the question text. E.g. "George wants to warm his
# hands quickly by rubbing them. Which skin surface will produce the most heat?" ->
# "George wants to warm his hands quickly by rubbing them. ___ skin surface will produce the most
# heat?
def get_fitb_from_question(question_text: str) -> str:
    fitb = replace_wh_word_with_blank(question_text)
    if not re.match(".*_+.*", fitb):
        # print("Can't create hypothesis from: '{}'. Appending {} !".format(question_text, BLANK_STR))
        # Strip space, period and question mark at the end of the question and add a blank
        fitb = re.sub(r"[\.\? ]*$", "", question_text.strip()) + " " + BLANK_STR
    return fitb


# Create a hypothesis statement from the the input fill-in-the-blank statement and answer choice.
def create_hypothesis(fitb: str, choice: str, ans_pos: bool) -> str:

    if ". " + BLANK_STR in fitb or fitb.startswith(BLANK_STR):
        choice = choice[0].upper() + choice[1:]
    else:
        choice = choice.lower()
    # Remove period from the answer choice, if the question doesn't end with the blank
    if not fitb.endswith(BLANK_STR):
        choice = choice.rstrip(".")
    # Some questions already have blanks indicated with 2+ underscores
    if not ans_pos:
        try:
            hypothesis = re.sub("__+", choice, fitb)
        except:
            print (choice, fitb)
        return hypothesis
    choice = choice.strip()
    m = re.search("__+", fitb)
    start = m.start()

    length = (len(choice) - 1) if fitb.endswith(BLANK_STR) and choice[-1] in ['.', '?', '!'] else len(choice)
    hypothesis = re.sub("__+", choice, fitb)

    return hypothesis, (start, start + length)


# Identify the wh-word in the question and replace with a blank
def replace_wh_word_with_blank(question_str: str):
    # if "What is the name of the government building that houses the U.S. Congress?" in question_str:
    #     print()
    question_str = question_str.replace("What's", "What is")
    question_str = question_str.replace("whats", "what")
    question_str = question_str.replace("U.S.", "US")
    wh_word_offset_matches = []
    wh_words = ["which", "what", "where", "when", "how", "who", "why"]
    for wh in wh_words:
        # Some Turk-authored SciQ questions end with wh-word
        # E.g. The passing of traits from parents to offspring is done through what?

        if wh == "who" and "people who" in question_str:
            continue

        m = re.search(wh + r"\?[^\.]*[\. ]*$", question_str.lower())
        if m:
            wh_word_offset_matches = [(wh, m.start())]
            break
        else:
            # Otherwise, find the wh-word in the last sentence
            m = re.search(wh + r"[ ,][^\.]*[\. ]*$", question_str.lower())
            if m:
                wh_word_offset_matches.append((wh, m.start()))
            # else:
            #     wh_word_offset_matches.append((wh, question_str.index(wh)))

    # If a wh-word is found
    if len(wh_word_offset_matches):
        # Pick the first wh-word as the word to be replaced with BLANK
        # E.g. Which is most likely needed when describing the change in position of an object?
        wh_word_offset_matches.sort(key=lambda x: x[1])
        wh_word_found = wh_word_offset_matches[0][0]
        wh_word_start_offset = wh_word_offset_matches[0][1]
        # Replace the last question mark with period.
        question_str = re.sub(r"\?$", ".", question_str.strip())
        # Introduce the blank in place of the wh-word
        fitb_question = (question_str[:wh_word_start_offset] + BLANK_STR +
                         question_str[wh_word_start_offset + len(wh_word_found):])
        # Drop "of the following" as it doesn't make sense in the absence of a multiple-choice
        # question. E.g. "Which of the following force ..." -> "___ force ..."
        final = fitb_question.replace(BLANK_STR + " of the following", BLANK_STR)
        final = final.replace(BLANK_STR + " of these", BLANK_STR)
        return final

    elif " them called?" in question_str:
        return question_str.replace(" them called?", " " + BLANK_STR + ".")
    elif " meaning he was not?" in question_str:
        return question_str.replace(" meaning he was not?", " he was not " + BLANK_STR + ".")
    elif " one of these?" in question_str:
        return question_str.replace(" one of these?", " " + BLANK_STR + ".")
    elif re.match(r".*[^\.\?] *$", question_str):
        # If no wh-word is found and the question ends without a period/question, introduce a
        # blank at the end. e.g. The gravitational force exerted by an object depends on its
        return question_str + " " + BLANK_STR
    else:
        # If all else fails, assume "this ?" indicates the blank. Used in Turk-authored questions
        # e.g. Virtually every task performed by living organisms requires this?
        return re.sub(r" this[ \?]", " ___ ", question_str)


# Create the output json dictionary from the input json, premise and hypothesis statement
def create_output_dict(input_json: dict, statement: str, label: bool, ans_pos: bool, pos=None) -> dict:
    if "statements" not in input_json:
        input_json["statements"] = []
    if not ans_pos:
        input_json["statements"].append({"label": label, "statement": statement})
    else:
        input_json["statements"].append({"label": label, "statement": statement, "ans_pos": pos})
    return input_json


# if __name__ == "__main__":
#     if len(sys.argv) < 3:
#         raise ValueError("Provide at least two arguments: "
#                          "json file with hits, output file name")
#     convert_to_entailment(sys.argv[1], sys.argv[2])


###from utils.convert_obqa import convert_to_obqa_statement

In [ ]:
import json
import re
import sys
from tqdm import tqdm

__all__ = ['convert_to_obqa_statement']

# String used to indicate a blank
BLANK_STR = "___"


def convert_to_obqa_statement(qa_file: str, output_file1: str, output_file2: str):
    print(f'converting {qa_file} to entailment dataset...')
    nrow = sum(1 for _ in open(qa_file, 'r'))
    with open(output_file1, 'w') as output_handle1, open(output_file2, 'w') as output_handle2, open(qa_file, 'r') as qa_handle:
        # print("Writing to {} from {}".format(output_file, qa_file))
        for line in tqdm(qa_handle, total=nrow):
            json_line = json.loads(line)
            output_dict = convert_qajson_to_entailment(json_line)
            output_handle1.write(json.dumps(output_dict))
            output_handle1.write("\n")
            output_handle2.write(json.dumps(output_dict))
            output_handle2.write("\n")
    print(f'converted statements saved to {output_file1}, {output_file2}')
    print()


# Convert the QA file json to output dictionary containing premise and hypothesis
def convert_obqa_convert_qajson_to_entailment(qa_json: dict):
    question_text = qa_json["question"]["stem"]
    choices = qa_json["question"]["choices"]
    for choice in choices:
        choice_text = choice["text"]
        statement = question_text + ' ' + choice_text
        create_output_dict(qa_json, statement, choice["label"] == qa_json.get("answerKey", "A"))

    return qa_json


# Create the output json dictionary from the input json, premise and hypothesis statement
def convert_obqa_create_output_dict(input_json: dict, statement: str, label: bool) -> dict:
    if "statements" not in input_json:
        input_json["statements"] = []
    input_json["statements"].append({"label": label, "statement": statement})
    return input_json


###from utils.conceptnet import extract_english, construct_graph

from .utils import check_file

In [ ]:
import json
import os
import time
import argparse


# def bool_flag(v):
#     if v.lower() in ('yes', 'true', 't', 'y', '1'):
#         return True
#     elif v.lower() in ('no', 'false', 'f', 'n', '0'):
#         return False
#     else:
#         raise argparse.ArgumentTypeError('Boolean value expected.')


# def check_path(path):
#     d = os.path.dirname(path)
#     if not os.path.exists(d):
#         os.makedirs(d)


def check_file(file):
    return os.path.isfile(file)


# def export_config(config, path):
#     param_dict = dict(vars(config))
#     check_path(path)
#     with open(path, 'w') as fout:
#         json.dump(param_dict, fout, indent=4)


# def freeze_net(module):
#     for p in module.parameters():
#         p.requires_grad = False


# def unfreeze_net(module):
#     for p in module.parameters():
#         p.requires_grad = True


# def test_data_loader_ms_per_batch(data_loader, max_steps=10000):
#     start = time.time()
#     n_batch = sum(1 for batch, _ in zip(data_loader, range(max_steps)))
#     return (time.time() - start) * 1000 / n_batch


In [ ]:
import networkx as nx
from networkx.utils import open_file

try:
    import cPickle as pickle
except ImportError:
    import pickle


@open_file(1, mode='wb')
def write_gpickle(G, path, protocol=pickle.HIGHEST_PROTOCOL):
    """Write graph in Python pickle format.

    Pickles are a serialized byte stream of a Python object [1]_.
    This format will preserve Python objects used as nodes or edges.

    Parameters
    ----------
    G : graph
       A NetworkX graph

    path : file or string
       File or filename to write.
       Filenames ending in .gz or .bz2 will be compressed.

    protocol : integer
        Pickling protocol to use. Default value: ``pickle.HIGHEST_PROTOCOL``.

    Examples
    --------
    >>> G = nx.path_graph(4)
    >>> nx.write_gpickle(G, "test.gpickle")

    References
    ----------
    .. [1] https://docs.python.org/2/library/pickle.html
    """
    pickle.dump(G, path, protocol)


@open_file(0, mode='rb')
def read_gpickle(path):
    """Read graph object in Python pickle format.

    Pickles are a serialized byte stream of a Python object [1]_.
    This format will preserve Python objects used as nodes or edges.

    Parameters
    ----------
    path : file or string
       File or filename to write.
       Filenames ending in .gz or .bz2 will be uncompressed.

    Returns
    -------
    G : graph
       A NetworkX graph

    Examples
    --------
    >>> G = nx.path_graph(4)
    >>> nx.write_gpickle(G, "test.gpickle")
    >>> G = nx.read_gpickle("test.gpickle")

    References
    ----------
    .. [1] https://docs.python.org/2/library/pickle.html
    """
    return pickle.load(path)

# fixture for nose tests


def teardown_module(module):
    import os
    os.unlink('test.gpickle')

In [ ]:
import networkx as nx
import nltk
import json
import math
from tqdm import tqdm
import numpy as np
import sys
import codecs


# try:
#     from .utils import check_file
# except ImportError:
#     from utils import check_file

__all__ = ['extract_english', 'construct_graph', 'merged_relations']

relation_groups = [
    'atlocation/locatednear',
    'capableof',
    'causes/causesdesire/*motivatedbygoal',
    'createdby',
    'desires',
    'antonym/distinctfrom',
    'hascontext',
    'hasproperty',
    'hassubevent/hasfirstsubevent/haslastsubevent/hasprerequisite/entails/mannerof',
    'isa/instanceof/definedas',
    'madeof',
    'notcapableof',
    'notdesires',
    'partof/*hasa',
    'relatedto/similarto/synonym',
    'usedfor',
    'receivesaction',
]

merged_relations = [
    'antonym',
    'atlocation',
    'capableof',
    'causes',
    'createdby',
    'isa',
    'desires',
    'hassubevent',
    'partof',
    'hascontext',
    'hasproperty',
    'madeof',
    'notcapableof',
    'notdesires',
    'receivesaction',
    'relatedto',
    'usedfor',
]

relation_text = [
    'is the antonym of',
    'is at location of',
    'is capable of',
    'causes',
    'is created by',
    'is a kind of',
    'desires',
    'has subevent',
    'is part of',
    'has context',
    'has property',
    'is made of',
    'is not capable of',
    'does not desires',
    'is',
    'is related to',
    'is used for',
]


def load_merge_relation():
    relation_mapping = dict()
    for line in relation_groups:
        ls = line.strip().split('/')
        rel = ls[0]
        for l in ls:
            if l.startswith("*"):
                relation_mapping[l[1:]] = "*" + rel
            else:
                relation_mapping[l] = rel
    return relation_mapping


def del_pos(s):
    """
    Deletes part-of-speech encoding from an entity string, if present.
    :param s: Entity string.
    :return: Entity string with part-of-speech encoding removed.
    """
    if s.endswith("/n") or s.endswith("/a") or s.endswith("/v") or s.endswith("/r"):
        s = s[:-2]
    return s


def extract_english(conceptnet_path, output_csv_path, output_vocab_path):
    """
    Reads original conceptnet csv file and extracts all English relations (head and tail are both English entities) into
    a new file, with the following format for each line: <relation> <head> <tail> <weight>.
    :return:
    """
    print('extracting English concepts and relations from ConceptNet...')
    relation_mapping = load_merge_relation()
    num_lines = sum(1 for line in open(conceptnet_path, 'r', encoding='utf-8'))
    cpnet_vocab = []
    concepts_seen = set()
    with open(conceptnet_path, 'r', encoding="utf8") as fin, \
            open(output_csv_path, 'w', encoding="utf8") as fout:
        for line in tqdm(fin, total=num_lines):
            toks = line.strip().split('\t')
            if toks[2].startswith('/c/en/') and toks[3].startswith('/c/en/'):
                """
                Some preprocessing:
                    - Remove part-of-speech encoding.
                    - Split("/")[-1] to trim the "/c/en/" and just get the entity name, convert all to
                    - Lowercase for uniformity.
                """
                rel = toks[1].split("/")[-1].lower()
                head = del_pos(toks[2]).split("/")[-1].lower()
                tail = del_pos(toks[3]).split("/")[-1].lower()

                if not head.replace("_", "").replace("-", "").isalpha():
                    continue
                if not tail.replace("_", "").replace("-", "").isalpha():
                    continue
                if rel not in relation_mapping:
                    continue

                rel = relation_mapping[rel]
                if rel.startswith("*"):
                    head, tail, rel = tail, head, rel[1:]

                data = json.loads(toks[4])

                fout.write('\t'.join([rel, head, tail, str(data["weight"])]) + '\n')

                for w in [head, tail]:
                    if w not in concepts_seen:
                        concepts_seen.add(w)
                        cpnet_vocab.append(w)

    with codecs.open(output_vocab_path, 'w', "utf-8") as fout:
        for word in cpnet_vocab:
            fout.write(word+"\n")

    print(f'extracted ConceptNet csv file saved to {output_csv_path}')
    print(f'extracted concept vocabulary saved to {output_vocab_path}')
    print()


def construct_graph(cpnet_csv_path, cpnet_vocab_path, output_path, prune=True):
    print('generating ConceptNet graph file...')

    nltk.download('stopwords', quiet=True)
    nltk_stopwords = nltk.corpus.stopwords.words('english')
    nltk_stopwords += ["like", "gone", "did", "going", "would", "could",
                       "get", "in", "up", "may", "wanter"]  # issue: mismatch with the stop words in grouding.py

    blacklist = set(["uk", "us", "take", "make", "object", "person", "people"])  # issue: mismatch with the blacklist in grouding.py

    concept2id = {}
    id2concept = {}
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        id2concept = [w.strip() for w in fin]
    concept2id = {w: i for i, w in enumerate(id2concept)}

    id2relation = merged_relations
    relation2id = {r: i for i, r in enumerate(id2relation)}

    graph = nx.MultiDiGraph()
    nrow = sum(1 for _ in open(cpnet_csv_path, 'r', encoding='utf-8'))
    with open(cpnet_csv_path, "r", encoding="utf8") as fin:

        def not_save(cpt):
            if cpt in blacklist:
                return True
            '''originally phrases like "branch out" would not be kept in the graph'''
            # for t in cpt.split("_"):
            #     if t in nltk_stopwords:
            #         return True
            return False

        attrs = set()

        for line in tqdm(fin, total=nrow):
            ls = line.strip().split('\t')
            rel = relation2id[ls[0]]
            subj = concept2id[ls[1]]
            obj = concept2id[ls[2]]
            weight = float(ls[3])
            if prune and (not_save(ls[1]) or not_save(ls[2]) or id2relation[rel] == "hascontext"):
                continue
            # if id2relation[rel] == "relatedto" or id2relation[rel] == "antonym":
            # weight -= 0.3
            # continue
            if subj == obj:  # delete loops
                continue
            # weight = 1 + float(math.exp(1 - weight))  # issue: ???

            if (subj, obj, rel) not in attrs:
                graph.add_edge(subj, obj, rel=rel, weight=weight)
                attrs.add((subj, obj, rel))
                graph.add_edge(obj, subj, rel=rel + len(relation2id), weight=weight)
                attrs.add((obj, subj, rel + len(relation2id)))

    # nx.write_gpickle(graph, output_path)
    write_gpickle(graph, output_path)
    print(f"graph file saved to {output_path}")
    print()


def glove_init(input, output, concept_file):
    embeddings_file = output + '.npy'
    vocabulary_file = output.split('.')[0] + '.vocab.txt'
    output_dir = '/'.join(output.split('/')[:-1])
    output_prefix = output.split('/')[-1]

    words = []
    vectors = []
    vocab_exist = check_file(vocabulary_file)
    print("loading embedding")
    with open(input, 'rb') as f:
        for line in f:
            fields = line.split()
            if len(fields) <= 2:
                continue
            if not vocab_exist:
                word = fields[0].decode('utf-8')
                words.append(word)
            vector = np.fromiter((float(x) for x in fields[1:]), dtype=np.float)

            vectors.append(vector)
        dim = vector.shape[0]
    print("converting")
    matrix = np.array(vectors, dtype="float32")
    print("writing")
    np.save(embeddings_file, matrix)
    text = '\n'.join(words)
    if not vocab_exist:
        with open(vocabulary_file, 'wb') as f:
            f.write(text.encode('utf-8'))

    def load_glove_from_npy(glove_vec_path, glove_vocab_path):
        vectors = np.load(glove_vec_path)
        with open(glove_vocab_path, "r", encoding="utf8") as f:
            vocab = [l.strip() for l in f.readlines()]

        assert (len(vectors) == len(vocab))

        glove_embeddings = {}
        for i in range(0, len(vectors)):
            glove_embeddings[vocab[i]] = vectors[i]
        print("Read " + str(len(glove_embeddings)) + " glove vectors.")
        return glove_embeddings

    def weighted_average(avg, new, n):
        # TODO: maybe a better name for this function?
        return ((n - 1) / n) * avg + (new / n)

    def max_pooling(old, new):
        # TODO: maybe a better name for this function?
        return np.maximum(old, new)

    def write_embeddings_npy(embeddings, embeddings_cnt, npy_path, vocab_path):
        words = []
        vectors = []
        for key, vec in embeddings.items():
            words.append(key)
            vectors.append(vec)

        matrix = np.array(vectors, dtype="float32")
        print(matrix.shape)

        print("Writing embeddings matrix to " + npy_path, flush=True)
        np.save(npy_path, matrix)
        print("Finished writing embeddings matrix to " + npy_path, flush=True)

        if not check_file(vocab_path):
            print("Writing vocab file to " + vocab_path, flush=True)
            to_write = ["\t".join([w, str(embeddings_cnt[w])]) for w in words]
            with open(vocab_path, "w", encoding="utf8") as f:
                f.write("\n".join(to_write))
            print("Finished writing vocab file to " + vocab_path, flush=True)

    def create_embeddings_glove(pooling="max", dim=100):
        print("Pooling: " + pooling)

        with open(concept_file, "r", encoding="utf8") as f:
            triple_str_json = json.load(f)
        print("Loaded " + str(len(triple_str_json)) + " triple strings.")

        glove_embeddings = load_glove_from_npy(embeddings_file, vocabulary_file)
        print("Loaded glove.", flush=True)

        concept_embeddings = {}
        concept_embeddings_cnt = {}
        rel_embeddings = {}
        rel_embeddings_cnt = {}

        for i in tqdm(range(len(triple_str_json))):
            data = triple_str_json[i]

            words = data["string"].strip().split(" ")

            rel = data["rel"]
            subj_start = data["subj_start"]
            subj_end = data["subj_end"]
            obj_start = data["obj_start"]
            obj_end = data["obj_end"]

            subj_words = words[subj_start:subj_end]
            obj_words = words[obj_start:obj_end]

            subj = " ".join(subj_words)
            obj = " ".join(obj_words)

            # counting the frequency (only used for the avg pooling)
            if subj not in concept_embeddings:
                concept_embeddings[subj] = np.zeros((dim,))
                concept_embeddings_cnt[subj] = 0
            concept_embeddings_cnt[subj] += 1

            if obj not in concept_embeddings:
                concept_embeddings[obj] = np.zeros((dim,))
                concept_embeddings_cnt[obj] = 0
            concept_embeddings_cnt[obj] += 1

            if rel not in rel_embeddings:
                rel_embeddings[rel] = np.zeros((dim,))
                rel_embeddings_cnt[rel] = 0
            rel_embeddings_cnt[rel] += 1

            if pooling == "avg":
                subj_encoding_sum = sum([glove_embeddings.get(word, np.zeros((dim,))) for word in subj])
                obj_encoding_sum = sum([glove_embeddings.get(word, np.zeros((dim,))) for word in obj])

                if rel in ["relatedto", "antonym"]:
                    # Symmetric relation.
                    rel_encoding_sum = sum([glove_embeddings.get(word, np.zeros((dim,))) for word in words]) - subj_encoding_sum - obj_encoding_sum
                else:
                    # Asymmetrical relation.
                    rel_encoding_sum = obj_encoding_sum - subj_encoding_sum

                subj_len = subj_end - subj_start
                obj_len = obj_end - obj_start

                subj_encoding = subj_encoding_sum / subj_len
                obj_encoding = obj_encoding_sum / obj_len
                rel_encoding = rel_encoding_sum / (len(words) - subj_len - obj_len)

                concept_embeddings[subj] = subj_encoding
                concept_embeddings[obj] = obj_encoding
                rel_embeddings[rel] = weighted_average(rel_embeddings[rel], rel_encoding, rel_embeddings_cnt[rel])

            elif pooling == "max":
                subj_encoding = np.amax([glove_embeddings.get(word, np.zeros((dim,))) for word in subj_words], axis=0)
                obj_encoding = np.amax([glove_embeddings.get(word, np.zeros((dim,))) for word in obj_words], axis=0)

                mask_rel = []
                for j in range(len(words)):
                    if subj_start <= j < subj_end or obj_start <= j < obj_end:
                        continue
                    mask_rel.append(j)
                rel_vecs = [glove_embeddings.get(words[i], np.zeros((dim,))) for i in mask_rel]
                rel_encoding = np.amax(rel_vecs, axis=0)

                # here it is actually avg over max for relation
                concept_embeddings[subj] = max_pooling(concept_embeddings[subj], subj_encoding)
                concept_embeddings[obj] = max_pooling(concept_embeddings[obj], obj_encoding)
                rel_embeddings[rel] = weighted_average(rel_embeddings[rel], rel_encoding, rel_embeddings_cnt[rel])

        print(str(len(concept_embeddings)) + " concept embeddings")
        print(str(len(rel_embeddings)) + " relation embeddings")

        write_embeddings_npy(concept_embeddings, concept_embeddings_cnt, f'{output_dir}/concept.{output_prefix}.{pooling}.npy',
                             f'{output_dir}/concept.glove.{pooling}.txt')
        write_embeddings_npy(rel_embeddings, rel_embeddings_cnt, f'{output_dir}/relation.{output_prefix}.{pooling}.npy',
                             f'{output_dir}/relation.glove.{pooling}.txt')

    create_embeddings_glove(dim=dim)


# if __name__ == "__main__":
#     glove_init("../data/glove/glove.6B.200d.txt", "../data/glove/glove.200d", '../data/glove/tp_str_corpus.json')


###from utils.grounding import create_matcher_patterns, ground

In [ ]:
from multiprocessing import Pool
import spacy
from spacy.matcher import Matcher
from tqdm import tqdm
import nltk
import json
import string


__all__ = ['create_matcher_patterns', 'ground']


# the lemma of it/them/mine/.. is -PRON-

blacklist = set(["-PRON-", "actually", "likely", "possibly", "want",
                 "make", "my", "someone", "sometimes_people", "sometimes", "would", "want_to",
                 "one", "something", "sometimes", "everybody", "somebody", "could", "could_be"
                 ])


nltk.download('stopwords', quiet=True)
nltk_stopwords = nltk.corpus.stopwords.words('english')

# CHUNK_SIZE = 1

CPNET_VOCAB = None
PATTERN_PATH = None
nlp = None
matcher = None


def load_cpnet_vocab(cpnet_vocab_path):
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        cpnet_vocab = [l.strip() for l in fin]
    cpnet_vocab = [c.replace("_", " ") for c in cpnet_vocab]
    return cpnet_vocab


def create_pattern(nlp, doc, debug=False):
    pronoun_list = set(["my", "you", "it", "its", "your", "i", "he", "she", "his", "her", "they", "them", "their", "our", "we"])
    # Filtering concepts consisting of all stop words and longer than four words.
    if len(doc) >= 5 or doc[0].text in pronoun_list or doc[-1].text in pronoun_list or \
            all([(token.text in nltk_stopwords or token.lemma_ in nltk_stopwords or token.lemma_ in blacklist) for token in doc]):
        if debug:
            return False, doc.text
        return None  # ignore this concept as pattern

    pattern = []
    for token in doc:  # a doc is a concept
        pattern.append({"LEMMA": token.lemma_})
    if debug:
        return True, doc.text
    return pattern


def create_matcher_patterns(cpnet_vocab_path, output_path, debug=False):
    cpnet_vocab = load_cpnet_vocab(cpnet_vocab_path)
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])
    docs = nlp.pipe(cpnet_vocab)
    all_patterns = {}

    if debug:
        f = open("filtered_concept.txt", "w")

    for doc in tqdm(docs, total=len(cpnet_vocab)):

        pattern = create_pattern(nlp, doc, debug)
        if debug:
            if not pattern[0]:
                f.write(pattern[1] + '\n')

        if pattern is None:
            continue
        all_patterns["_".join(doc.text.split(" "))] = pattern

    print("Created " + str(len(all_patterns)) + " patterns.")
    with open(output_path, "w", encoding="utf8") as fout:
        json.dump(all_patterns, fout)
    if debug:
        f.close()


def lemmatize(nlp, concept):

    doc = nlp(concept.replace("_", " "))
    lcs = set()
    # for i in range(len(doc)):
    #     lemmas = []
    #     for j, token in enumerate(doc):
    #         if j == i:
    #             lemmas.append(token.lemma_)
    #         else:
    #             lemmas.append(token.text)
    #     lc = "_".join(lemmas)
    #     lcs.add(lc)
    lcs.add("_".join([token.lemma_ for token in doc]))  # all lemma
    return lcs


def load_matcher(nlp, pattern_path):
    with open(pattern_path, "r", encoding="utf8") as fin:
        all_patterns = json.load(fin)

    matcher = Matcher(nlp.vocab)
    for concept, pattern in all_patterns.items():
        # matcher.add(concept, None, pattern)
        matcher.add(concept, [pattern])
    return matcher


def ground_qa_pair(qa_pair):
    global nlp, matcher
    if nlp is None or matcher is None:
        nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
        # nlp.add_pipe(nlp.create_pipe('sentencizer'))
        nlp.add_pipe('sentencizer')
        matcher = load_matcher(nlp, PATTERN_PATH)

    s, a = qa_pair
    all_concepts = ground_mentioned_concepts(nlp, matcher, s, a)
    answer_concepts = ground_mentioned_concepts(nlp, matcher, a)
    question_concepts = all_concepts - answer_concepts
    if len(question_concepts) == 0:
        question_concepts = hard_ground(nlp, s, CPNET_VOCAB)  # not very possible

    if len(answer_concepts) == 0:
        answer_concepts = hard_ground(nlp, a, CPNET_VOCAB)  # some case

    # question_concepts = question_concepts -  answer_concepts
    question_concepts = sorted(list(question_concepts))
    answer_concepts = sorted(list(answer_concepts))
    return {"sent": s, "ans": a, "qc": question_concepts, "ac": answer_concepts}


def ground_mentioned_concepts(nlp, matcher, s, ans=None):

    s = s.lower()
    doc = nlp(s)
    matches = matcher(doc)

    mentioned_concepts = set()
    span_to_concepts = {}

    if ans is not None:
        ans_matcher = Matcher(nlp.vocab)
        ans_words = nlp(ans)
        # print(ans_words)
        ans_matcher.add(ans, [[{'TEXT': token.text.lower()}] for token in ans_words])

        ans_match = ans_matcher(doc)
        ans_mentions = set()
        for _, ans_start, ans_end in ans_match:
            ans_mentions.add((ans_start, ans_end))

    for match_id, start, end in matches:
        if ans is not None:
            if (start, end) in ans_mentions:
                continue

        span = doc[start:end].text  # the matched span

        # a word that appears in answer is not considered as a mention in the question
        # if len(set(span.split(" ")).intersection(set(ans.split(" ")))) > 0:
        #     continue
        original_concept = nlp.vocab.strings[match_id]
        original_concept_set = set()
        original_concept_set.add(original_concept)

        # print("span", span)
        # print("concept", original_concept)
        # print("Matched '" + span + "' to the rule '" + string_id)

        # why do you lemmatize a mention whose len == 1?

        if len(original_concept.split("_")) == 1:
            # tag = doc[start].tag_
            # if tag in ['VBN', 'VBG']:

            original_concept_set.update(lemmatize(nlp, nlp.vocab.strings[match_id]))

        if span not in span_to_concepts:
            span_to_concepts[span] = set()

        span_to_concepts[span].update(original_concept_set)

    for span, concepts in span_to_concepts.items():
        concepts_sorted = list(concepts)
        # print("span:")
        # print(span)
        # print("concept_sorted:")
        # print(concepts_sorted)
        concepts_sorted.sort(key=len)

        # mentioned_concepts.update(concepts_sorted[0:2])

        shortest = concepts_sorted[0:3]

        for c in shortest:
            if c in blacklist:
                continue

            # a set with one string like: set("like_apples")
            lcs = lemmatize(nlp, c)
            intersect = lcs.intersection(shortest)
            if len(intersect) > 0:
                mentioned_concepts.add(list(intersect)[0])
            else:
                mentioned_concepts.add(c)

        # if a mention exactly matches with a concept

        exact_match = set([concept for concept in concepts_sorted if concept.replace("_", " ").lower() == span.lower()])
        # print("exact match:")
        # print(exact_match)
        assert len(exact_match) < 2
        mentioned_concepts.update(exact_match)

    return mentioned_concepts


def hard_ground(nlp, sent, cpnet_vocab):
    sent = sent.lower()
    doc = nlp(sent)
    res = set()
    for t in doc:
        if t.lemma_ in cpnet_vocab:
            res.add(t.lemma_)
    sent = " ".join([t.text for t in doc])
    if sent in cpnet_vocab:
        res.add(sent)
    try:
        assert len(res) > 0
    except Exception:
        print(f"for {sent}, concept not found in hard grounding.")
    return res


def match_mentioned_concepts(sents, answers, num_processes):
    res = []
    with Pool(num_processes) as p:
        res = list(tqdm(p.imap(ground_qa_pair, zip(sents, answers)), total=len(sents)))
    return res


# To-do: examine prune
def prune(data, cpnet_vocab_path):
    # reload cpnet_vocab
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        cpnet_vocab = [l.strip() for l in fin]

    prune_data = []
    for item in tqdm(data):
        qc = item["qc"]
        prune_qc = []
        for c in qc:
            if c[-2:] == "er" and c[:-2] in qc:
                continue
            if c[-1:] == "e" and c[:-1] in qc:
                continue
            have_stop = False
            # remove all concepts having stopwords, including hard-grounded ones
            for t in c.split("_"):
                if t in nltk_stopwords:
                    have_stop = True
            if not have_stop and c in cpnet_vocab:
                prune_qc.append(c)

        ac = item["ac"]
        prune_ac = []
        for c in ac:
            if c[-2:] == "er" and c[:-2] in ac:
                continue
            if c[-1:] == "e" and c[:-1] in ac:
                continue
            all_stop = True
            for t in c.split("_"):
                if t not in nltk_stopwords:
                    all_stop = False
            if not all_stop and c in cpnet_vocab:
                prune_ac.append(c)

        try:
            assert len(prune_ac) > 0 and len(prune_qc) > 0
        except Exception as e:
            pass
            # print("In pruning")
            # print(prune_qc)
            # print(prune_ac)
            # print("original:")
            # print(qc)
            # print(ac)
            # print()
        item["qc"] = prune_qc
        item["ac"] = prune_ac

        prune_data.append(item)
    return prune_data


def ground(statement_path, cpnet_vocab_path, pattern_path, output_path, num_processes=1, debug=False):
    global PATTERN_PATH, CPNET_VOCAB
    if PATTERN_PATH is None:
        PATTERN_PATH = pattern_path
        CPNET_VOCAB = load_cpnet_vocab(cpnet_vocab_path)

    sents = []
    answers = []
    with open(statement_path, 'r') as fin:
        lines = [line for line in fin]

    if debug:
        lines = lines[192:195]
        print(len(lines))
    for line in lines:
        if line == "":
            continue
        j = json.loads(line)
        # {'answerKey': 'B',
        #   'id': 'b8c0a4703079cf661d7261a60a1bcbff',
        #   'question': {'question_concept': 'magazines',
        #                 'choices': [{'label': 'A', 'text': 'doctor'}, {'label': 'B', 'text': 'bookstore'}, {'label': 'C', 'text': 'market'}, {'label': 'D', 'text': 'train station'}, {'label': 'E', 'text': 'mortuary'}],
        #                 'stem': 'Where would you find magazines along side many other printed works?'},
        #   'statements': [{'label': False, 'statement': 'Doctor would you find magazines along side many other printed works.'}, {'label': True, 'statement': 'Bookstore would you find magazines along side many other printed works.'}, {'label': False, 'statement': 'Market would you find magazines along side many other printed works.'}, {'label': False, 'statement': 'Train station would you find magazines along side many other printed works.'}, {'label': False, 'statement': 'Mortuary would you find magazines along side many other printed works.'}]}

        for statement in j["statements"]:
            sents.append(statement["statement"])

        for answer in j["question"]["choices"]:
            ans = answer['text']
            # ans = " ".join(answer['text'].split("_"))
            try:
                assert all([i != "_" for i in ans])
            except Exception:
                print(ans)
            answers.append(ans)

    res = match_mentioned_concepts(sents, answers, num_processes)
    res = prune(res, cpnet_vocab_path)

    # check_path(output_path)
    with open(output_path, 'w') as fout:
        for dic in res:
            fout.write(json.dumps(dic) + '\n')

    print(f'grounded concepts saved to {output_path}')
    print()


# if __name__ == "__main__":
#     create_matcher_patterns("../data/cpnet/concept.txt", "./matcher_res.txt", True)

    # ground("../data/statement/dev.statement.jsonl", "../data/cpnet/concept.txt", "../data/cpnet/matcher_patterns.json", "./ground_res.jsonl", 10, True)

    # s = "a revolving door is convenient for two direction travel, but it also serves as a security measure at a bank."
    # a = "bank"
    # nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
    # nlp.add_pipe(nlp.create_pipe('sentencizer'))
    # ans_words = nlp(a)
    # doc = nlp(s)
    # ans_matcher = Matcher(nlp.vocab)
    # print([{'TEXT': token.text.lower()} for token in ans_words])
    # ans_matcher.add("ok", None, [{'TEXT': token.text.lower()} for token in ans_words])
    #
    # matches = ans_matcher(doc)
    # for a, b, c in matches:
    #     print(a, b, c)


###from utils.graph import generate_adj_data_from_grounded_concepts__use_LM

math

In [ ]:
concept2id = None
id2concept = None
relation2id = None
id2relation = None

cpnet = None
cpnet_all = None
cpnet_simple = None

In [ ]:
import torch
import networkx as nx
import itertools
import json
from tqdm import tqdm
# from .conceptnet import merged_relations
import numpy as np
from scipy import sparse
import pickle
from scipy.sparse import csr_matrix, coo_matrix
from multiprocessing import Pool
from collections import OrderedDict


# from .maths import *

__all__ = ['generate_graph']


def load_resources(cpnet_vocab_path):
    global concept2id, id2concept, relation2id, id2relation

    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        id2concept = [w.strip() for w in fin]
    concept2id = {w: i for i, w in enumerate(id2concept)}

    id2relation = merged_relations
    relation2id = {r: i for i, r in enumerate(id2relation)}


def load_cpnet(cpnet_graph_path):
    global cpnet, cpnet_simple
    # cpnet = nx.read_gpickle(cpnet_graph_path)
    cpnet = read_gpickle(cpnet_graph_path)
    cpnet_simple = nx.Graph()
    for u, v, data in cpnet.edges(data=True):
        w = data['weight'] if 'weight' in data else 1.0
        if cpnet_simple.has_edge(u, v):
            cpnet_simple[u][v]['weight'] += w
        else:
            cpnet_simple.add_edge(u, v, weight=w)


def relational_graph_generation(qcs, acs, paths, rels):
    raise NotImplementedError()  # TODO


# plain graph generation
def plain_graph_generation(qcs, acs, paths, rels):
    global cpnet, concept2id, relation2id, id2relation, id2concept, cpnet_simple

    graph = nx.Graph()
    for p in paths:
        for c_index in range(len(p) - 1):
            h = p[c_index]
            t = p[c_index + 1]
            # TODO: the weight can computed by concept embeddings and relation embeddings of TransE
            graph.add_edge(h, t, weight=1.0)

    for qc1, qc2 in list(itertools.combinations(qcs, 2)):
        if cpnet_simple.has_edge(qc1, qc2):
            graph.add_edge(qc1, qc2, weight=1.0)

    for ac1, ac2 in list(itertools.combinations(acs, 2)):
        if cpnet_simple.has_edge(ac1, ac2):
            graph.add_edge(ac1, ac2, weight=1.0)

    if len(qcs) == 0:
        qcs.append(-1)

    if len(acs) == 0:
        acs.append(-1)

    if len(paths) == 0:
        for qc in qcs:
            for ac in acs:
                graph.add_edge(qc, ac, rel=-1, weight=0.1)

    g = nx.convert_node_labels_to_integers(graph, label_attribute='cid')  # re-index
    return nx.node_link_data(g)


def generate_adj_matrix_per_inst(nxg_str):
    global id2relation
    n_rel = len(id2relation)

    nxg = nx.node_link_graph(json.loads(nxg_str))
    n_node = len(nxg.nodes)
    cids = np.zeros(n_node, dtype=np.int32)
    for node_id, node_attr in nxg.nodes(data=True):
        cids[node_id] = node_attr['cid']

    adj = np.zeros((n_rel, n_node, n_node), dtype=np.uint8)
    for s in range(n_node):
        for t in range(n_node):
            s_c, t_c = cids[s], cids[t]
            if cpnet_all.has_edge(s_c, t_c):
                for e_attr in cpnet_all[s_c][t_c].values():
                    if e_attr['rel'] >= 0 and e_attr['rel'] < n_rel:
                        adj[e_attr['rel']][s][t] = 1
    cids += 1
    adj = coo_matrix(adj.reshape(-1, n_node))
    return (adj, cids)


def concepts2adj(node_ids):
    global id2relation
    cids = np.array(node_ids, dtype=np.int32)
    n_rel = len(id2relation)
    n_node = cids.shape[0]
    adj = np.zeros((n_rel, n_node, n_node), dtype=np.uint8)
    for s in range(n_node):
        for t in range(n_node):
            s_c, t_c = cids[s], cids[t]
            if cpnet.has_edge(s_c, t_c):
                for e_attr in cpnet[s_c][t_c].values():
                    if e_attr['rel'] >= 0 and e_attr['rel'] < n_rel:
                        adj[e_attr['rel']][s][t] = 1
    # cids += 1  # note!!! index 0 is reserved for padding
    # try:
    adj = coo_matrix(adj.reshape(-1, n_node))
    # except:
      # print("corrupted data")
    return adj, cids


def concepts_to_adj_matrices_1hop_neighbours(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for u in set(qc_ids) | set(ac_ids):
        if u in cpnet.nodes:
            extra_nodes |= set(cpnet[u])
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask


def concepts_to_adj_matrices_1hop_neighbours_without_relatedto(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for u in set(qc_ids) | set(ac_ids):
        if u in cpnet.nodes:
            for v in cpnet[u]:
                for data in cpnet[u][v].values():
                    if data['rel'] not in (15, 32):
                        extra_nodes.add(v)
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask


def concepts_to_adj_matrices_2hop_qa_pair(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for qid in qc_ids:
        for aid in ac_ids:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask


def concepts_to_adj_matrices_2hop_all_pair(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for qid in qa_nodes:
        for aid in qa_nodes:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask


def concepts_to_adj_matrices_2step_relax_all_pair(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for qid in qc_ids:
        for aid in ac_ids:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    intermediate_ids = extra_nodes - qa_nodes
    for qid in intermediate_ids:
        for aid in ac_ids:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    for qid in qc_ids:
        for aid in intermediate_ids:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask


def concepts_to_adj_matrices_3hop_qa_pair(data):
    qc_ids, ac_ids = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for qid in qc_ids:
        for aid in ac_ids:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                for u in cpnet_simple[qid]:
                    for v in cpnet_simple[aid]:
                        if cpnet_simple.has_edge(u, v):  # ac is a 3-hop neighbour of qc
                            extra_nodes.add(u)
                            extra_nodes.add(v)
                        if u == v:  # ac is a 2-hop neighbour of qc
                            extra_nodes.add(u)
    extra_nodes = extra_nodes - qa_nodes
    schema_graph = sorted(qc_ids) + sorted(ac_ids) + sorted(extra_nodes)
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return adj, concepts, qmask, amask



######################################################################
from transformers import RobertaTokenizer, RobertaForMaskedLM

class RobertaForMaskedLMwithLoss(RobertaForMaskedLM):
    #
    def __init__(self, config):
        super().__init__(config)
    #
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, masked_lm_labels=None):
        #
        assert attention_mask is not None
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask)
        sequence_output = outputs[0] #hidden_states of final layer (batch_size, sequence_length, hidden_size)
        prediction_scores = self.lm_head(sequence_output)
        outputs = (prediction_scores, sequence_output) + outputs[2:]
        if masked_lm_labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
            bsize, seqlen = input_ids.size()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), masked_lm_labels.view(-1)).view(bsize, seqlen)
            masked_lm_loss = (masked_lm_loss * attention_mask).sum(dim=1)
            outputs = (masked_lm_loss,) + outputs
            # (masked_lm_loss), prediction_scores, sequence_output, (hidden_states), (attentions)
        return outputs

print ('loading pre-trained LM...')
TOKENIZER = RobertaTokenizer.from_pretrained('roberta-large')
LM_MODEL = RobertaForMaskedLMwithLoss.from_pretrained('roberta-large')
LM_MODEL.cuda(); LM_MODEL.eval()
print ('loading done')

def get_LM_score(cids, question):
    cids = cids[:]
    cids.insert(0, -1) #QAcontext node
    sents, scores = [], []
    for cid in cids:
        if cid==-1:
            sent = question.lower()
        else:
            sent = '{} {}.'.format(question.lower(), ' '.join(id2concept[cid].split('_')))
        sent = TOKENIZER.encode(sent, add_special_tokens=True)
        sents.append(sent)
    n_cids = len(cids)
    cur_idx = 0
    batch_size = 50
    while cur_idx < n_cids:
        #Prepare batch
        input_ids = sents[cur_idx: cur_idx+batch_size]
        max_len = max([len(seq) for seq in input_ids])
        for j, seq in enumerate(input_ids):
            seq += [TOKENIZER.pad_token_id] * (max_len-len(seq))
            input_ids[j] = seq
        input_ids = torch.tensor(input_ids).cuda() #[B, seqlen]
        mask = (input_ids!=1).long() #[B, seq_len]
        #Get LM score
        with torch.no_grad():
            outputs = LM_MODEL(input_ids, attention_mask=mask, masked_lm_labels=input_ids)
            loss = outputs[0] #[B, ]
            _scores = list(-loss.detach().cpu().numpy()) #list of float
        scores += _scores
        cur_idx += batch_size
    assert len(sents) == len(scores) == len(cids)
    cid2score = OrderedDict(sorted(list(zip(cids, scores)), key=lambda x: -x[1])) #score: from high to low
    return cid2score

def concepts_to_adj_matrices_2hop_all_pair__use_LM__Part1(data):
    qc_ids, ac_ids, question = data
    qa_nodes = set(qc_ids) | set(ac_ids)
    extra_nodes = set()
    for qid in qa_nodes:
        for aid in qa_nodes:
            if qid != aid and qid in cpnet_simple.nodes and aid in cpnet_simple.nodes:
                extra_nodes |= set(cpnet_simple[qid]) & set(cpnet_simple[aid])
    extra_nodes = extra_nodes - qa_nodes
    return (sorted(qc_ids), sorted(ac_ids), question, sorted(extra_nodes))

def concepts_to_adj_matrices_2hop_all_pair__use_LM__Part2(data):
    qc_ids, ac_ids, question, extra_nodes = data
    cid2score = get_LM_score(qc_ids+ac_ids+extra_nodes, question)
    return (qc_ids, ac_ids, question, extra_nodes, cid2score)

def concepts_to_adj_matrices_2hop_all_pair__use_LM__Part3(data):
    qc_ids, ac_ids, question, extra_nodes, cid2score = data
    schema_graph = qc_ids + ac_ids + sorted(extra_nodes, key=lambda x: -cid2score[x]) #score: from high to low
    arange = np.arange(len(schema_graph))
    qmask = arange < len(qc_ids)
    amask = (arange >= len(qc_ids)) & (arange < (len(qc_ids) + len(ac_ids)))
    adj, concepts = concepts2adj(schema_graph)
    return {'adj': adj, 'concepts': concepts, 'qmask': qmask, 'amask': amask, 'cid2score': cid2score}

################################################################################



#####################################################################################################
#                     functions below this line will be called by preprocess.py                     #
#####################################################################################################


def generate_graph(grounded_path, pruned_paths_path, cpnet_vocab_path, cpnet_graph_path, output_path):
    print(f'generating schema graphs for {grounded_path} and {pruned_paths_path}...')

    global concept2id, id2concept, relation2id, id2relation
    if any(x is None for x in [concept2id, id2concept, relation2id, id2relation]):
        load_resources(cpnet_vocab_path)

    global cpnet, cpnet_simple
    if cpnet is None or cpnet_simple is None:
        load_cpnet(cpnet_graph_path)

    nrow = sum(1 for _ in open(grounded_path, 'r'))
    with open(grounded_path, 'r') as fin_gr, \
            open(pruned_paths_path, 'r') as fin_pf, \
            open(output_path, 'w') as fout:
        for line_gr, line_pf in tqdm(zip(fin_gr, fin_pf), total=nrow):
            mcp = json.loads(line_gr)
            qa_pairs = json.loads(line_pf)

            statement_paths = []
            statement_rel_list = []
            for qas in qa_pairs:
                if qas["pf_res"] is None:
                    cur_paths = []
                    cur_rels = []
                else:
                    cur_paths = [item["path"] for item in qas["pf_res"]]
                    cur_rels = [item["rel"] for item in qas["pf_res"]]
                statement_paths.extend(cur_paths)
                statement_rel_list.extend(cur_rels)

            qcs = [concept2id[c] for c in mcp["qc"]]
            acs = [concept2id[c] for c in mcp["ac"]]

            gobj = plain_graph_generation(qcs=qcs, acs=acs,
                                          paths=statement_paths,
                                          rels=statement_rel_list)
            fout.write(json.dumps(gobj) + '\n')

    print(f'schema graphs saved to {output_path}')
    print()


def generate_adj_matrices(ori_schema_graph_path, cpnet_graph_path, cpnet_vocab_path, output_path, num_processes, num_rels=34, debug=False):
    print(f'generating adjacency matrices for {ori_schema_graph_path} and {cpnet_graph_path}...')

    global concept2id, id2concept, relation2id, id2relation
    if any(x is None for x in [concept2id, id2concept, relation2id, id2relation]):
        load_resources(cpnet_vocab_path)

    global cpnet_all
    if cpnet_all is None:
        # cpnet_all = nx.read_gpickle(cpnet_graph_path)
        cpnet_all = read_gpickle(cpnet_graph_path)

    with open(ori_schema_graph_path, 'r') as fin:
        nxg_strs = [line for line in fin]

    if debug:
        nxgs = nxgs[:1]

    with Pool(num_processes) as p:
        res = list(tqdm(p.imap(generate_adj_matrix_per_inst, nxg_strs), total=len(nxg_strs)))

    with open(output_path, 'wb') as fout:
        pickle.dump(res, fout)

    print(f'adjacency matrices saved to {output_path}')
    print()


def generate_adj_data_from_grounded_concepts(grounded_path, cpnet_graph_path, cpnet_vocab_path, output_path, num_processes):
    """
    This function will save
        (1) adjacency matrics (each in the form of a (R*N, N) coo sparse matrix)
        (2) concepts ids
        (3) qmask that specifices whether a node is a question concept
        (4) amask that specifices whether a node is a answer concept
    to the output path in python pickle format

    grounded_path: str
    cpnet_graph_path: str
    cpnet_vocab_path: str
    output_path: str
    num_processes: int
    """
    print(f'generating adj data for {grounded_path}...')

    global concept2id, id2concept, relation2id, id2relation, cpnet_simple, cpnet
    if any(x is None for x in [concept2id, id2concept, relation2id, id2relation]):
        load_resources(cpnet_vocab_path)
    if cpnet is None or cpnet_simple is None:
        load_cpnet(cpnet_graph_path)

    qa_data = []
    with open(grounded_path, 'r', encoding='utf-8') as fin:
        for line in fin:
            dic = json.loads(line)
            q_ids = set(concept2id[c] for c in dic['qc'])
            a_ids = set(concept2id[c] for c in dic['ac'])
            q_ids = q_ids - a_ids
            qa_data.append((q_ids, a_ids))

    with Pool(num_processes) as p:
        res = list(tqdm(p.imap(concepts_to_adj_matrices_2hop_all_pair, qa_data), total=len(qa_data)))

    # res is a list of tuples, each tuple consists of four elements (adj, concepts, qmask, amask)
    with open(output_path, 'wb') as fout:
        pickle.dump(res, fout)

    print(f'adj data saved to {output_path}')
    print()



def generate_adj_data_from_grounded_concepts__use_LM(grounded_path, cpnet_graph_path, cpnet_vocab_path, output_path, num_processes):
    """
    This function will save
        (1) adjacency matrics (each in the form of a (R*N, N) coo sparse matrix)
        (2) concepts ids
        (3) qmask that specifices whether a node is a question concept
        (4) amask that specifices whether a node is a answer concept
        (5) cid2score that maps a concept id to its relevance score given the QA context
    to the output path in python pickle format

    grounded_path: str
    cpnet_graph_path: str
    cpnet_vocab_path: str
    output_path: str
    num_processes: int
    """
    print(f'generating adj data for {grounded_path}...')

    global concept2id, id2concept, relation2id, id2relation, cpnet_simple, cpnet
    if any(x is None for x in [concept2id, id2concept, relation2id, id2relation]):
        load_resources(cpnet_vocab_path)
    if cpnet is None or cpnet_simple is None:
        load_cpnet(cpnet_graph_path)

    qa_data = []
    statement_path = grounded_path.replace('grounded', 'statement')
    with open(grounded_path, 'r', encoding='utf-8') as fin_ground, open(statement_path, 'r', encoding='utf-8') as fin_state:
        lines_ground = fin_ground.readlines()
        lines_state  = fin_state.readlines()
        assert len(lines_ground) % len(lines_state) == 0
        n_choices = len(lines_ground) // len(lines_state)
        for j, line in enumerate(lines_ground):
            dic = json.loads(line)
            q_ids = set(concept2id[c] for c in dic['qc'])
            a_ids = set(concept2id[c] for c in dic['ac'])
            q_ids = q_ids - a_ids
            statement_obj = json.loads(lines_state[j//n_choices])
            QAcontext = "{} {}.".format(statement_obj['question']['stem'], dic['ans'])
            qa_data.append((q_ids, a_ids, QAcontext))

    with Pool(num_processes) as p:
        res1 = list(tqdm(p.imap(concepts_to_adj_matrices_2hop_all_pair__use_LM__Part1, qa_data), total=len(qa_data)))

    return res1

    # res2 = []
    # for j, _data in enumerate(res1):
    #     if j % 100 == 0: print (j)
    #     res2.append(concepts_to_adj_matrices_2hop_all_pair__use_LM__Part2(_data))

    # with Pool(num_processes) as p:
    #     res3 = list(tqdm(p.imap(concepts_to_adj_matrices_2hop_all_pair__use_LM__Part3, res2), total=len(res2)))

    # # res is a list of responses
    # with open(output_path, 'wb') as fout:
    #     pickle.dump(res3, fout)

    # print(f'adj data saved to {output_path}')
    # print()


def do_part2(data_index_range, res1):

  res2 = []
  questions_with_problem=[]
  # for j, _data in enumerate(res1):
  for j in range(data_index_range[0], data_index_range[1]):
    _data = res1[j]
    if j % 10 == 0: print (j)
    try:
      res2.append(concepts_to_adj_matrices_2hop_all_pair__use_LM__Part2(_data))
    except:
      print(f"q index with problem: {j}")
      questions_with_problem.append(j)

  return res2, questions_with_problem


def do_part3(res2, num_processes, cpnet_graph_path, cpnet_vocab_path, output_path):
  global concept2id, id2concept, relation2id, id2relation, cpnet_simple, cpnet
  if any(x is None for x in [concept2id, id2concept, relation2id, id2relation]):
      load_resources(cpnet_vocab_path)
  if cpnet is None or cpnet_simple is None:
      load_cpnet(cpnet_graph_path)

  with Pool(num_processes) as p:
    res3 = list(tqdm(p.imap(concepts_to_adj_matrices_2hop_all_pair__use_LM__Part3, res2), total=len(res2)))

  # res is a list of responses
  with open(output_path, 'wb') as fout:
      pickle.dump(res3, fout)

  print(f'adj data saved to {output_path}')
  print()



#################### adj to sparse ####################

# def coo_to_normalized_per_inst(data):
#     adj, concepts, qm, am, max_node_num = data
#     ori_adj_len = len(concepts)
#     concepts = torch.tensor(concepts[:min(len(concepts), max_node_num)])
#     adj_len = len(concepts)
#     qm = torch.tensor(qm[:adj_len], dtype=torch.uint8)
#     am = torch.tensor(am[:adj_len], dtype=torch.uint8)
#     ij = adj.row
#     k = adj.col
#     n_node = adj.shape[1]
#     n_rel = 2 * adj.shape[0] // n_node
#     i, j = ij // n_node, ij % n_node
#     mask = (j < max_node_num) & (k < max_node_num)
#     i, j, k = i[mask], j[mask], k[mask]
#     i, j, k = np.concatenate((i, i + n_rel // 2), 0), np.concatenate((j, k), 0), np.concatenate((k, j), 0)  # add inverse relations
#     adj_list = []
#     for r in range(n_rel):
#         mask = i == r
#         ones = np.ones(mask.sum(), dtype=np.float32)
#         A = sparse.csr_matrix((ones, (k[mask], j[mask])), shape=(max_node_num, max_node_num))  # A is transposed by exchanging the order of j and k
#         adj_list.append(normalize_sparse_adj(A, 'coo'))
#     adj_list.append(sparse.identity(max_node_num, dtype=np.float32, format='coo'))
#     return ori_adj_len, adj_len, concepts, adj_list, qm, am


# def coo_to_normalized(adj_path, output_path, max_node_num, num_processes):
#     print(f'converting {adj_path} to normalized adj')

#     with open(adj_path, 'rb') as fin:
#         adj_data = pickle.load(fin)
#     data = [(adj, concepts, qmask, amask, max_node_num) for adj, concepts, qmask, amask in adj_data]

#     ori_adj_lengths = torch.zeros((len(data),), dtype=torch.int64)
#     adj_lengths = torch.zeros((len(data),), dtype=torch.int64)
#     concepts_ids = torch.zeros((len(data), max_node_num), dtype=torch.int64)
#     qmask = torch.zeros((len(data), max_node_num), dtype=torch.uint8)
#     amask = torch.zeros((len(data), max_node_num), dtype=torch.uint8)

#     adj_data = []
#     with Pool(num_processes) as p:
#         for i, (ori_adj_len, adj_len, concepts, adj_list, qm, am) in tqdm(enumerate(p.imap(coo_to_normalized_per_inst, data)), total=len(data)):
#             ori_adj_lengths[i] = ori_adj_len
#             adj_lengths[i] = adj_len
#             concepts_ids[i][:adj_len] = concepts
#             qmask[i][:adj_len] = qm
#             amask[i][:adj_len] = am
#             adj_list = [(torch.LongTensor(np.stack((adj.row, adj.col), 0)),
#                          torch.FloatTensor(adj.data)) for adj in adj_list]
#             adj_data.append(adj_list)

#     torch.save((ori_adj_lengths, adj_lengths, concepts_ids, adj_data), output_path)

#     print(f'normalized adj saved to {output_path}')
#     print()

# if __name__ == '__main__':
#     generate_adj_matrices_from_grounded_concepts('./data/csqa/grounded/train.grounded.jsonl',
#                                                  './data/cpnet/conceptnet.en.pruned.graph',
#                                                  './data/cpnet/concept.txt',
#                                                  '/tmp/asdf', 40)


loading pre-trained LM...


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

loading done


###ConceptNet Preprocessing

In [ ]:
import os
import argparse
from multiprocessing import cpu_count
# from utils.convert_csqa import convert_to_entailment
# from utils.convert_obqa import convert_to_obqa_statement
# from utils.conceptnet import extract_english, construct_graph
# from utils.grounding import create_matcher_patterns, ground
# from utils.graph import generate_adj_data_from_grounded_concepts__use_LM

input_paths = {
    'cpnet': {
        'csv': './data/cpnet/conceptnet-assertions-5.6.0.csv',
    },
    'csqa': {
        'train': './data/csqa/train_rand_split.jsonl',
        'dev': './data/csqa/dev_rand_split.jsonl',
        'test': './data/csqa/test_rand_split_no_answers.jsonl',
    },
    'riddle_sense':{
        'train': '/content/drive/MyDrive/brain_teaser/datasets/rs_train_preprocessed_pruned.jsonl',
        'dev': '/content/drive/MyDrive/brain_teaser/datasets/rs_dev_preprocessed_pruned.jsonl',
    },
    'brain_teaser_ds':{
        'SP_train': '/content/drive/MyDrive/brain_teaser/datasets/SP_train_preprocessed_pruned.jsonl',
        'SP_dev': '/content/drive/MyDrive/brain_teaser/datasets/SP_dev_preprocessed_pruned.jsonl',
        'WP_train': '/content/drive/MyDrive/brain_teaser/datasets/WP_train_preprocessed_pruned.jsonl',
        'WP_dev': '/content/drive/MyDrive/brain_teaser/datasets/WP_dev_preprocessed_pruned.jsonl',
    }
}

output_paths = {
    'cpnet': {
        'csv': './data/cpnet/conceptnet.en.csv',
        'vocab': './data/cpnet/concept.txt',
        'patterns': './data/cpnet/matcher_patterns.json',
        'unpruned-graph': './data/cpnet/conceptnet.en.unpruned.graph',
        'pruned-graph': './data/cpnet/conceptnet.en.pruned.graph',
    },
    'csqa': {
        'statement': {
            'train': './data/csqa/statement/train.statement.jsonl',
            'dev': './data/csqa/statement/dev.statement.jsonl',
            'test': './data/csqa/statement/test.statement.jsonl',
        },
        'grounded': {
            'train': './data/csqa/grounded/train.grounded.jsonl',
            'dev': './data/csqa/grounded/dev.grounded.jsonl',
            'test': './data/csqa/grounded/test.grounded.jsonl',
        },
        'graph': {
            'adj-train': './data/csqa/graph/train.graph.adj.pk',
            'adj-dev': './data/csqa/graph/dev.graph.adj.pk',
            'adj-test': './data/csqa/graph/test.graph.adj.pk',
        },
    },
    'riddle_sense': {
        'statement': {
            'train': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl',
            'dev': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/dev.statement.jsonl',
            # 'test': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/test.statement.jsonl',
        },
        'grounded': {
            'train': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/train.grounded.jsonl',
            'dev': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/dev.grounded.jsonl',
            # 'test': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/test.grounded.jsonl',
        },
        'graph': {
            'adj-train': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/train.graph.adj.pk',
            'adj-dev': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/dev.graph.adj.pk',
            # 'adj-test': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/test.graph.adj.pk',
        },
    },
    'brain_teaser_ds':{
         'statement': {
            'SP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_train.statement.jsonl',
            'SP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_dev.statement.jsonl',
            'WP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl',
            'WP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_dev.statement.jsonl',
            # 'test': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/test.statement.jsonl',
        },
        'grounded': {
            'SP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_train.grounded.jsonl',
            'SP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_dev.grounded.jsonl',
            'WP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_train.grounded.jsonl',
            'WP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_dev.grounded.jsonl',
            # 'test': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/test.grounded.jsonl',
        },
        'graph': {
            'adj-SP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_train.graph.adj.pk',
            'adj-SP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_dev.graph.adj.pk',
            'adj-WP_train': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_train.graph.adj.pk',
            'adj-WP_dev': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_dev.graph.adj.pk',
            # 'adj-test': '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/test.graph.adj.pk',
        },
    }

}



def preprocess_func(args, routines):

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


In [ ]:
import locale
def getlocale():
  return "UTF-8"
locale.getpreferredencoding=getlocale

In [ ]:
# routines = {
#     'common': [
#         {'func': extract_english, 'args': (input_paths['cpnet']['csv'], output_paths['cpnet']['csv'], output_paths['cpnet']['vocab'])},
#         {'func': construct_graph, 'args': (output_paths['cpnet']['csv'], output_paths['cpnet']['vocab'],
#                                             output_paths['cpnet']['unpruned-graph'], False)},
#         {'func': construct_graph, 'args': (output_paths['cpnet']['csv'], output_paths['cpnet']['vocab'],
#                                             output_paths['cpnet']['pruned-graph'], True)},
#         {'func': create_matcher_patterns, 'args': (output_paths['cpnet']['vocab'], output_paths['cpnet']['patterns'])},
#     ],
# }


# args=dict()
# args["run"]=["common"]
# args["path_prune_threshold"]=0.12
# args["max_node_num"]=200
# args["nprocs"]=cpu_count()
# args["seed"]=0
# args["debug"]=False

# preprocess_func(args, routines)

##CSQA and Riddle-sense preprocessing

### CSQA Preprocessing

<h3>Run convert_to_entailment func</h3>

In [ ]:
args=dict()
args["run"]=["csqa"]
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

In [ ]:
routines = {
    'csqa': [
        {'func': convert_to_entailment, 'args': (input_paths['csqa']['train'], output_paths['csqa']['statement']['train'])},
        {'func': convert_to_entailment, 'args': (input_paths['csqa']['dev'], output_paths['csqa']['statement']['dev'])},
        {'func': convert_to_entailment, 'args': (input_paths['csqa']['test'], output_paths['csqa']['statement']['test'])},
    ],
}
preprocess_func(args, routines)

<h3>Run ground func</h3>

In [ ]:
routines = {
    'csqa': [
        {'func': ground, 'args': (output_paths['csqa']['statement']['train'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['csqa']['grounded']['train'], args["nprocs"])},
    ],
}
preprocess_func(args, routines)

In [ ]:
routines = {
    'csqa': [
        {'func': ground, 'args': (output_paths['csqa']['statement']['dev'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['csqa']['grounded']['dev'], args["nprocs"])},
    ],
}
preprocess_func(args, routines)

In [ ]:
routines = {
      'csqa': [
          {'func': ground, 'args': (output_paths['csqa']['statement']['test'], output_paths['cpnet']['vocab'],
                                    output_paths['cpnet']['patterns'], output_paths['csqa']['grounded']['test'], args["nprocs"])},
      ],
  }
preprocess_func(args, routines)

csqa
{'func': <function ground at 0x7f9ede204dc0>, 'args': ('./data/csqa/statement/test.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', './data/csqa/grounded/test.grounded.jsonl', 2)}


 25%|██▌       | 1444/5700 [15:39<27:35,  2.57it/s]

for mzzleloader, concept not found in hard grounding.


 31%|███       | 1750/5700 [18:27<53:09,  1.24it/s]

for piramid, concept not found in hard grounding.


...

for accomadable, concept not found in hard grounding.


 96%|█████████▌| 5449/5700 [51:41<02:56,  1.42it/s]

for stds, concept not found in hard grounding.


100%|██████████| 5700/5700 [01:19<00:00, 71.87it/s]

grounded concepts saved to ./data/csqa/grounded/test.grounded.jsonl

Successfully run csqa


<h3>Run generate_adj_data_from_grounded_concepts__use_LM func</h3>

In [ ]:
routines = {
        'csqa': [
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['csqa']['grounded']['train'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['csqa']['graph']['adj-train'], args["nprocs"])},
        ],
    }
preprocess_func(args, routines)

In [ ]:
routines = {
        'csqa': [
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['csqa']['grounded']['dev'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['csqa']['graph']['adj-dev'], args["nprocs"])},
        ],
    }
preprocess_func(args, routines)

In [ ]:
routines = {
    'csqa': [
        {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['csqa']['grounded']['test'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['csqa']['graph']['adj-test'], args["nprocs"])},
    ],
}
preprocess_func(args, routines)

###Riddle_Sense Preprocessing

####convert_to_entailment

In [ ]:
routines = {
    'riddle_sense': [
        {'func': convert_to_entailment, 'args': (input_paths['riddle_sense']['dev'], output_paths['riddle_sense']['statement']['dev'])},
    ],
}

args=dict()
# args["run"]=["common"]
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
preprocess_func(args, routines)

riddle_sense
{'func': <function convert_to_entailment at 0x7ebc1c77a050>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/rs_dev_preprocessed_pruned.jsonl', '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/dev.statement.jsonl')}
converting /content/drive/MyDrive/brain_teaser/datasets/rs_dev_preprocessed_pruned.jsonl to entailment dataset...


100%|██████████| 1021/1021 [00:00<00:00, 6567.70it/s]

converted statements saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/dev.statement.jsonl

Successfully run riddle_sense


In [ ]:
routines = {
    'riddle_sense': [
        {'func': convert_to_entailment, 'args': (input_paths['riddle_sense']['train'], output_paths['riddle_sense']['statement']['train'])},
    ],
}

args=dict()
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
preprocess_func(args, routines)

riddle_sense
{'func': <function convert_to_entailment at 0x7ebc1c77a050>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/rs_train_preprocessed_pruned.jsonl', '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl')}
converting /content/drive/MyDrive/brain_teaser/datasets/rs_train_preprocessed_pruned.jsonl to entailment dataset...


100%|██████████| 3510/3510 [00:00<00:00, 6188.64it/s]

converted statements saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl

Successfully run riddle_sense


####ground

In [ ]:
routines = {
    'riddle_sense': [
        {'func': ground, 'args': (output_paths['riddle_sense']['statement']['dev'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['riddle_sense']['grounded']['dev'], args["nprocs"])},
    ],
}

args=dict()
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
preprocess_func(args, routines)

riddle_sense
{'func': <function ground at 0x7eba779cdd80>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/dev.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/dev.grounded.jsonl', 2)}


 14%|█▎        | 701/5105 [09:43<51:58,  1.41it/s]  

for xray, concept not found in hard grounding.


 15%|█▍        | 764/5105 [10:11<29:21,  2.46it/s]

for i.q, concept not found in hard grounding.


...

for 21, concept not found in hard grounding.


 98%|█████████▊| 5014/5105 [1:05:01<01:47,  1.18s/it]

for sitan, concept not found in hard grounding.


100%|██████████| 5105/5105 [01:19<00:00, 64.20it/s]


grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/dev.grounded.jsonl

Successfully run riddle_sense


####generate_adj_data_from_grounded_concepts__use_LM

In [ ]:
# routines = {
#         'riddle_sense': [
#             {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['riddle_sense']['grounded']['dev'],
#                   output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-dev'], args["nprocs"])},
#         ],
#     }
# preprocess_func(args, routines)
args=dict()
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['riddle_sense']['grounded']['dev'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-dev'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/dev.grounded.jsonl...


100%|██████████| 5105/5105 [00:32<00:00, 156.36it/s]


#####do part2

In [ ]:
data_range_to_process=[0,1000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
100
200
300
400
500
600
700
800
900


In [ ]:
%%time
data_range_to_process=[1000,2000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [ ]:
%%time
data_range_to_process=[2000, 3000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
CPU times: user 37min 5s, sys: 3.15 s, total: 37min 8s
Wall time: 37min 57s


In [ ]:
%%time
data_range_to_process=[3000, 4000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
CPU times: user 46min 37s, sys: 3.92 s, total: 46min 41s
Wall time: 47min 12s


In [ ]:
%%time
data_range_to_process=[4000, 5105]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
CPU times: user 43min 37s, sys: 4.22 s, total: 43min 42s
Wall time: 44min 2s


#####combine parts

In [ ]:
args=dict()
# args["run"]=["common"]
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

In [ ]:
temp_res2=[]
i=1000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

1000

In [ ]:
i=2000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

2000

In [ ]:
i=3000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

3000

In [ ]:
i=4000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

4000

In [ ]:
i=5105
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

5105

#####do part 3

In [ ]:
do_part3(temp_res2, args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-dev'])

100%|██████████| 5105/5105 [18:37<00:00,  4.57it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/dev.graph.adj.pk



####train data

#####ground

In [ ]:
args=dict()
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

routines = {
    'riddle_sense': [
        {'func': ground, 'args': (output_paths['riddle_sense']['statement']['train'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['riddle_sense']['grounded']['train'], args["nprocs"])},
    ],
}


preprocess_func(args, routines)

riddle_sense
{'func': <function ground at 0x79f3d0f46680>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/train.grounded.jsonl', 2)}
nothing n_n
ice_cream
towel_


  2%|▏         | 333/17550 [06:50<5:07:07,  1.07s/it]

for ong, concept not found in hard grounding.


  3%|▎         | 569/17550 [10:08<4:45:07,  1.01s/it]

for 20, concept not found in hard grounding.


...

for 11111100, concept not found in hard grounding.


100%|█████████▉| 17492/17550 [3:33:11<00:15,  3.69it/s]

for batmreturns, concept not found in hard grounding.


100%|██████████| 17550/17550 [04:12<00:00, 69.59it/s]


grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/train.grounded.jsonl

Successfully run riddle_sense


#####generate_adj_data_from_grounded_concepts__use_LM

In [ ]:
# routines = {
#         'riddle_sense': [
#             {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['riddle_sense']['grounded']['dev'],
#                   output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-dev'], args["nprocs"])},
#         ],
#     }
# preprocess_func(args, routines)
args=dict()
args["run"]=['riddle_sense']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['riddle_sense']['grounded']['train'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-train'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/grounded/train.grounded.jsonl...


100%|██████████| 17550/17550 [01:40<00:00, 174.30it/s]


#####do part 2

In [ ]:
%%time
data_range_to_process=[0,1000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
100
200
300
400
500
600
700
800
900
CPU times: user 32min 26s, sys: 3.11 s, total: 32min 29s
Wall time: 32min 47s


In [ ]:
%%time
data_range_to_process=[1000, 2000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
CPU times: user 39min 51s, sys: 3.07 s, total: 39min 54s
Wall time: 40min 8s


In [ ]:
%%time
data_range_to_process=[2000, 3000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
CPU times: user 21min 18s, sys: 1.6 s, total: 21min 20s
Wall time: 21min 25s


In [ ]:
%%time
data_range_to_process=[3000, 4000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
CPU times: user 37min 8s, sys: 2.95 s, total: 37min 11s
Wall time: 37min 27s


In [ ]:
%%time
data_range_to_process=[4000, 5000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
CPU times: user 45min 40s, sys: 3.37 s, total: 45min 44s
Wall time: 45min 54s


In [ ]:
%%time
data_range_to_process=[5000, 6000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
CPU times: user 48min 20s, sys: 3.03 s, total: 48min 23s
Wall time: 48min 32s


In [ ]:
%%time
data_range_to_process=[6000, 7000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

6000
6100
6200
6300
6400
6500


Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


q index with problem: 6560
q index with problem: 6561
q index with problem: 6562
q index with problem: 6563
q index with problem: 6564
6600
6700
6800
6900
CPU times: user 34min 25s, sys: 3.18 s, total: 34min 28s
Wall time: 35min 15s


In [ ]:
%%time
data_range_to_process=[7000, 8000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
CPU times: user 47min 45s, sys: 3.56 s, total: 47min 48s
Wall time: 48min 12s


In [ ]:
%%time
data_range_to_process=[8000, 9000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
CPU times: user 35min 59s, sys: 2.45 s, total: 36min 1s
Wall time: 36min 7s


In [ ]:
%%time
data_range_to_process=[9000, 10000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
CPU times: user 40min 26s, sys: 2.83 s, total: 40min 29s
Wall time: 40min 51s


In [ ]:
%%time
data_range_to_process=[10000, 11000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
CPU times: user 28min 52s, sys: 2.14 s, total: 28min 54s
Wall time: 29min 11s


In [ ]:
%%time
data_range_to_process=[11000, 12000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
CPU times: user 25min 57s, sys: 1.87 s, total: 25min 59s
Wall time: 26min 17s


In [ ]:
%%time
data_range_to_process=[12000, 13000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
CPU times: user 31min 49s, sys: 1.98 s, total: 31min 51s
Wall time: 31min 57s


In [ ]:
%%time
data_range_to_process=[13000, 14000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
CPU times: user 28min 7s, sys: 2.57 s, total: 28min 10s
Wall time: 28min 19s


In [ ]:
%%time
data_range_to_process=[14000, 15000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
CPU times: user 24min 34s, sys: 1.94 s, total: 24min 36s
Wall time: 24min 42s


In [ ]:
%%time
data_range_to_process=[15000, 16000]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
CPU times: user 28min 5s, sys: 2.17 s, total: 28min 7s
Wall time: 28min 14s


In [ ]:
%%time
data_range_to_process=[16000, 17550]
res2, questions_with_problem=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
CPU times: user 16min 23s, sys: 1.26 s, total: 16min 24s
Wall time: 16min 28s


#####Combine parts

In [ ]:
temp_res2=[]
i=1000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

1000

In [ ]:
i=2000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

2000

In [ ]:
i=3000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

3000

In [ ]:
i=4000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

4000

In [ ]:
i=5000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

5000

In [ ]:
i=6000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

6000

In [ ]:
i=7000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

6995

In [ ]:
i=8000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

7995

In [ ]:
i=9000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

8995

In [ ]:
i=10000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

9995

In [ ]:
i=11000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

10995

In [ ]:
i=12000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

11995

In [ ]:
i=13000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

12995

In [ ]:
i=14000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

13995

In [ ]:
i=15000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

14995

In [ ]:
i=16000
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

15995

In [ ]:
i=17550
path=f"/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp)
len(temp_res2)

17545

#####do part3

In [ ]:
temp_res2[6160]

([],
 [],
 '9+9=9 how can that be? 9918 189.',
 [],
 OrderedDict([(-1, -5.411866)]))

In [ ]:
do_part3(temp_res2, args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-train'])

 35%|███▌      | 6154/17545 [18:39<31:14,  6.08it/s]

corrupted data


 46%|████▌     | 8069/17545 [25:08<18:53,  8.36it/s]

corrupted data


 47%|████▋     | 8259/17545 [25:41<14:13, 10.88it/s]

...

corrupted data


 98%|█████████▊| 17141/17545 [45:03<00:29, 13.83it/s]

corrupted data


100%|██████████| 17545/17545 [45:17<00:00,  6.46it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/train.graph.adj.pk



In [ ]:
# routines = {
#         'riddle_sense': [
#             {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['riddle_sense']['grounded']['train'],
#                   output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['riddle_sense']['graph']['adj-train'], args["nprocs"])},
#         ],
#     }
# preprocess_func(routines)

##<font color=lightgreen>Extract concepts from questions and answer choices</font>

In [ ]:
import nltk
from tqdm import tqdm
import json
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

###riddle-sense dataset

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"


linked_qa_file=f"{DATA_FOLDER}/rs_train.jsonl"

with open(linked_qa_file) as f:
  rs_train_instances = [json.loads(line) for line in f.read().split("\n") if line]

linked_qa_file=f"{DATA_FOLDER}/rs_dev.jsonl"

with open(linked_qa_file) as f:
  rs_dev_instances = [json.loads(line) for line in f.read().split("\n") if line]

print(f"Riddle-Sense train size: {len(rs_train_instances)}, Riddle-Sense dev size: {len(rs_dev_instances)}")

Riddle-Sense train size: 3510, Riddle-Sense dev size: 1021


In [ ]:
rs_train_instances[0]

{'id': 'cxzvpoiuzckf-3123',
 'question': {'stem': 'A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?',
  'choices': [{'label': 'A', 'text': 'throw'},
   {'label': 'B', 'text': 'bit'},
   {'label': 'C', 'text': 'gallon'},
   {'label': 'D', 'text': 'mouse'},
   {'label': 'E', 'text': 'hole'}]},
 'answerKey': 'E'}

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"
concepts_path=f"{DATA_FOLDER}/concept.txt"

def load_concept_vocab(cpnet_vocab_path):
    global concept2id, id2concept

    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        id2concept = [w.strip() for w in fin]
    concept2id = {w: i for i, w in enumerate(id2concept)}


load_concept_vocab(concepts_path)
print(len(id2concept))

799273


In [ ]:
def extract_concepts(question_list):
  processed_qs=[]

  for i in tqdm(range(len(question_list))):

    sample=question_list[i]
    q_id=sample['id']
    q_answerKey=sample['answerKey']
    q_stem=sample['question']['stem'].strip().lower()
    q_choices=sample['question']['choices']

    question_concepts=[]
    for i in range(len(id2concept)):
      concept=id2concept[i].strip().lower().replace("_"," ")
      if len(concept)<=2:
        continue
      if concept in q_stem:
        question_concepts.append(concept)


    new_question_concepts=[]
    for i in range(len(question_concepts)):
      first_concept=question_concepts[i]
      is_sub_concept=False
      for j in range(len(question_concepts)):
        if i==j:
          continue
        if first_concept in question_concepts[j]:
          is_sub_concept=True
          break

      if is_sub_concept==False:
        new_question_concepts.append(first_concept)


    meaningful_question_concepts=set()
    for concept in new_question_concepts:
      if concept in wn.words():
        meaningful_question_concepts.add(concept)
        continue

      concept_subwords=concept.split(" ")
      all_are_meaningful=True
      meaningfuls=[]
      for subword in concept_subwords:
        if len(subword)<=2:
          all_are_meaningful=False
          continue

        if subword in wn.words():
          meaningfuls.append(subword)
        else:
          all_are_meaningful=False
      if all_are_meaningful:
        meaningful_question_concepts.add(concept)
      else:
        for c in meaningfuls:
          meaningful_question_concepts.add(c)


    q_dict={
        'id':q_id,
        'answerKey':q_answerKey,
        'question':{
            'question_concept': list(meaningful_question_concepts),
            'choices': q_choices,
            'stem': q_stem,
        }
    }
    processed_qs.append(q_dict)

  return processed_qs

In [ ]:
processed_dev_qs=extract_concepts(rs_dev_instances)

100%|██████████| 1021/1021 [11:13<00:00,  1.52it/s]


In [ ]:
len(processed_dev_qs)

1021

In [ ]:
with open(f'{DATA_FOLDER}/rs_dev_preprocessed.jsonl', 'w') as outfile:
  for entry in processed_dev_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
processed_train_qs=extract_concepts(rs_train_instances)

100%|██████████| 3510/3510 [41:29<00:00,  1.41it/s]


In [ ]:
len(processed_train_qs)

3510

In [ ]:
with open(f'{DATA_FOLDER}/rs_train_preprocessed.jsonl', 'w') as outfile:
  for entry in processed_train_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
extract_concepts([rs_train_instances[1000]])

100%|██████████| 1/1 [00:04<00:00,  4.15s/it]


[{'id': 'cxzvpoiuzckf-902',
  'answerKey': 'C',
  'question': {'question_concept': ['more',
    'frozen',
    'people',
    'time',
    'explorer',
    'closely',
    'antarctica',
    'some',
    'adam',
    'know'],
   'choices': [{'label': 'A', 'text': 'cold water'},
    {'label': 'B', 'text': 'feel cold'},
    {'label': 'C', 'text': 'they have no belly button'},
    {'label': 'D', 'text': 'cooler'},
    {'label': 'E', 'text': 'bone'}],
   'stem': 'an explorer in antarctica is examining some of his ice samples and during this time, he finds two frozen people.  as examines them more closely he realizes that these people are adam and eve.  how does he know this?'}}]

In [ ]:
rs_train_instances[0]

{'id': 'cxzvpoiuzckf-3123',
 'question': {'stem': 'A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?',
  'choices': [{'label': 'A', 'text': 'throw'},
   {'label': 'B', 'text': 'bit'},
   {'label': 'C', 'text': 'gallon'},
   {'label': 'D', 'text': 'mouse'},
   {'label': 'E', 'text': 'hole'}]},
 'answerKey': 'E'}

In [ ]:
processed_train_qs[1000]

{'id': 'cxzvpoiuzckf-902',
 'answerKey': 'C',
 'question': {'question_concept': ['closely',
   'antarctica',
   'are',
   'know',
   'some',
   'eve',
   'more',
   'people',
   'ice',
   'time',
   'explorer',
   'two',
   'adam',
   'frozen'],
  'choices': [{'label': 'A', 'text': 'cold water'},
   {'label': 'B', 'text': 'feel cold'},
   {'label': 'C', 'text': 'they have no belly button'},
   {'label': 'D', 'text': 'cooler'},
   {'label': 'E', 'text': 'bone'}],
  'stem': 'an explorer in antarctica is examining some of his ice samples and during this time, he finds two frozen people.  as examines them more closely he realizes that these people are adam and eve.  how does he know this?'}}

In [ ]:
"""
from the list of concepts below, select three concepts in the following question that helps get to the answer:
[list]: ['closely',
   'antarctica',
   'are',
   'know',
   'some',
   'eve',
   'more',
   'people',
   'ice',
   'time',
   'explorer',
   'two',
   'adam',
   'frozen']
[question]: an explorer in antarctica is examining some of his ice samples and during this time, he finds two frozen people.
as examines them more closely he realizes that these people are adam and eve.  how does he know this?"""

###brain-teaser dataset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
SP_dataset_train = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP-train.npy', allow_pickle=True,encoding='bytes')
print(len(SP_dataset_train))

data_dict = {'id':[],
             'question':[],
             'answer':[],
             'distractor1':[],
             'distractor2':[],
             'distractor(unsure)':[],
             'label':[],
             'choice_list':[],
             'choice_order':[]}
for i in SP_dataset_train:
  for k,v in i.items():
    data_dict[k].append(v)

SP_train_df = pd.DataFrame(data_dict, columns=['id', 'question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)', 'label', 'choice_list', 'choice_order'])
SP_train_df.head(2)

507


,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order
0,SP-0,Mr. and Mrs. Mustard have six daughters and ea...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,1,[Some daughters get married and have their own...,"[1, 0, 2, 3]"
1,SP-0_SR,The six daughters of Mr. and Mrs. Mustard each...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,None of above.,2,[Some brothers were not loved by family and mo...,"[2, 1, 0, 3]"


In [ ]:
WP_dataset_train = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP-train.npy', allow_pickle=True,encoding='bytes')
print(len(WP_dataset_train))

data_dict = {'id':[],
             'question':[],
             'answer':[],
             'distractor1':[],
             'distractor2':[],
             'distractor(unsure)':[],
             'label':[],
             'choice_list':[],
             'choice_order':[]}
for i in WP_dataset_train:
  for k,v in i.items():
    data_dict[k].append(v)

WP_train_df = pd.DataFrame(data_dict, columns=['id', 'question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)', 'label', 'choice_list', 'choice_order'])
WP_train_df.head(2)

396


,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order
0,WP-0,How do you spell COW in thirteen letters?,SEE O DOUBLE YOU.,COWCOWCOWCOWW,SEE OH DEREFORD,None of above.,1,"[SEE OH DEREFORD, SEE O DOUBLE YOU., COWCOWCOW...","[2, 0, 1, 3]"
1,WP-0_SR,"In thirteen letters, how do you spell COW?",SEE O DOUBLE YOU.,COWCOWCOWCOWW,SEE OH DEREFORD,None of above.,2,"[SEE OH DEREFORD, COWCOWCOWCOWW, SEE O DOUBLE ...","[2, 1, 0, 3]"


In [ ]:
SP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
print(len(SP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in SP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

SP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
SP_val_df.head(2)

120


,question,choice_list
0,"Everyone called him ""Batman,"" but he knew noth...","[He tries to be friendly., He is afraid others..."
1,"All of Mrs. Smith pets are dogs except one, an...",[Mrs.Smith has one additional pet that is neit...


In [ ]:
WP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
print(len(WP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in WP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

WP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
WP_val_df.head(2)

120


,question,choice_list
0,What kind of nut has no shell?,"[A peanut., A Doughnut., A walnut., None of ab..."
1,Which nut doesn't have a shell?,"[A Doughnut., A walnut., A peanut., None of ab..."


In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
nlp.add_pipe('sentencizer')

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"
concepts_path=f"{DATA_FOLDER}/concept.txt"

# def load_cpnet_vocab(cpnet_vocab_path):
#     with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
#         cpnet_vocab = [l.strip() for l in fin]
#     cpnet_vocab = [c.replace("_", " ") for c in cpnet_vocab]
#     return cpnet_vocab

def load_concept_vocab(cpnet_vocab_path):
    global concept2id, id2concept

    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        id2concept = [w.strip() for w in fin]
    concept2id = {w: i for i, w in enumerate(id2concept)}


load_concept_vocab(concepts_path)
print(len(id2concept))

799273


In [ ]:
concept2id['none_of']

5873

In [ ]:
'hello' in concept2id

True

In [ ]:
# def hard_ground(nlp, sent, cpnet_vocab):
#     sent = sent.lower()
#     doc = nlp(sent)
#     res = set()
#     for t in doc:
#         if t.lemma_ in cpnet_vocab:
#             res.add(t.lemma_)
#     sent = " ".join([t.text for t in doc])
#     if sent in cpnet_vocab:
#         res.add(sent)
#     try:
#         assert len(res) > 0
#     except Exception:
#         print(f"for {sent}, concept not found in hard grounding.")
#     return res

In [ ]:
def extract_concepts(q_stem):

  sent = q_stem.lower()
  doc = nlp(sent)
  question_concepts = set()
  for t in doc:
    if t.lemma_ in concept2id:
      question_concepts.add(t.lemma_)
  sent = " ".join([t.text for t in doc])
  if sent in concept2id:
    question_concepts.add(sent)
  try:
    assert len(question_concepts) > 0
  except Exception:
      print(f"for {sent}, concept not found in hard grounding.")

  # new_question_concepts=[]
  # for i in range(len(question_concepts)):
  #   first_concept=question_concepts[i]
  #   is_sub_concept=False
  #   for j in range(len(question_concepts)):
  #     if i==j:
  #       continue
  #     if first_concept in question_concepts[j]:
  #       is_sub_concept=True
  #       break

  #   if is_sub_concept==False:
  #     new_question_concepts.append(first_concept)


  # meaningful_question_concepts=set()
  # for concept in new_question_concepts:
  #   if concept in wn.words():
  #     meaningful_question_concepts.add(concept)
  #     continue

  #   concept_subwords=concept.split(" ")
  #   all_are_meaningful=True
  #   meaningfuls=[]
  #   for subword in concept_subwords:
  #     if len(subword)<=2:
  #       all_are_meaningful=False
  #       continue

  #     if subword in wn.words():
  #       meaningfuls.append(subword)
  #     else:
  #       all_are_meaningful=False
  #   if all_are_meaningful:
  #     meaningful_question_concepts.add(concept)
  #   else:
  #     for c in meaningfuls:
  #       meaningful_question_concepts.add(c)

  return list(question_concepts)


def preprocess_qa(question_list, is_train_data):

  processed_qs=[]
  id_to_label={0:"A", 1:"B", 2:"C", 3:"D"}

  for i in tqdm(range(len(question_list))):

    sample=question_list.iloc[i]
    q_stem=sample['question'].strip().lower()
    q_choices_list=sample['choice_list']
    if is_train_data:
      q_answerKey=id_to_label[sample['label']]
      q_id=sample['id']
    else:
      q_id=i


    meaningful_question_concepts = extract_concepts(q_stem)

    q_choices=[]

    for j in range(len(q_choices_list)):
      if j==0:
        choice_label="A"
      elif j==1:
        choice_label="B"
      elif j==2:
        choice_label="C"
      else:
        choice_label="D"

      choice_text=q_choices_list[j]
      choice_dict={
          'label': choice_label,
          'text': choice_text.strip().lower()
      }
      meaningful_choice_concepts = extract_concepts(choice_dict['text'])

      choice_dict['choice_concepts']= meaningful_choice_concepts
      q_choices.append(choice_dict)

    if is_train_data:
      q_dict={
            'id':q_id,
            'answerKey':q_answerKey,
            'question':{
                'question_concept': meaningful_question_concepts,
                'choices': q_choices,
                'stem': q_stem,
            }
        }
    else:
        q_dict={
            'id':q_id,
            'question':{
                'question_concept': meaningful_question_concepts,
                'choices': q_choices,
                'stem': q_stem,
            }
        }
    processed_qs.append(q_dict)

  return processed_qs

In [ ]:
WP_val_df.head(1)

,question,choice_list
0,What kind of nut has no shell?,"[A peanut., A Doughnut., A walnut., None of ab..."


In [ ]:
# processed_dev_qs=preprocess_qa(SP_val_df, is_train_data=False)
processed_dev_qs=preprocess_qa(WP_val_df, is_train_data=False)

 23%|██▎       | 28/120 [00:00<00:02, 36.73it/s]

for abbit ., concept not found in hard grounding.
for abbit ., concept not found in hard grounding.
for smie ., concept not found in hard grounding.


 30%|███       | 36/120 [00:01<00:02, 33.57it/s]

for cowcowcowcoww, concept not found in hard grounding.
for cowcowcowcoww, concept not found in hard grounding.
for cobcobb, concept not found in hard grounding.


 62%|██████▎   | 75/120 [00:02<00:01, 41.86it/s]

for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.
for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.
for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.


 88%|████████▊ | 105/120 [00:02<00:00, 43.40it/s]

for 28.0, concept not found in hard grounding.
for 30.0, concept not found in hard grounding.


100%|██████████| 120/120 [00:03<00:00, 39.47it/s]


In [ ]:
processed_dev_qs[0]

{'id': 0,
 'question': {'question_concept': ['kind',
   'no',
   'of',
   'have',
   'shell',
   'what',
   'nut'],
  'choices': [{'label': 'A',
    'text': 'a peanut.',
    'choice_concepts': ['peanut', 'a']},
   {'label': 'B', 'text': 'a doughnut.', 'choice_concepts': ['doughnut', 'a']},
   {'label': 'C', 'text': 'a walnut.', 'choice_concepts': ['walnut', 'a']},
   {'label': 'D',
    'text': 'none of above.',
    'choice_concepts': ['above', 'none', 'of']}],
  'stem': 'what kind of nut has no shell?'}}

In [ ]:
# with open(f'{DATA_FOLDER}/SP_dev_preprocessed.jsonl', 'w') as outfile:
with open(f'{DATA_FOLDER}/WP_dev_preprocessed2.jsonl', 'w') as outfile:
  for entry in processed_dev_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
WP_train_df.head(1)

,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order
0,WP-0,How do you spell COW in thirteen letters?,SEE O DOUBLE YOU.,COWCOWCOWCOWW,SEE OH DEREFORD,None of above.,1,"[SEE OH DEREFORD, SEE O DOUBLE YOU., COWCOWCOW...","[2, 0, 1, 3]"


In [ ]:
processed_train_qs=preprocess_qa(WP_train_df, is_train_data=True)

  1%|▏         | 5/396 [00:00<00:15, 24.91it/s]

for cowcowcowcoww, concept not found in hard grounding.
for cowcowcowcoww, concept not found in hard grounding.
for cobcobb, concept not found in hard grounding.


  7%|▋         | 27/396 [00:01<00:14, 26.07it/s]

for 28.0, concept not found in hard grounding.
for 30.0, concept not found in hard grounding.


 11%|█         | 42/396 [00:01<00:11, 30.64it/s]

...

 86%|████████▋ | 342/396 [00:09<00:01, 46.78it/s]

for slingsham ., concept not found in hard grounding.
for enclamp ., concept not found in hard grounding.
for gaslamp ., concept not found in hard grounding.


 98%|█████████▊| 388/396 [00:10<00:00, 47.39it/s]

for nightwatch ., concept not found in hard grounding.


100%|██████████| 396/396 [00:11<00:00, 35.71it/s]


In [ ]:
# with open(f'{DATA_FOLDER}/SP_train_preprocessed.jsonl', 'w') as outfile:
with open(f'{DATA_FOLDER}/WP_train_preprocessed2.jsonl', 'w') as outfile:
  for entry in processed_train_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
processed_train_qs[0]

{'id': 'WP-0',
 'answerKey': 'B',
 'question': {'question_concept': ['in',
   'cow',
   'spell',
   'do',
   'thirteen',
   'how',
   'you',
   'letter'],
  'choices': [{'label': 'A',
    'text': 'see oh dereford',
    'choice_concepts': ['oh', 'see']},
   {'label': 'B',
    'text': 'see o double you.',
    'choice_concepts': ['you', 'double', 'see', 'o']},
   {'label': 'C', 'text': 'cowcowcowcoww', 'choice_concepts': []},
   {'label': 'D',
    'text': 'none of above.',
    'choice_concepts': ['above', 'none', 'of']}],
  'stem': 'how do you spell cow in thirteen letters?'}}

In [ ]:
"""
{'id': 'cxzvpoiuzckf-3123',
 'question': {'stem': 'A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?',
  'choices': [{'label': 'A', 'text': 'throw'},
   {'label': 'B', 'text': 'bit'},
   {'label': 'C', 'text': 'gallon'},
   {'label': 'D', 'text': 'mouse'},
   {'label': 'E', 'text': 'hole', 'choice_concepts':[]}]},
 'answerKey': 'E'}
"""

In [ ]:
# def extract_concepts(q_stem):

#   question_concepts=[]
#   for i in range(len(id2concept)):
#     concept=id2concept[i].strip().lower().replace("_"," ")
#     if len(concept)<=2:
#       continue
#     if concept in q_stem:
#       question_concepts.append(concept)


#   new_question_concepts=[]
#   for i in range(len(question_concepts)):
#     first_concept=question_concepts[i]
#     is_sub_concept=False
#     for j in range(len(question_concepts)):
#       if i==j:
#         continue
#       if first_concept in question_concepts[j]:
#         is_sub_concept=True
#         break

#     if is_sub_concept==False:
#       new_question_concepts.append(first_concept)


#   meaningful_question_concepts=set()
#   for concept in new_question_concepts:
#     if concept in wn.words():
#       meaningful_question_concepts.add(concept)
#       continue

#     concept_subwords=concept.split(" ")
#     all_are_meaningful=True
#     meaningfuls=[]
#     for subword in concept_subwords:
#       if len(subword)<=2:
#         all_are_meaningful=False
#         continue

#       if subword in wn.words():
#         meaningfuls.append(subword)
#       else:
#         all_are_meaningful=False
#     if all_are_meaningful:
#       meaningful_question_concepts.add(concept)
#     else:
#       for c in meaningfuls:
#         meaningful_question_concepts.add(c)

#   return list(meaningful_question_concepts)

##<font color=lightgreen>Calc concept occurance frequesncy</font>

###riddle_sense

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"


linked_qa_file=f"{DATA_FOLDER}/rs_train_preprocessed.jsonl"

with open(linked_qa_file) as f:
  rs_train_instances = [json.loads(line) for line in f.read().split("\n") if line]

linked_qa_file=f"{DATA_FOLDER}/rs_dev_preprocessed.jsonl"

with open(linked_qa_file) as f:
  rs_dev_instances = [json.loads(line) for line in f.read().split("\n") if line]

print(f"Riddle-Sense train size: {len(rs_train_instances)}, Riddle-Sense dev size: {len(rs_dev_instances)}")

Riddle-Sense train size: 3510, Riddle-Sense dev size: 1021


In [ ]:
concepts_frequency={}

for i in tqdm(range(len(rs_train_instances))):
  qa_sample=rs_train_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  for concept in question_concepts:
    concept_counter=concepts_frequency.get(concept)
    if concept_counter!=None:
      concepts_frequency[concept]=(concept_counter+1)
    else:
      concepts_frequency[concept]=1

len(concepts_frequency.keys())

100%|██████████| 3510/3510 [00:00<00:00, 147817.25it/s]


5501

In [ ]:
for i in tqdm(range(len(rs_dev_instances))):
  qa_sample=rs_dev_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  for concept in question_concepts:
    concept_counter=concepts_frequency.get(concept)
    if concept_counter!=None:
      concepts_frequency[concept]=(concept_counter+1)
    else:
      concepts_frequency[concept]=1

len(concepts_frequency.keys())

100%|██████████| 1021/1021 [00:00<00:00, 46236.56it/s]


6435

In [ ]:
sorted_dict={k: v for k, v in sorted(concepts_frequency.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
most_common_concepts=['but',
 'can',
 'have',
 'not',
 'are',
 'there',
 'man',
 'one',
 'never',
 'who',
 'will',
 'yet',
 'all',
 'get',
 'why',
 'many',
 'out',
 'two',
 'only',
 'then',
 'make',
 'like',
 'always',
 'possible',
 'people',
 'name',
 'down',
 'time',
 'say',
 'use',
 'more',
 'know',
 'same',
 'said',
 'come',
 'son',
 'take',
 'left',
 'don',
 'first',
 'around',
 'kind',
 'want',
 'find',
 'some',
 'there are',
 'way',
 'still',
 'through',
 'thing',
 'end',
 'men',
 'sometimes',
 'put',
 'nothing',
 'away',
 'person',
 'each',
 'other',
 'stop',
 'right',
 'hold',
 'going',
 'once',
 'may',
 'just',
 'made',
 'used',
 'most',
 'found',
 'after',
 'second',
 'even',
 'both',
 'much',
 'here',
 'give',
 'every',
 'also',
 'number',
 'though',
 'while',
 'someone',
 'word',
 'full',
 'third',
 'any',
 'body',
 'before',
 'tail',
 'cant',
 'hear',
 'half',
 'about',
 'keep',
 'stand',"ways",
 'now',"red", "black", "brown", "white", "round","nearly","green","ever","near","very",
"which", "what", "where", "when", "how", "who", "why"
 ]

In [ ]:
len(most_common_concepts)

115

In [ ]:
sorted_dict

{'but': 950,
 'can': 721,
 'have': 463,
 'not': 385,
 'are': 353,
 'there': 349,
 'man': 323,
 'one': 269,
 'never': 264,
 'who': 236,
 'will': 222,
 'yet': 213,
 'all': 208,
 'see': 193,
 'get': 176,
 'why': 175,
 'many': 170,
 'out': 165,
 'two': 144,
 'only': 144,
 'then': 128,
 'make': 126,
 'like': 126,
 'always': 121,
 'white': 113,
 'black': 111,
 'possible': 111,
 'people': 108,
 'three': 107,
 'name': 102,
 'down': 101,
 'water': 100,
 'day': 100,
 'time': 98,
 'say': 97,
 'red': 97,
 'use': 97,
 'car': 96,
 'more': 95,
 'know': 93,
 'same': 91,
 'said': 90,
 'head': 89,
 'come': 88,
 'son': 88,
 'back': 87,
 'take': 85,
 'left': 85,
 'call': 80,
 'don': 78,
 'first': 78,
 'around': 77,
 'kind': 76,
 'want': 76,
 'find': 74,
 'world': 74,
 'eat': 74,
 'father': 74,
 'some': 73,
 'night': 72,
 'light': 71,
 'eyes': 70,
 'there are': 70,
 'four': 64,
 'long': 64,
 'way': 64,
 'life': 63,
 'still': 63,
 'through': 63,
 'die': 62,
 'thing': 62,
 'end': 62,
 'house': 62,
 'men': 60

###brain_teaser

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"


# linked_qa_file=f"{DATA_FOLDER}/SP_train_preprocessed.jsonl"
linked_qa_file=f"{DATA_FOLDER}/WP_train_preprocessed2.jsonl"

with open(linked_qa_file) as f:
  train_instances = [json.loads(line) for line in f.read().split("\n") if line]

# linked_qa_file=f"{DATA_FOLDER}/SP_dev_preprocessed.jsonl"
linked_qa_file=f"{DATA_FOLDER}/WP_dev_preprocessed2.jsonl"

with open(linked_qa_file) as f:
  dev_instances = [json.loads(line) for line in f.read().split("\n") if line]

print(f"Breain-Teaser SP_train size: {len(train_instances)}, Breain-Teaser SP_dev size: {len(dev_instances)}")

Breain-Teaser SP_train size: 396, Breain-Teaser SP_dev size: 120


In [ ]:
concept_count=[]

for i in tqdm(range(len(dev_instances))):
  qa_sample=dev_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 120/120 [00:00<00:00, 344973.60it/s]


10.016666666666667

In [ ]:
concept_count=[]

for i in tqdm(range(len(train_instances))):
  qa_sample=train_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 396/396 [00:00<00:00, 491549.09it/s]


9.618686868686869

In [ ]:
concepts_frequency={}

for i in tqdm(range(len(train_instances))):
  qa_sample=train_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  choices=qa_sample['question']['choices']
  for concept in question_concepts:
    concept_counter=concepts_frequency.get(concept)
    if concept_counter!=None:
      concepts_frequency[concept]=(concept_counter+1)
    else:
      concepts_frequency[concept]=1

  for j in range(len(choices)):
    choice_concepts=choices[j]['choice_concepts']
    for concept in choice_concepts:
      concept_counter=concepts_frequency.get(concept)
      if concept_counter!=None:
        concepts_frequency[concept]=(concept_counter+1)
      else:
        concepts_frequency[concept]=1

len(concepts_frequency.keys())

100%|██████████| 396/396 [00:00<00:00, 30964.08it/s]


1143

In [ ]:
for i in tqdm(range(len(dev_instances))):
  qa_sample=dev_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  choices=qa_sample['question']['choices']
  for concept in question_concepts:
    concept_counter=concepts_frequency.get(concept)
    if concept_counter!=None:
      concepts_frequency[concept]=(concept_counter+1)
    else:
      concepts_frequency[concept]=1

  for j in range(len(choices)):
    choice_concepts=choices[j]['choice_concepts']
    for concept in choice_concepts:
      concept_counter=concepts_frequency.get(concept)
      if concept_counter!=None:
        concepts_frequency[concept]=(concept_counter+1)
      else:
        concepts_frequency[concept]=1

len(concepts_frequency.keys())

100%|██████████| 120/120 [00:00<00:00, 41810.64it/s]


1143

In [ ]:
sorted_dict={k: v for k, v in sorted(concepts_frequency.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
most_common_concepts=[]

In [ ]:
most_common_concepts=['but',
 'can',
 'have',
 'not',
 'are',
 'there',
 'never',
 'who',
 'will',
 'yet',
 'all',
 'get',
 'why',
 'many',
 'only',
 'then',
 'make',
 'like',
 'always',
 'possible',
 'people',
 'name',
 'down',
 'say',
 'more',
 'know',
 'same',
 'said',
 'come',
 'son',
 'take',
 'left',
 'don',
 'around',
 'want',
 'some',
 'there are',
 'way',
 'still',
 'through',
 'thing',
 'end',
 'men',
 'sometimes',
 'put',
 'nothing',
 'away',
 'person',
 'each',
 'other',
 'stop',
 'right',
 'hold',
 'going',
 'once',
 'just',
 'made',
 'used',
 'most',
 'both',
 'much',
 'here',
 'give',
 'every',
 'also',
 'number',
 'though',
 'while',
 'someone',
 'full',
 'any',
 'cant',
 'about',
 'keep',
 'stand',
 'now',"nearly","ever","near","very",
"which", "what", "where", "when", "how", "who", "why",
######################################################################################################################################################
 'not',
 'possible',
 'there',
 'are',
 'all',
 'but',
 'why',
 'only',
 'have',
 'same',
 'each',
 'too',
 'get',
 'people',
 'there are',
 'will',
 'then',
 'other',
 'however',
 'still',
 'enough',
 'who',
 'more',
 'just',
 'yet',
 'any',
 'some',
 'made',
 'back',
 'son',
 'never','haven','wit',
 'see','till','mr.','mrs','mrs.','ms.','ied','led','err','yon','aten',
'don','george','smith','also','once','tried','why not','might', 'john','many people','ever','bob','yes','sam','such','cis','lent','but one',
'isn','some people', 'likely', 'very', 'sandy','every other','names','lan','tum','nsa', 'rad','mary', 'eris','peri','sir', 'isi','ddi','din','rom']

In [ ]:
sorted_dict['isi']

2

In [ ]:
# list(sorted_dict.keys())[2500:]

In [ ]:
sorted_dict

{'of': 804,
 'the': 531,
 'above': 518,
 'none': 516,
 'a': 400,
 'what': 343,
 'be': 336,
 'letter': 317,
 'in': 238,
 'and': 188,
 'do': 137,
 'you': 133,
 'how': 85,
 'kind': 81,
 'word': 78,
 'not': 75,
 'can': 73,
 'one': 72,
 'to': 67,
 'with': 59,
 'which': 54,
 'have': 52,
 'it': 48,
 'two': 44,
 'on': 42,
 'e': 42,
 'but': 42,
 'four': 41,
 'at': 41,
 'alphabet': 40,
 'an': 40,
 'number': 39,
 'type': 39,
 'o': 38,
 'when': 38,
 'take': 37,
 'never': 36,
 'add': 35,
 'three': 33,
 'spell': 32,
 'that': 32,
 'n': 32,
 'music': 32,
 't': 31,
 'each': 31,
 'no': 31,
 'find': 29,
 'or': 29,
 'make': 29,
 'old': 29,
 'house': 29,
 'if': 28,
 'than': 28,
 'end': 28,
 'world': 28,
 'five': 27,
 'they': 27,
 'm': 27,
 'eat': 27,
 'for': 26,
 'year': 26,
 'wall': 26,
 'from': 25,
 'remove': 25,
 'there': 25,
 'r': 25,
 'new': 25,
 'contain': 25,
 'blue': 24,
 'leave': 23,
 'by': 23,
 'f': 23,
 'as': 23,
 'day': 23,
 'become': 22,
 'middle': 21,
 'out': 21,
 'only': 21,
 'more': 21,
 's

##<font color=lightgreen>extract concepts with respect to concept frequencies</font>

###riddle_sense

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"


linked_qa_file=f"{DATA_FOLDER}/rs_train.jsonl"

with open(linked_qa_file) as f:
  rs_train_instances = [json.loads(line) for line in f.read().split("\n") if line]

linked_qa_file=f"{DATA_FOLDER}/rs_dev.jsonl"

with open(linked_qa_file) as f:
  rs_dev_instances = [json.loads(line) for line in f.read().split("\n") if line]

print(f"Riddle-Sense train size: {len(rs_train_instances)}, Riddle-Sense dev size: {len(rs_dev_instances)}")

Riddle-Sense train size: 3510, Riddle-Sense dev size: 1021


In [ ]:
def extract_concepts(question_list):
  processed_qs=[]

  for i in tqdm(range(len(question_list))):

    sample=question_list[i]
    q_id=sample['id']
    q_answerKey=sample['answerKey']
    q_stem=sample['question']['stem'].strip().lower()
    q_choices=sample['question']['choices']

    question_concepts=[]
    for i in range(len(id2concept)):
      concept=id2concept[i].strip().lower().replace("_"," ")
      if concept in most_common_concepts or len(concept)<=3:
        continue
      if concept in q_stem:
        question_concepts.append(concept)


    new_question_concepts=[]
    for i in range(len(question_concepts)):
      first_concept=question_concepts[i]
      is_sub_concept=False
      for j in range(len(question_concepts)):
        if i==j:
          continue
        if first_concept in question_concepts[j]:
          is_sub_concept=True
          break

      if is_sub_concept==False:
        new_question_concepts.append(first_concept)


    meaningful_question_concepts=set()
    for concept in new_question_concepts:
      if concept in wn.words():
        meaningful_question_concepts.add(concept)
        continue

      concept_subwords=concept.split(" ")
      all_are_meaningful=True
      meaningfuls=[]
      for subword in concept_subwords:
        if len(subword)<=3:
          all_are_meaningful=False
          continue

        if subword in wn.words():
          meaningfuls.append(subword)
        else:
          all_are_meaningful=False
      if all_are_meaningful:
        meaningful_question_concepts.add(concept)
      else:
        for c in meaningfuls:
          if c not in most_common_concepts:
            meaningful_question_concepts.add(c)


    q_dict={
        'id':q_id,
        'answerKey':q_answerKey,
        'question':{
            'question_concept': list(meaningful_question_concepts),
            'choices': q_choices,
            'stem': q_stem,
        }
    }
    processed_qs.append(q_dict)

  return processed_qs

In [ ]:
processed_dev_qs=extract_concepts(rs_dev_instances)

100%|██████████| 1021/1021 [36:47<00:00,  2.16s/it]


In [ ]:
with open(f'{DATA_FOLDER}/rs_dev_preprocessed_pruned.jsonl', 'w') as outfile:
  for entry in processed_dev_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
concept_count=[]

for i in tqdm(range(len(processed_dev_qs))):
  qa_sample=processed_dev_qs[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 1021/1021 [00:00<00:00, 410543.99it/s]


5.09696376101861

check len qc

In [ ]:
processed_train_qs=extract_concepts(rs_train_instances)

100%|██████████| 3510/3510 [2:04:17<00:00,  2.12s/it]


In [ ]:
with open(f'{DATA_FOLDER}/rs_train_preprocessed_pruned.jsonl', 'w') as outfile:
  for entry in processed_train_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
concept_count=[]

for i in tqdm(range(len(processed_train_qs))):
  qa_sample=processed_train_qs[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 3510/3510 [00:00<00:00, 490047.50it/s]


4.323361823361823

mean concept count per q

dev not pruned

In [ ]:
linked_qa_file=f"{DATA_FOLDER}/rs_dev_preprocessed.jsonl"

with open(linked_qa_file) as f:
  rs_dev_instances = [json.loads(line) for line in f.read().split("\n") if line]

In [ ]:
concept_count=[]

for i in tqdm(range(len(rs_dev_instances))):
  qa_sample=rs_dev_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))


100%|██████████| 1021/1021 [00:00<00:00, 307399.64it/s]


In [ ]:
np.mean(np.array(concept_count))

8.46620959843291

pruned dev

In [ ]:
concept_count=[]

for i in tqdm(range(len(processed_dev_qs))):
  qa_sample=processed_dev_qs[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))


100%|██████████| 1021/1021 [00:00<00:00, 475134.18it/s]


In [ ]:
import numpy as np

np.mean(np.array(concept_count))

7.078354554358472

train not pruned

In [ ]:
linked_qa_file=f"{DATA_FOLDER}/rs_train_preprocessed.jsonl"

with open(linked_qa_file) as f:
  rs_train_instances = [json.loads(line) for line in f.read().split("\n") if line]

In [ ]:
concept_count=[]

for i in tqdm(range(len(rs_train_instances))):
  qa_sample=rs_train_instances[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 3510/3510 [00:00<00:00, 406471.94it/s]


7.445014245014245

###Brain_teaser

In [ ]:
# SP_dataset_train = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP-train.npy', allow_pickle=True,encoding='bytes')
dataset_train = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP-train.npy', allow_pickle=True,encoding='bytes')
print(len(dataset_train))

data_dict = {'id':[],
             'question':[],
             'answer':[],
             'distractor1':[],
             'distractor2':[],
             'distractor(unsure)':[],
             'label':[],
             'choice_list':[],
             'choice_order':[]}
for i in dataset_train:
  for k,v in i.items():
    data_dict[k].append(v)

train_df = pd.DataFrame(data_dict, columns=['id', 'question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)', 'label', 'choice_list', 'choice_order'])
train_df.head(1)

396


,id,question,answer,distractor1,distractor2,distractor(unsure),label,choice_list,choice_order
0,WP-0,How do you spell COW in thirteen letters?,SEE O DOUBLE YOU.,COWCOWCOWCOWW,SEE OH DEREFORD,None of above.,1,"[SEE OH DEREFORD, SEE O DOUBLE YOU., COWCOWCOW...","[2, 0, 1, 3]"


In [ ]:
# SP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
print(len(dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
val_df.head(1)

120


,question,choice_list
0,What kind of nut has no shell?,"[A peanut., A Doughnut., A walnut., None of ab..."


In [ ]:
from spacy.matcher import Matcher

def load_matcher(nlp, pattern_path):
    with open(pattern_path, "r", encoding="utf8") as fin:
        all_patterns = json.load(fin)

    matcher = Matcher(nlp.vocab)
    for concept, pattern in all_patterns.items():
        # matcher.add(concept, None, pattern)
        matcher.add(concept, [pattern])
    return matcher

In [ ]:
PATTERN_PATH="/content/drive/MyDrive/brain_teaser/datasets/cpnet/matcher_patterns.json"

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser', 'textcat'])
# nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.add_pipe('sentencizer')

def load_cpnet_vocab(cpnet_vocab_path):
    with open(cpnet_vocab_path, "r", encoding="utf8") as fin:
        cpnet_vocab = [l.strip() for l in fin]
    cpnet_vocab = [c.replace("_", " ") for c in cpnet_vocab]
    return cpnet_vocab
# matcher = load_matcher(nlp, PATTERN_PATH)

In [ ]:
train_df.iloc[0]

id                                                                 WP-0
question                      How do you spell COW in thirteen letters?
answer                                                SEE O DOUBLE YOU.
distractor1                                               COWCOWCOWCOWW
distractor2                                             SEE OH DEREFORD
distractor(unsure)                                       None of above.
label                                                                 1
choice_list           [SEE OH DEREFORD, SEE O DOUBLE YOU., COWCOWCOW...
choice_order                                               [2, 0, 1, 3]
Name: 0, dtype: object

In [ ]:
doc = nlp(train_df.iloc[0]['question'])
doc

How do you spell COW in thirteen letters?

In [ ]:
for t in doc:
  print(t.lemma_)

how
do
you
spell
COW
in
thirteen
letter
?


In [ ]:

# def hard_ground(nlp, sent, cpnet_vocab):
#     sent = sent.lower()
#     doc = nlp(sent)
#     res = set()
#     for t in doc:
#         if t.lemma_ in cpnet_vocab:
#             res.add(t.lemma_)
#     sent = " ".join([t.text for t in doc])
#     if sent in cpnet_vocab:
#         res.add(sent)
#     try:
#         assert len(res) > 0
#     except Exception:
#         print(f"for {sent}, concept not found in hard grounding.")
#     return res

In [ ]:
def extract_concepts(sentence, char_count_thresh):

  # question_concepts=[]
  # for i in range(len(id2concept)):
  #   concept=id2concept[i].strip().lower().replace("_"," ")
  #   if len(concept)<=char_count_thresh:
  #     continue
  #   if concept in sentence:
  #     question_concepts.append(concept)

  # meaningful_question_concepts=set()
  # for concept in question_concepts:
  #   if concept in wn.words():
  #     meaningful_question_concepts.add(concept)
  #     continue

  #   concept_subwords=concept.split(" ")
  #   all_are_meaningful=True
  #   meaningfuls=[]
  #   for subword in concept_subwords:
  #     if len(subword)<=char_count_thresh:
  #       all_are_meaningful=False
  #       continue

  #     if subword in wn.words():
  #       meaningfuls.append(subword)
  #     else:
  #       all_are_meaningful=False
  #   if all_are_meaningful:
  #     meaningful_question_concepts.add(concept)
  #   else:
  #     for c in meaningfuls:
  #       meaningful_question_concepts.add(c)

  # meaningful_question_concepts=list(meaningful_question_concepts)

  # new_question_concepts=[]
  # for i in range(len(meaningful_question_concepts)):
  #   first_concept=meaningful_question_concepts[i]
  #   is_sub_concept=False
  #   for j in range(len(meaningful_question_concepts)):
  #     if i==j:
  #       continue
  #     if first_concept in meaningful_question_concepts[j]:
  #       is_sub_concept=True
  #       break

  #   if is_sub_concept==False:
  #     new_question_concepts.append(first_concept)
  sent = sentence.lower()
  doc = nlp(sent)
  question_concepts = set()
  for t in doc:
    if t.lemma_ in concept2id:
      question_concepts.add(t.lemma_)
  sent = " ".join([t.text for t in doc])
  if sent in concept2id:
    question_concepts.add(sent)
  try:
    assert len(question_concepts) > 0
  except Exception:
      print(f"for {sent}, concept not found in hard grounding.")

  final_concepts=[]
  for concept in question_concepts:
    if concept in most_common_concepts:
      continue
    else:
      final_concepts.append(concept)

  return final_concepts


def preprocess_qa(question_list, is_train_data, char_count_thresh):

  processed_qs=[]
  id_to_label={0:"A", 1:"B", 2:"C", 3:"D"}

  for i in tqdm(range(len(question_list))):

    sample=question_list.iloc[i]
    q_stem=sample['question'].strip().lower()
    q_choices_list=sample['choice_list']
    if is_train_data:
      q_answerKey=id_to_label[sample['label']]
      q_id=sample['id']
    else:
      q_id=i


    meaningful_question_concepts = extract_concepts(q_stem, char_count_thresh)

    q_choices=[]

    for j in range(len(q_choices_list)):
      if j==0:
        choice_label="A"
      elif j==1:
        choice_label="B"
      elif j==2:
        choice_label="C"
      else:
        choice_label="D"

      choice_text=q_choices_list[j]
      choice_dict={
          'label': choice_label,
          'text': choice_text.strip().lower()
      }

      if "none of above" in choice_dict['text']:
        meaningful_choice_concepts=['none']
      else:
        meaningful_choice_concepts = extract_concepts(choice_dict['text'], char_count_thresh)

      choice_dict['choice_concepts']= meaningful_choice_concepts
      q_choices.append(choice_dict)

    if is_train_data:
      q_dict={
            'id':q_id,
            'answerKey':q_answerKey,
            'question':{
                'question_concept': meaningful_question_concepts,
                'choices': q_choices,
                'stem': q_stem,
            }
        }
    else:
        q_dict={
            'id':q_id,
            'question':{
                'question_concept': meaningful_question_concepts,
                'choices': q_choices,
                'stem': q_stem,
            }
        }
    processed_qs.append(q_dict)
    # break
  return processed_qs

In [ ]:
# processed_dev_qs=preprocess_qa(SP_val_df, is_train_data=False, char_count_thresh=2)
processed_dev_qs=preprocess_qa(val_df, is_train_data=False, char_count_thresh=1)

 30%|███       | 36/120 [00:00<00:00, 87.54it/s]

for abbit ., concept not found in hard grounding.
for abbit ., concept not found in hard grounding.
for smie ., concept not found in hard grounding.
for cowcowcowcoww, concept not found in hard grounding.
for cowcowcowcoww, concept not found in hard grounding.
for cobcobb, concept not found in hard grounding.


 72%|███████▎  | 87/120 [00:00<00:00, 94.63it/s]

for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.
for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.
for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.


 89%|████████▉ | 107/120 [00:01<00:00, 90.80it/s]

for 28.0, concept not found in hard grounding.
for 30.0, concept not found in hard grounding.


100%|██████████| 120/120 [00:01<00:00, 88.54it/s]


In [ ]:
processed_dev_qs[0]

{'id': 0,
 'question': {'question_concept': ['kind',
   'no',
   'of',
   'have',
   'shell',
   'what',
   'nut'],
  'choices': [{'label': 'A',
    'text': 'a peanut.',
    'choice_concepts': ['peanut', 'a']},
   {'label': 'B', 'text': 'a doughnut.', 'choice_concepts': ['doughnut', 'a']},
   {'label': 'C', 'text': 'a walnut.', 'choice_concepts': ['walnut', 'a']},
   {'label': 'D', 'text': 'none of above.', 'choice_concepts': ['none']}],
  'stem': 'what kind of nut has no shell?'}}

In [ ]:
# with open(f'{DATA_FOLDER}/SP_dev_preprocessed_pruned.jsonl', 'w') as outfile:
with open(f'{DATA_FOLDER}/WP_dev_preprocessed_pruned2.jsonl', 'w') as outfile:
  for entry in processed_dev_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
concept_count=[]

for i in tqdm(range(len(processed_dev_qs))):
  qa_sample=processed_dev_qs[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 120/120 [00:00<00:00, 381878.97it/s]


10.016666666666667

In [ ]:
# processed_train_qs=preprocess_qa(SP_train_df, is_train_data=True, char_count_thresh=2)
processed_train_qs=preprocess_qa(WP_train_df, is_train_data=True, char_count_thresh=1)

  2%|▏         | 7/396 [00:00<00:06, 58.56it/s]

for cowcowcowcoww, concept not found in hard grounding.
for cowcowcowcoww, concept not found in hard grounding.
for cobcobb, concept not found in hard grounding.


  8%|▊         | 30/396 [00:00<00:07, 50.29it/s]

for 28.0, concept not found in hard grounding.
for 30.0, concept not found in hard grounding.


 12%|█▏        | 48/396 [00:00<00:06, 50.23it/s]

...

 87%|████████▋ | 344/396 [00:07<00:00, 58.68it/s]

for slingsham ., concept not found in hard grounding.
for enclamp ., concept not found in hard grounding.
for gaslamp ., concept not found in hard grounding.


 98%|█████████▊| 390/396 [00:08<00:00, 59.22it/s]

for nightwatch ., concept not found in hard grounding.


100%|██████████| 396/396 [00:08<00:00, 47.31it/s]


In [ ]:
# with open(f'{DATA_FOLDER}/SP_train_preprocessed_pruned.jsonl', 'w') as outfile:
with open(f'{DATA_FOLDER}/WP_train_preprocessed_pruned2.jsonl', 'w') as outfile:
  for entry in processed_train_qs:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
concept_count=[]

for i in tqdm(range(len(processed_train_qs))):
  qa_sample=processed_train_qs[i]
  question_concepts=qa_sample['question']['question_concept']
  concept_count.append(len(question_concepts))

np.mean(np.array(concept_count))

100%|██████████| 396/396 [00:00<00:00, 459331.96it/s]


9.618686868686869

##<font color=pink>Brain-Teaser Preprocessing</font>

###convert_to_entailment

In [ ]:
DATA_FOLDER="/content/drive/MyDrive/brain_teaser/datasets"


# linked_qa_file=f"{DATA_FOLDER}/SP_train_preprocessed_pruned.jsonl"
linked_qa_file=f"{DATA_FOLDER}/WP_train_preprocessed_pruned2.jsonl"

with open(linked_qa_file) as f:
  processed_train_qs = [json.loads(line) for line in f.read().split("\n") if line]

# linked_qa_file=f"{DATA_FOLDER}/SP_dev_preprocessed_pruned.jsonl"
linked_qa_file=f"{DATA_FOLDER}/WP_dev_preprocessed_pruned2.jsonl"

with open(linked_qa_file) as f:
  processed_dev_qs = [json.loads(line) for line in f.read().split("\n") if line]

print(f"Brain-teaser train size: {len(processed_train_qs)}, Brain-teaser dev size: {len(processed_dev_qs)}")

Brain-teaser train size: 396, Brain-teaser dev size: 120


In [ ]:
"""

convert to entailment output

{"id": "xzc81234ioioucv-1052",
"answerKey": "B",

"question": {
"question_concept": ["foot", "different", "long"],

"choices": [{"label": "A", "text": "object"},
{"label": "B", "text": "shoe"},
{"label": "C", "text": "footing"},
{"label": "D", "text": "grave"},
{"label": "E", "text": "column"}],

"stem": "what comes in many different sizes but is always only 1 foot long?"},

"statements": [
{"label": false, "statement": "Object comes in many different sizes but is always only 1 foot long."},
{"label": true, "statement": "Shoe comes in many different sizes but is always only 1 foot long."},
{"label": false, "statement": "Footing comes in many different sizes but is always only 1 foot long."},
{"label": false, "statement": "Grave comes in many different sizes but is always only 1 foot long."},
{"label": false, "statement": "Column comes in many different sizes but is always only 1 foot long."}]}

"""

In [ ]:
def bt_convert_to_entailemnt(qa_dataset, is_train_data):
  qa_statements=[]

  for i in tqdm(range(len(qa_dataset))):
    sample=qa_dataset[i]
    id=sample['id']

    if is_train_data:
      answerKey=sample['answerKey']
    question=sample['question']['stem']
    question_concept=sample['question']['question_concept']
    q_choices=sample['question']['choices']

    statements=[]

    for choice in q_choices:
      statement={}
      if is_train_data:
        statement['label']=choice["label"] == sample.get("answerKey")
      else:
        statement['label']=None

      statement['statement']=question+" "+choice['text']

      statements.append(statement)


    if is_train_data:
      q_dict={
                'id':id,
                'answerKey':answerKey,
                'question':{
                    'question_concept': question_concept,
                    'choices': q_choices,
                    'stem': question,
                },
                'statements':statements
            }
    else:
      q_dict={
                'id':id,
                'question':{
                    'question_concept': question_concept,
                    'choices': q_choices,
                    'stem': question,
                },
                'statements':statements
            }
    qa_statements.append(q_dict)
    # break

  return qa_statements

In [ ]:
train_entailemnts= bt_convert_to_entailemnt(processed_train_qs, is_train_data=True)

100%|██████████| 396/396 [00:00<00:00, 82445.37it/s]


In [ ]:
statement_folder=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement"

# with open(f'{statement_folder}/SP_train.statement.jsonl', 'w') as outfile:
with open(f'{statement_folder}/WP_train.statement.jsonl', 'w') as outfile:
  for entry in train_entailemnts:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
dev_entailemnts= bt_convert_to_entailemnt(processed_dev_qs, is_train_data=False)

100%|██████████| 120/120 [00:00<00:00, 82214.39it/s]


In [ ]:
# with open(f'{statement_folder}/SP_dev.statement.jsonl', 'w') as outfile:
with open(f'{statement_folder}/WP_dev.statement.jsonl', 'w') as outfile:
  for entry in dev_entailemnts:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
train_entailemnts[1]

{'id': 'WP-0_SR',
 'answerKey': 'C',
 'question': {'question_concept': ['cow',
   'spell',
   'you',
   'thirteen',
   'do',
   'how',
   'in',
   'letter'],
  'choices': [{'label': 'A',
    'text': 'see oh dereford',
    'choice_concepts': ['oh', 'see']},
   {'label': 'B', 'text': 'cowcowcowcoww', 'choice_concepts': []},
   {'label': 'C',
    'text': 'see o double you.',
    'choice_concepts': ['you', 'double', 'see', 'o']},
   {'label': 'D', 'text': 'none of above.', 'choice_concepts': ['none']}],
  'stem': 'in thirteen letters, how do you spell cow?'},
 'statements': [{'label': False,
   'statement': 'in thirteen letters, how do you spell cow? see oh dereford'},
  {'label': False,
   'statement': 'in thirteen letters, how do you spell cow? cowcowcowcoww'},
  {'label': True,
   'statement': 'in thirteen letters, how do you spell cow? see o double you.'},
  {'label': False,
   'statement': 'in thirteen letters, how do you spell cow? none of above.'}]}

In [ ]:
train_entailemnts[0]

{'id': 'SP-0',
 'answerKey': 'B',
 'question': {'question_concept': ['family',
   'brother',
   'daughter',
   'six',
   'one',
   'mustard',
   'hat'],
  'choices': [{'label': 'A',
    'text': 'some daughters get married and have their own family.',
    'choice_concepts': ['heir', 'daughter', 'own family', 'get married']},
   {'label': 'B',
    'text': 'each daughter shares the same brother.',
    'choice_concepts': ['ares', 'daughter', 'share', 'brother']},
   {'label': 'C',
    'text': 'some brothers were not loved by family and moved away.',
    'choice_concepts': ['loved', 'moved', 'family', 'brother']},
   {'label': 'D', 'text': 'none of above.', 'choice_concepts': ['none']}],
  'stem': 'mr. and mrs. mustard have six daughters and each daughter has one brother. but there are only 9 people in the family, how is that possible?'},
 'statements': [{'label': False,
   'statement': 'mr. and mrs. mustard have six daughters and each daughter has one brother. but there are only 9 people i

grounding --> done with original code

In [ ]:
"""

grounding output

{"sent": "Object comes in many different sizes but is always only 1 foot long.",
"ans": "object",
"qc": ["always", "come", "comes", "different", "different_sizes", "foot", "long", "many", "size", "sizes"],
"ac": ["object"]}

{"sent": "Shoe comes in many different sizes but is always only 1 foot long.", "ans": "shoe", "qc": ["always", "come", "comes", "different", "different_sizes", "foot", "long", "many", "size", "sizes"], "ac": ["shoe"]}
{"sent": "Footing comes in many different sizes but is always only 1 foot long.", "ans": "footing", "qc": ["always", "come", "comes", "different", "different_sizes", "long", "many", "size", "sizes"], "ac": ["foot"]}
{"sent": "Grave comes in many different sizes but is always only 1 foot long.", "ans": "grave", "qc": ["always", "come", "comes", "different", "different_sizes", "foot", "long", "many", "size", "sizes"], "ac": ["grave", "graven"]}
{"sent": "Column comes in many different sizes but is always only 1 foot long.", "ans": "column", "qc": ["always", "come", "comes", "different", "different_sizes", "foot", "long", "many", "size", "sizes"], "ac": ["column"]}



"""

generate_adj_data_from_grounded_concepts__use_LM --> done with original code

###grounding

####SP

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

routines = {
    'brain_teaser_ds': [
        {'func': ground, 'args': (output_paths['brain_teaser_ds']['statement']['SP_dev'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['brain_teaser_ds']['grounded']['SP_dev'], args["nprocs"])},
    ],
}


preprocess_func(args, routines)

brain_teaser_ds
{'func': <function ground at 0x7b5c99019870>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_dev.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_dev.grounded.jsonl', 2)}


 94%|█████████▍| 451/480 [16:58<01:05,  2.24s/it]

for tumnioavy4g4yjtpm7nddbi9chaab9ruep, concept not found in hard grounding.


100%|██████████| 480/480 [00:19<00:00, 24.04it/s]


grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_dev.grounded.jsonl

Successfully run brain_teaser_ds


In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

routines = {
    'brain_teaser_ds': [
        {'func': ground, 'args': (output_paths['brain_teaser_ds']['statement']['SP_train'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['brain_teaser_ds']['grounded']['SP_train'], args["nprocs"])},
    ],
}


preprocess_func(args, routines)

brain_teaser_ds
{'func': <function ground at 0x7d0c6ab5da20>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_train.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_train.grounded.jsonl', 2)}


 10%|█         | 209/2028 [08:48<35:36,  1.17s/it]

for tumnioavy4g4yjtpm7nddbi9chaab9ruep, concept not found in hard grounding.


 42%|████▏     | 842/2028 [25:24<28:56,  1.46s/it]

for jil ., concept not found in hard grounding.


...

for 1776.0, concept not found in hard grounding.
for 2020.0, concept not found in hard grounding.


 53%|█████▎    | 1071/2028 [30:03<23:38,  1.48s/it]

for tumnioavy4g4yjtpm7nddbi9chaab9ruep, concept not found in hard grounding.


100%|██████████| 2028/2028 [01:22<00:00, 24.52it/s]


grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_train.grounded.jsonl

Successfully run brain_teaser_ds


####WP

In [ ]:
cpnet_vocab = load_cpnet_vocab(output_paths['cpnet']['vocab'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

routines = {
    'brain_teaser_ds': [
        {'func': ground, 'args': (output_paths['brain_teaser_ds']['statement']['WP_dev'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['brain_teaser_ds']['grounded']['WP_dev'], args["nprocs"])},
    ],
}


preprocess_func(args, routines)

brain_teaser_ds
{'func': <function ground at 0x7d8596e37010>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_dev.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_dev.grounded.jsonl', 2)}


 18%|█▊        | 84/480 [04:16<06:02,  1.09it/s]

for abbit ., concept not found in hard grounding.


 18%|█▊        | 87/480 [04:18<05:19,  1.23it/s]

for abbit ., concept not found in hard grounding.


...

for abcdefghijklmnopqrstuvwxyz ., concept not found in hard grounding.


 81%|████████▏ | 390/480 [07:16<01:00,  1.49it/s]

for 28.0, concept not found in hard grounding.
for 30.0, concept not found in hard grounding.


100%|██████████| 480/480 [00:07<00:00, 60.27it/s] 


grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_dev.grounded.jsonl

Successfully run brain_teaser_ds


In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

routines = {
    'brain_teaser_ds': [
        {'func': ground, 'args': (output_paths['brain_teaser_ds']['statement']['WP_train'], output_paths['cpnet']['vocab'],
                                  output_paths['cpnet']['patterns'], output_paths['brain_teaser_ds']['grounded']['WP_train'], args["nprocs"])},
    ],
}


preprocess_func(args, routines)

brain_teaser_ds
{'func': <function ground at 0x7d8596e37010>, 'args': ('/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl', './data/cpnet/concept.txt', './data/cpnet/matcher_patterns.json', '/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_train.grounded.jsonl', 2)}


  0%|          | 1/1584 [02:58<78:18:35, 178.09s/it]

for cowcowcowcoww, concept not found in hard grounding.


  0%|          | 5/1584 [03:00<8:30:50, 19.41s/it] 

for cowcowcowcoww, concept not found in hard grounding.


...

for gaslamp ., concept not found in hard grounding.


 96%|█████████▌| 1521/1584 [17:25<00:36,  1.73it/s]

for nightwatch ., concept not found in hard grounding.


100%|██████████| 1584/1584 [00:21<00:00, 72.59it/s]

grounded concepts saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_train.grounded.jsonl

Successfully run brain_teaser_ds


###generate_adj_data_from_grounded_concepts__use_LM

####SP_dev

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['brain_teaser_ds']['grounded']['SP_dev'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-SP_dev'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_dev.grounded.jsonl...


100%|██████████| 480/480 [00:11<00:00, 40.69it/s]


In [ ]:
data_range_to_process=[0,480]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470


In [ ]:
len(res2[0])

480

In [ ]:
do_part3(res2[0], args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-SP_dev'])

100%|██████████| 480/480 [06:16<00:00,  1.28it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_dev.graph.adj.pk



In [ ]:
temp_res2[0][0]

([291,
  1988,
  2805,
  3153,
  4426,
  4629,
  6422,
  8912,
  9853,
  14901,
  17632,
  17635,
  23346,
  36809,
  72046,
  147003,
  184131,
  376734,
  472083],
 [6195, 15375, 52510, 229219, 404519],
 'everyone called him "batman," but he knew nothing about bats and thought they were disgusting. he still cherished being referred to as batman! how is this possible? he tries to be friendly..',
 [287,
  289,
  295,
  306,
  309,
  567,
  572,
  575,
  579,
  648,
  691,
  695,
  786,
  874,
  884,
  1001,
  1004,
  1172,
  1192,
  1363,
  1564,
  1594,
  1692,
  1713,
  1726,
  1769,
  2002,
  2106,
  2130,
  2194,
  2274,
  2288,
  2350,
  2352,
  2412,
  2413,
  2571,
  2573,
  2606,
  2611,
  3013,
  3050,
  3071,
  3078,
  3126,
  3319,
  3450,
  3517,
  3804,
  3921,
  4107,
  4299,
  4363,
  4368,
  4495,
  4529,
  4574,
  4627,
  5354,
  5586,
  5698,
  5703,
  5854,
  6263,
  6423,
  6424,
  6531,
  6532,
  6706,
  6714,
  6717,
  6745,
  6753,
  6757,
  6758,
  6788,
  6915,

####SP_train

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['brain_teaser_ds']['grounded']['SP_train'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-SP_train'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/SP_train.grounded.jsonl...


100%|██████████| 2028/2028 [00:39<00:00, 51.43it/s]


In [ ]:
data_range_to_process=[0,500]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [ ]:
data_range_to_process=[500,1000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [ ]:
data_range_to_process=[1000, 1500]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490


In [ ]:
data_range_to_process=[1500,2028]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020


combine

In [ ]:
temp_res2=[]
i=500
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

500

In [ ]:
i=1000
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

1000

In [ ]:
i=1500
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

1500

In [ ]:
i=2028
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

2028

part3

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

do_part3(temp_res2, args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-SP_train'])

100%|██████████| 2028/2028 [22:52<00:00,  1.48it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_train.graph.adj.pk



In [ ]:
%cd "/content/data"

%cp -av /content/data/csqa   /content/drive/MyDrive/brain_teaser/datasets/

'/content/data/csqa' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa'
'/content/data/csqa/grounded' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/grounded'
'/content/data/csqa/grounded/test.grounded.jsonl' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/grounded/test.grounded.jsonl'
'/content/data/csqa/grounded/train.grounded.jsonl' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/grounded/train.grounded.jsonl'
'/content/data/csqa/grounded/dev.grounded.jsonl' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/grounded/dev.grounded.jsonl'
'/content/data/csqa/graph' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/graph'
'/content/data/csqa/graph/test.graph.adj.pk' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/graph/test.graph.adj.pk'
'/content/data/csqa/graph/dev.graph.adj.pk.loaded_cache' -> '/content/drive/MyDrive/brain_teaser/datasets/csqa/csqa/graph/dev.graph.adj.pk.loaded_cache'
'/content/data/csqa/graph/tr

####WP_dev

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['brain_teaser_ds']['grounded']['WP_dev'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-WP_dev'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_dev.grounded.jsonl...


100%|██████████| 480/480 [00:01<00:00, 314.71it/s]


In [ ]:
data_range_to_process=[0,480]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/WP_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470


In [ ]:
do_part3(res2[0], args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-WP_dev'])

100%|██████████| 480/480 [00:38<00:00, 12.33it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_dev.graph.adj.pk



####WP_train

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False
res1=generate_adj_data_from_grounded_concepts__use_LM(output_paths['brain_teaser_ds']['grounded']['WP_train'],
                  output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-WP_train'], args["nprocs"])

generating adj data for /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/WP_train.grounded.jsonl...


100%|██████████| 1584/1584 [00:03<00:00, 406.43it/s]


part2

In [ ]:
data_range_to_process=[0,500]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490


In [ ]:
data_range_to_process=[500,1000]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [ ]:
data_range_to_process=[1000, 1584]
res2=do_part2(data_range_to_process, res1)

res2_path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{data_range_to_process[1]}.pickle"
with open(res2_path, 'wb') as fout:
  pickle.dump(res2, fout)

1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580


combining

In [ ]:
temp_res2=[]
i=500
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

500

In [ ]:
i=1000
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

1000

In [ ]:
i=1584
path=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/train_WP_res2_{i}.pickle"

with open(path, 'rb') as file:
  temp=pickle.load(file)

temp_res2.extend(temp[0])
len(temp_res2)

1584

part3

In [ ]:
args=dict()
args["run"]=['brain_teaser_ds']
args["path_prune_threshold"]=0.12
args["max_node_num"]=200
args["nprocs"]=cpu_count()
args["seed"]=0
args["debug"]=False

do_part3(temp_res2, args["nprocs"], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['brain_teaser_ds']['graph']['adj-WP_train'])

100%|██████████| 1584/1584 [01:27<00:00, 18.08it/s]


adj data saved to /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_train.graph.adj.pk



##<font color=lightblue>Brain-teaser In-house Train-split</font>

In [ ]:
DATA_FOLDER=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement"

# linked_qa_file=f"{DATA_FOLDER}/original_SP_train.statement.jsonl"
linked_qa_file=f"{DATA_FOLDER}/original_WP_train.statement.jsonl"

with open(linked_qa_file) as f:
  processed_train_qs = [json.loads(line) for line in f.read().split("\n") if line]


print(f"Brain-teaser train size: {len(processed_train_qs)}")

Brain-teaser train size: 396


In [ ]:
#SP
# train_keys_path="/content/drive/MyDrive/brain_teaser/datasets/SP_train_best_split_ids.pickle"
#WP
train_keys_path="/content/drive/MyDrive/brain_teaser/datasets/WP_train_best_split_ids.pickle"

In [ ]:
#SP
# dev_keys_path="/content/drive/MyDrive/brain_teaser/datasets/SP_dev_best_split_ids.pickle"
#WP
dev_keys_path="/content/drive/MyDrive/brain_teaser/datasets/WP_dev_best_split_ids.pickle"

In [ ]:
import pickle

with open(train_keys_path, 'rb') as f:
  train_qa_ids = pickle.load(f)

len(train_qa_ids)

356

In [ ]:
with open(dev_keys_path, 'rb') as f:
  dev_qa_ids = pickle.load(f)

len(dev_qa_ids)

40

In [ ]:
# len(processed_train_qs)*0.1

In [ ]:
# all_qa_ids=[]

# for i in range(len(processed_train_qs)):
#   all_qa_ids.append(processed_train_qs[i]['id'])

In [ ]:
train_qa_ids[:5]

['WP-24_CR', 'WP-155_SR', 'WP-27', 'WP-27_CR', 'WP-138_CR']

In [ ]:
dev_qa_ids[:5]

['WP-66', 'WP-24_SR', 'WP-104_SR', 'WP-115_SR', 'WP-45_SR']

In [ ]:
# import random
# SEED = 42

# random.seed(SEED)
# random.shuffle(all_qa_ids)

# print(all_qa_ids[:5])

In [ ]:
# train_qa_ids_split_upper_idx=int(len(all_qa_ids)*0.9)
# train_qa_ids_split_upper_idx

In [ ]:
# train_qa_ids=all_qa_ids[:train_qa_ids_split_upper_idx]
# dev_qa_ids=all_qa_ids[train_qa_ids_split_upper_idx:]
# print(len(train_qa_ids), len(dev_qa_ids))

In [ ]:
# folder=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds"

# with open(f'{folder}/WP_inhouse_split_qids.txt', 'w') as outfile:
#   for entry in train_qa_ids:
#       json.dump(entry, outfile)
#       outfile.write('\n')

In [ ]:
'WP-16_SR' in train_qa_ids

True

In [ ]:
'WP-16_SR' in dev_qa_ids

False

In [ ]:
dataset="brain_teaser_ds"
folder=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds"


# train_graph=f"{folder}/graph/original_SP_train.graph.adj.pk"
train_graph=f"{folder}/graph/original_WP_train.graph.adj.pk"

# train_statements=f"{folder}/statement/original_SP_train.statement.jsonl"
train_statements=f"{folder}/statement/original_WP_train.statement.jsonl"

# train_grounded=f"{folder}/grounded/original_SP_train.grounded.jsonl"
train_grounded=f"{folder}/grounded/original_WP_train.grounded.jsonl"

In [ ]:
train_grounded

'/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/grounded/original_WP_train.grounded.jsonl'

In [ ]:
train_statements_examples=[]
with open(train_statements, "r", encoding="utf-8") as f:
  for line in f.readlines():
      json_dic = json.loads(line)
      train_statements_examples.append(json_dic)

train_grounded_examples=[]
with open(train_grounded, "r", encoding="utf-8") as f:
  for line in f.readlines():
      json_dic = json.loads(line)
      train_grounded_examples.append(json_dic)

with open(train_graph, 'rb') as fin:
   train_graph_adj_concept_pairs = pickle.load(fin)

In [ ]:
len(train_statements_examples), len(train_grounded_examples), len(train_graph_adj_concept_pairs)

(396, 1584, 1584)

In [ ]:
new_train_statements=[]
new_dev_statements=[]
new_train_grounded=[]
new_dev_grounded=[]
new_train_graph=[]
new_dev_graph=[]

for i in range(len(train_statements_examples)):
  sample=train_statements_examples[i]
  id=sample['id']
  try:
    if id in train_qa_ids:
      new_train_statements.append(sample)
      idx=i*4
      for j in range(4):
        new_train_grounded.append(train_grounded_examples[idx+j])
        new_train_graph.append(train_graph_adj_concept_pairs[idx+j])
    else:
      new_dev_statements.append(sample)
      dx=i*4
      for j in range(4):
        new_dev_grounded.append(train_grounded_examples[idx+j])
        new_dev_graph.append(train_graph_adj_concept_pairs[idx+j])
  except:
    print(i)


In [ ]:
len(new_dev_graph)//4, len(new_train_graph)//4

(40, 356)

In [ ]:
len(new_train_statements),len(new_dev_statements),len(new_train_grounded), len(new_dev_grounded), len(new_train_graph), len(new_dev_graph)

(356, 40, 1424, 160, 1424, 160)

In [ ]:
dataset="brain_teaser_ds"
folder=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds"


# train_graph_path=f"{folder}/graph/SP_train.graph.adj.pk"
# dev_graph_path=f"{folder}/graph/SP_dev.graph.adj.pk"

# train_statements_path=f"{folder}/statement/SP_train.statement.jsonl"
# dev_statements_path=f"{folder}/statement/SP_dev.statement.jsonl"

# train_grounded_path=f"{folder}/grounded/SP_train.grounded.jsonl"
# dev_grounded_path=f"{folder}/grounded/SP_dev.grounded.jsonl"

train_graph_path=f"{folder}/graph/WP_train.graph.adj.pk"
dev_graph_path=f"{folder}/graph/WP_dev.graph.adj.pk"

train_statements_path=f"{folder}/statement/WP_train.statement.jsonl"
dev_statements_path=f"{folder}/statement/WP_dev.statement.jsonl"

train_grounded_path=f"{folder}/grounded/WP_train.grounded.jsonl"
dev_grounded_path=f"{folder}/grounded/WP_dev.grounded.jsonl"

Saving train data

In [ ]:
with open(train_graph_path, 'wb') as f:
  pickle.dump(new_train_graph, f)

In [ ]:
with open(train_grounded_path, 'w') as outfile:
  for entry in new_train_grounded:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
with open(train_statements_path, 'w') as outfile:
  for entry in new_train_statements:
      json.dump(entry, outfile)
      outfile.write('\n')

Saving dev data

In [ ]:
with open(dev_graph_path, 'wb') as f:
  pickle.dump(new_dev_graph, f)

In [ ]:
with open(dev_grounded_path, 'w') as outfile:
  for entry in new_dev_grounded:
      json.dump(entry, outfile)
      outfile.write('\n')

In [ ]:
with open(dev_statements_path, 'w') as outfile:
  for entry in new_dev_statements:
      json.dump(entry, outfile)
      outfile.write('\n')

####temp

In [ ]:
# def extract_concepts(q_stem):

#   question_concepts=[]
#   for i in range(len(id2concept)):
#     concept=id2concept[i].strip().lower().replace("_"," ")
#     # if concept in most_common_concepts or len(concept)<=3:
#     if len(concept)<=3:
#       continue
#     if concept in q_stem:
#       question_concepts.append(concept)


#   new_question_concepts=[]
#   for i in range(len(question_concepts)):
#     first_concept=question_concepts[i]
#     is_sub_concept=False
#     for j in range(len(question_concepts)):
#       if i==j:
#         continue
#       if first_concept in question_concepts[j]:
#         is_sub_concept=True
#         break

#     if is_sub_concept==False:
#       new_question_concepts.append(first_concept)

#   meaningful_question_concepts=set()
#   for concept in new_question_concepts:
#     if concept in wn.words():
#       meaningful_question_concepts.add(concept)
#       continue

#     concept_subwords=concept.split(" ")
#     all_are_meaningful=True
#     meaningfuls=[]
#     for subword in concept_subwords:
#       if len(subword)<=3:
#         all_are_meaningful=False
#         continue

#       if subword in wn.words():
#         meaningfuls.append(subword)
#       else:
#         all_are_meaningful=False
#     if all_are_meaningful:
#       meaningful_question_concepts.add(concept)
#     else:
#       for c in meaningfuls:
#         meaningful_question_concepts.add(c)

#   meaningful_question_concepts=list(meaningful_question_concepts)
#   final_concepts=[]
#   for concept in meaningful_question_concepts:
#     if concept in most_common_concepts:
#       continue
#     else:
#       final_concepts.append(concept)

#   return final_concepts
#   # return list(meaningful_question_concepts)


# def preprocess_qa(question_list, is_train_data):

#   processed_qs=[]
#   id_to_label={0:"A", 1:"B", 2:"C", 3:"D"}

#   for i in tqdm(range(len(question_list))):

#     sample=question_list.iloc[i]
#     q_stem=sample['question'].strip().lower()
#     q_choices_list=sample['choice_list']
#     if is_train_data:
#       q_answerKey=id_to_label[sample['label']]
#       q_id=sample['id']
#     else:
#       q_id=i


#     meaningful_question_concepts = extract_concepts(q_stem)

#     q_choices=[]

#     for j in range(len(q_choices_list)):
#       if j==0:
#         choice_label="A"
#       elif j==1:
#         choice_label="B"
#       elif j==2:
#         choice_label="C"
#       else:
#         choice_label="D"

#       choice_text=q_choices_list[j]
#       choice_dict={
#           'label': choice_label,
#           'text': choice_text.strip().lower()
#       }
#       meaningful_choice_concepts = extract_concepts(choice_dict['text'])

#       choice_dict['choice_concepts']= meaningful_choice_concepts
#       q_choices.append(choice_dict)

#     if is_train_data:
#       q_dict={
#             'id':q_id,
#             'answerKey':q_answerKey,
#             'question':{
#                 'question_concept': meaningful_question_concepts,
#                 'choices': q_choices,
#                 'stem': q_stem,
#             }
#         }
#     else:
#         q_dict={
#             'id':q_id,
#             'question':{
#                 'question_concept': meaningful_question_concepts,
#                 'choices': q_choices,
#                 'stem': q_stem,
#             }
#         }
#     processed_qs.append(q_dict)
#     break
#   return processed_qs

####test on a few samples

In [ ]:
processed_qs=extract_concepts(riddle_sense_temp_q_list)

100%|██████████| 3/3 [00:03<00:00,  1.08s/it]


In [ ]:
processed_qs[0]['question'].keys()

dict_keys(['question_concept', 'choices', 'stem'])

In [ ]:
print(processed_qs[0]['question']['stem'])

a man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  what is the one thing he can put in it to make it lighter?


In [ ]:
processed_qs[0]['question']['question_concept']

['tonne',
 'in prison',
 'bag',
 'can',
 'football pitch',
 'lighter',
 'he',
 'punishment',
 'size',
 'across',
 'sand',
 'backwards',
 'forwards',
 'put in',
 'man',
 'field',
 'make it',
 'in it',
 'carry',
 'one thing']

In [ ]:
processed_qs[1]['question']['stem']

'what gets smaller as it gets fuller?'

In [ ]:
processed_qs[1]['question']['question_concept']

['fuller', 'smaller', 'it', 'as']

In [ ]:
processed_qs[2]['question']['stem']

'whats weightless, visible to the naked eye, and when you put it in a barrel of water it will make the barrel lighter in weight. ?'

In [ ]:
processed_qs[2]['question']['question_concept']

['lighter',
 'weightless',
 'visible',
 'put it',
 'barrel',
 'water',
 'it in',
 'put',
 'naked eye',
 'will',
 'water it',
 'make']

####**obqa**

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': convert_to_obqa_statement, 'args': (input_paths['obqa']['train'], output_paths['obqa']['statement']['train'], output_paths['obqa']['statement']['train-fairseq'])},
            {'func': convert_to_obqa_statement, 'args': (input_paths['obqa']['dev'], output_paths['obqa']['statement']['dev'], output_paths['obqa']['statement']['dev-fairseq'])},
            {'func': convert_to_obqa_statement, 'args': (input_paths['obqa']['test'], output_paths['obqa']['statement']['test'], output_paths['obqa']['statement']['test-fairseq'])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': ground, 'args': (output_paths['obqa']['statement']['train'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['train'], args["nprocs"])},
            {'func': ground, 'args': (output_paths['obqa']['statement']['dev'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['dev'], args["nprocs"])},
            {'func': ground, 'args': (output_paths['obqa']['statement']['test'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['test'], args["nprocs"])},
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['train'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-train'], args["nprocs"])},
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['dev'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-dev'], args["nprocs"])},
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['test'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-test'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': ground, 'args': (output_paths['obqa']['statement']['train'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['train'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': ground, 'args': (output_paths['obqa']['statement']['dev'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['dev'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': ground, 'args': (output_paths['obqa']['statement']['test'], output_paths['cpnet']['vocab'],
                                      output_paths['cpnet']['patterns'], output_paths['obqa']['grounded']['test'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['train'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-train'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['dev'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-dev'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

In [ ]:
def main_3():
    args=dict()
    args["run"]=["obqa"]
    args["path_prune_threshold"]=0.12
    args["max_node_num"]=200
    args["nprocs"]=cpu_count()
    args["seed"]=0
    args["debug"]=False

    routines = {

        'obqa': [
            {'func': generate_adj_data_from_grounded_concepts__use_LM, 'args': (output_paths['obqa']['grounded']['test'], output_paths['cpnet']['pruned-graph'], output_paths['cpnet']['vocab'], output_paths['obqa']['graph']['adj-test'], args["nprocs"])},
        ],
    }

    for rt in args["run"]:
        print(rt)
        for rt_dic in routines[rt]:
            print(rt_dic)
            rt_dic['func'](*rt_dic['args'])

    print('Successfully run {}'.format(' '.join(args["run"])))


main_3()

#Train QA-GNN

###from modeling.modeling_qagnn import *

####from utils.utils import freeze_net


In [4]:
import json
import os
import time
import argparse


def bool_flag(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def check_path(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)


def check_file(file):
    return os.path.isfile(file)


def export_config(config, path):
    # param_dict = dict(vars(config))
    param_dict = config
    check_path(path)
    with open(path, 'w') as fout:
        json.dump(param_dict, fout, indent=4)


def freeze_net(module):
    for p in module.parameters():
        p.requires_grad = False


def unfreeze_net(module):
    for p in module.parameters():
        p.requires_grad = True


def test_data_loader_ms_per_batch(data_loader, max_steps=10000):
    start = time.time()
    n_batch = sum(1 for batch, _ in zip(data_loader, range(max_steps)))
    return (time.time() - start) * 1000 / n_batch


####from utils.layers import *

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import math
# from utils.utils import freeze_net


def gelu(x):
    """ Implementation of the gelu activation function currently in Google Bert repo (identical to OpenAI GPT).
        Also see https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return gelu(x)


class TypedLinear(nn.Linear):
    def __init__(self, in_features, out_features, n_type):
        super().__init__(in_features, n_type * out_features)
        self.in_features = in_features
        self.out_features = out_features
        self.n_type = n_type

    def forward(self, X, type_ids=None):
        """
        X: tensor of shape (*, in_features)
        type_ids: long tensor of shape (*)
        """
        output = super().forward(X)
        if type_ids is None:
            return output
        output_shape = output.size()[:-1] + (self.out_features,)
        output = output.view(-1, self.n_type, self.out_features)
        idx = torch.arange(output.size(0), dtype=torch.long, device=type_ids.device)
        output = output[idx, type_ids.view(-1)].view(*output_shape)
        return output


class MLP(nn.Module):
    """
    Multi-layer perceptron

    Parameters
    ----------
    num_layers: number of hidden layers
    """
    activation_classes = {'gelu': GELU, 'relu': nn.ReLU, 'tanh': nn.Tanh}

    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout, batch_norm=False,
                 init_last_layer_bias_to_zero=False, layer_norm=False, activation='gelu'):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.layer_norm = layer_norm

        assert not (self.batch_norm and self.layer_norm)    # assert statement has a condition or expression which is supposed to be always true

        self.layers = nn.Sequential()
        for i in range(self.num_layers + 1):
            n_in = self.input_size if i == 0 else self.hidden_size
            n_out = self.hidden_size if i < self.num_layers else self.output_size
            self.layers.add_module(f'{i}-Linear', nn.Linear(n_in, n_out))
            if i < self.num_layers:
                self.layers.add_module(f'{i}-Dropout', nn.Dropout(self.dropout))
                if self.batch_norm:
                    self.layers.add_module(f'{i}-BatchNorm1d', nn.BatchNorm1d(self.hidden_size))
                if self.layer_norm:
                    self.layers.add_module(f'{i}-LayerNorm', nn.LayerNorm(self.hidden_size))
                self.layers.add_module(f'{i}-{activation}', self.activation_classes[activation.lower()]())
        if init_last_layer_bias_to_zero:
            self.layers[-1].bias.data.fill_(0)

    def forward(self, input):
        return self.layers(input)


class MaxPoolLayer(nn.Module):
    """
    A layer that performs max pooling along the sequence dimension
    """

    def __init__(self):
        super().__init__()

    def forward(self, inputs, mask_or_lengths):
        """
        inputs: tensor of shape (batch_size, seq_len, hidden_size)
        mask_or_lengths: tensor of shape (batch_size) or (batch_size, seq_len)

        returns: tensor of shape (batch_size, hidden_size)
        """
        bs, sl, _ = inputs.size()
        if len(mask_or_lengths.size()) == 1:
            mask = (torch.arange(sl, device=inputs.device).unsqueeze(0).expand(bs, sl) >= mask_or_lengths.unsqueeze(1))
        else:
            mask = mask_or_lengths
        masked_inputs = inputs.masked_fill(mask.unsqueeze(-1).expand_as(inputs), float('-inf'))
        max_pooled = masked_inputs.max(1)[0]
        return max_pooled


class MeanPoolLayer(nn.Module):
    """
    A layer that performs mean pooling along the sequence dimension
    """

    def __init__(self):
        super().__init__()

    def forward(self, inputs, mask_or_lengths):
        """
        inputs: tensor of shape (batch_size, seq_len, hidden_size)
        mask_or_lengths: tensor of shape (batch_size) or (batch_size, seq_len)

        returns: tensor of shape (batch_size, hidden_size)
        """
        bs, sl, _ = inputs.size()
        if len(mask_or_lengths.size()) == 1:
            mask = (torch.arange(sl, device=inputs.device).unsqueeze(0).expand(bs, sl) >= mask_or_lengths.unsqueeze(1))
            lengths = mask_or_lengths.float()
        else:
            mask, lengths = mask_or_lengths, (1 - mask_or_lengths.float()).sum(1)
        masked_inputs = inputs.masked_fill(mask.unsqueeze(-1).expand_as(inputs), 0.0)
        mean_pooled = masked_inputs.sum(1) / lengths.unsqueeze(-1)
        return mean_pooled


def dropout_mask(x, sz, p: float):
    """
    Return a dropout mask of the same type as `x`, size `sz`, with probability `p` to cancel an element.

    (adapted from https://github.com/fastai/fastai/blob/1.0.42/fastai/text/models/awd_lstm.py)
    """
    return x.new(*sz).bernoulli_(1 - p).div_(1 - p)


class EmbeddingDropout(nn.Module):
    """
    Apply dropout with probabily `embed_p` to an embedding layer `emb`.

    (adapted from https://github.com/fastai/fastai/blob/1.0.42/fastai/text/models/awd_lstm.py)
    """

    def __init__(self, emb: nn.Module, embed_p: float):
        super().__init__()
        self.emb, self.embed_p = emb, embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None:
            self.pad_idx = -1

    def forward(self, words):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0), 1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else:
            masked_embed = self.emb.weight
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)


class RNNDropout(nn.Module):
    "Dropout with probability `p` that is consistent on the seq_len dimension."

    def __init__(self, p: float = 0.5):
        super().__init__()
        self.p = p

    def forward(self, x):
        if not self.training or self.p == 0.:
            return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m


class LSTMEncoder(nn.Module):

    def __init__(self, vocab_size=300, emb_size=300, hidden_size=300, num_layers=2, bidirectional=True,
                 emb_p=0, input_p=0, hidden_p=0, output_p=0, pretrained_emb=None, pooling=True, pad=False):
        super().__init__()
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.emb_p = emb_p
        self.input_p = input_p
        self.hidden_p = hidden_p
        self.output_p = output_p
        self.pooling = pooling

        self.emb = EmbeddingDropout(nn.Embedding(vocab_size, emb_size), emb_p)
        if pretrained_emb is not None:
            self.emb.emb.weight.data.copy_(pretrained_emb)
        else:
            bias = np.sqrt(6.0 / emb_size)
            nn.init.uniform_(self.emb.emb.weight, -bias, bias)
        self.input_dropout = nn.Dropout(input_p)
        self.output_dropout = nn.Dropout(output_p)
        self.rnn = nn.LSTM(input_size=emb_size, hidden_size=(hidden_size // 2 if self.bidirectional else hidden_size),
                           num_layers=num_layers, dropout=hidden_p, bidirectional=bidirectional,
                           batch_first=True)
        self.max_pool = MaxPoolLayer()

    def forward(self, inputs, lengths):
        """
        inputs: tensor of shape (batch_size, seq_len)
        lengths: tensor of shape (batch_size)

        returns: tensor of shape (batch_size, hidden_size)
        """
        bz, full_length = inputs.size()
        embed = self.emb(inputs)
        embed = self.input_dropout(embed)
        lstm_inputs = pack_padded_sequence(embed, lengths, batch_first=True, enforce_sorted=False)
        rnn_outputs, _ = self.rnn(lstm_inputs)
        rnn_outputs, _ = pad_packed_sequence(rnn_outputs, batch_first=True, total_length=full_length)
        rnn_outputs = self.output_dropout(rnn_outputs)
        return self.max_pool(rnn_outputs, lengths) if self.pooling else rnn_outputs


class TripleEncoder(nn.Module):
    def __init__(self, emb_dim, hidden_dim, input_p, output_p, hidden_p, num_layers, bidirectional=True, pad=False,
                 concept_emb=None, relation_emb=None
                 ):
        super().__init__()
        if pad:
            raise NotImplementedError
        self.input_p = input_p
        self.output_p = output_p
        self.hidden_p = hidden_p
        self.cpt_emb = concept_emb
        self.rel_emb = relation_emb
        self.input_dropout = nn.Dropout(input_p)
        self.output_dropout = nn.Dropout(output_p)
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(input_size=emb_dim, hidden_size=(hidden_dim // 2 if self.bidirectional else hidden_dim),
                          num_layers=num_layers, dropout=hidden_p, bidirectional=bidirectional,
                          batch_first=True)

    def forward(self, inputs):
        '''
        inputs: (batch_size, seq_len)

        returns: (batch_size, h_dim(*2))
        '''
        bz, sl = inputs.size()
        h, r, t = torch.chunk(inputs, 3, dim=1)  # (bz, 1)

        h, t = self.input_dropout(self.cpt_emb(h)), self.input_dropout(self.cpt_emb(t))  # (bz, 1, dim)
        r = self.input_dropout(self.rel_emb(r))
        inputs = torch.cat((h, r, t), dim=1)  # (bz, 3, dim)
        rnn_outputs, _ = self.rnn(inputs)  # (bz, 3, dim)
        if self.bidirectional:
            outputs_f, outputs_b = torch.chunk(rnn_outputs, 2, dim=2)
            outputs = torch.cat((outputs_f[:, -1, :], outputs_b[:, 0, :]), 1)  # (bz, 2 * h_dim)
        else:
            outputs = rnn_outputs[:, -1, :]

        return self.output_dropout(outputs)


class MatrixVectorScaledDotProductAttention(nn.Module):

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, q, k, v, mask=None):
        """
        q: tensor of shape (n*b, d_k)
        k: tensor of shape (n*b, l, d_k)
        v: tensor of shape (n*b, l, d_v)

        returns: tensor of shape (n*b, d_v), tensor of shape(n*b, l)
        """
        attn = (q.unsqueeze(1) * k).sum(2)  # (n*b, l)
        attn = attn / self.temperature
        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = (attn.unsqueeze(2) * v).sum(1)
        return output, attn


class AttPoolLayer(nn.Module):

    def __init__(self, d_q, d_k, dropout=0.1):
        super().__init__()
        self.w_qs = nn.Linear(d_q, d_k)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_q + d_k)))
        self.attention = MatrixVectorScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, mask=None):
        """
        q: tensor of shape (b, d_q)
        k: tensor of shape (b, l, d_k)
        mask: tensor of shape (b, l) (optional, default None)
        returns: tensor of shape (b, d_k)
        """
        qs = self.w_qs(q)  # (b, d_k)
        output, attn = self.attention(qs, k, k, mask=mask)
        output = self.dropout(output)
        return output, attn


class MultiheadAttPoolLayer(nn.Module):

    def __init__(self, n_head, d_q_original, d_k_original, dropout=0.1):
        super().__init__()
        assert d_k_original % n_head == 0  # make sure the outpute dimension equals to d_k_origin
        self.n_head = n_head
        self.d_k = d_k_original // n_head
        self.d_v = d_k_original // n_head

        self.w_qs = nn.Linear(d_q_original, n_head * self.d_k)
        self.w_ks = nn.Linear(d_k_original, n_head * self.d_k)
        self.w_vs = nn.Linear(d_k_original, n_head * self.d_v)

        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_q_original + self.d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_k_original + self.d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_k_original + self.d_v)))

        self.attention = MatrixVectorScaledDotProductAttention(temperature=np.power(self.d_k, 0.5))
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, mask=None):
        """
        q: tensor of shape (b, d_q_original)
        k: tensor of shape (b, l, d_k_original)
        mask: tensor of shape (b, l) (optional, default None)
        returns: tensor of shape (b, n*d_v)
        """
        n_head, d_k, d_v = self.n_head, self.d_k, self.d_v

        bs, _ = q.size()
        bs, len_k, _ = k.size()

        qs = self.w_qs(q).view(bs, n_head, d_k)  # (b, n, dk)
        ks = self.w_ks(k).view(bs, len_k, n_head, d_k)  # (b, l, n, dk)
        vs = self.w_vs(k).view(bs, len_k, n_head, d_v)  # (b, l, n, dv)

        qs = qs.permute(1, 0, 2).contiguous().view(n_head * bs, d_k)
        ks = ks.permute(2, 0, 1, 3).contiguous().view(n_head * bs, len_k, d_k)
        vs = vs.permute(2, 0, 1, 3).contiguous().view(n_head * bs, len_k, d_v)

        if mask is not None:
            mask = mask.repeat(n_head, 1)
        output, attn = self.attention(qs, ks, vs, mask=mask)

        output = output.view(n_head, bs, d_v)
        output = output.permute(1, 0, 2).contiguous().view(bs, n_head * d_v)  # (b, n*dv)
        output = self.dropout(output)
        return output, attn


class TypedMultiheadAttPoolLayer(nn.Module):

    def __init__(self, n_head, d_q_original, d_k_original, dropout=0.1, n_type=1):
        super().__init__()
        assert d_k_original % n_head == 0  # make sure the outpute dimension equals to d_k_origin
        self.n_head = n_head
        self.d_k = d_k_original // n_head
        self.d_v = d_k_original // n_head

        self.w_qs = nn.Linear(d_q_original, n_head * self.d_k)
        self.w_ks = TypedLinear(d_k_original, n_head * self.d_k, n_type)
        self.w_vs = TypedLinear(d_k_original, n_head * self.d_v, n_type)

        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_q_original + self.d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_k_original + self.d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_k_original + self.d_v)))

        self.attention = MatrixVectorScaledDotProductAttention(temperature=np.power(self.d_k, 0.5))
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, mask=None, type_ids=None):
        """
        q: tensor of shape (b, d_q_original)
        k: tensor of shape (b, l, d_k_original)
        mask: bool tensor of shape (b, l) (optional, default None)
        type_ids: long tensor of shape (b, l) (optional, default None)
        returns: tensor of shape (b, n*d_v)
        """
        n_head, d_k, d_v = self.n_head, self.d_k, self.d_v

        bs, _ = q.size()
        bs, len_k, _ = k.size()

        qs = self.w_qs(q).view(bs, n_head, d_k)  # (b, n, dk)
        ks = self.w_ks(k, type_ids=type_ids).view(bs, len_k, n_head, d_k)  # (b, l, n, dk)
        vs = self.w_vs(k, type_ids=type_ids).view(bs, len_k, n_head, d_v)  # (b, l, n, dv)

        qs = qs.permute(1, 0, 2).contiguous().view(n_head * bs, d_k)
        ks = ks.permute(2, 0, 1, 3).contiguous().view(n_head * bs, len_k, d_k)
        vs = vs.permute(2, 0, 1, 3).contiguous().view(n_head * bs, len_k, d_v)

        if mask is not None:
            mask = mask.repeat(n_head, 1)
        output, attn = self.attention(qs, ks, vs, mask=mask)

        output = output.view(n_head, bs, d_v)
        output = output.permute(1, 0, 2).contiguous().view(bs, n_head * d_v)  # (b, n*dv)
        output = self.dropout(output)
        return output, attn


class BilinearAttentionLayer(nn.Module):

    def __init__(self, query_dim, value_dim):
        super().__init__()
        self.linear = nn.Linear(value_dim, query_dim, bias=False)
        self.softmax = nn.Softmax(1)

    def forward(self, query, value, node_mask=None):
        """
        query: tensor of shape (batch_size, query_dim)
        value: tensor of shape (batch_size, seq_len, value_dim)
        node_mask: tensor of shape (batch_size, seq_len)

        returns: tensor of shape (batch_size, value_dim)
        """
        attn = self.linear(value).bmm(query.unsqueeze(-1))
        attn = self.softmax(attn.squeeze(-1))
        if node_mask is not None:
            attn = attn * node_mask
            attn = attn / attn.sum(1, keepdim=True)
        pooled = attn.unsqueeze(1).bmm(value).squeeze(1)
        return pooled, attn


def masked_softmax(vector: torch.Tensor,
                   mask: torch.Tensor,
                   dim: int = -1,
                   memory_efficient: bool = True,
                   mask_fill_value: float = -1e32) -> torch.Tensor:
    """
    ``torch.nn.functional.softmax(vector)`` does not work if some elements of ``vector`` should be
    masked.  This performs a softmax on just the non-masked portions of ``vector``.  Passing
    ``None`` in for the mask is also acceptable; you'll just get a regular softmax.
    ``vector`` can have an arbitrary number of dimensions; the only requirement is that ``mask`` is
    broadcastable to ``vector's`` shape.  If ``mask`` has fewer dimensions than ``vector``, we will
    unsqueeze on dimension 1 until they match.  If you need a different unsqueezing of your mask,
    do it yourself before passing the mask into this function.
    If ``memory_efficient`` is set to true, we will simply use a very large negative number for those
    masked positions so that the probabilities of those positions would be approximately 0.
    This is not accurate in math, but works for most cases and consumes less memory.
    In the case that the input vector is completely masked and ``memory_efficient`` is false, this function
    returns an array of ``0.0``. This behavior may cause ``NaN`` if this is used as the last layer of
    a model that uses categorical cross-entropy loss. Instead, if ``memory_efficient`` is true, this function
    will treat every element as equal, and do softmax over equal numbers.
    """
    if mask is None:
        result = nn.functional.softmax(vector, dim=dim)
    else:
        mask = mask.float()
        while mask.dim() < vector.dim():
            mask = mask.unsqueeze(1)
        if not memory_efficient:
            # # To limit numerical errors from large vector elements outside the mask, we zero these out.
            # result = nn.functional.softmax(vector * mask, dim=dim)
            # result = result * mask
            # result = result / (result.sum(dim=dim, keepdim=True) + 1e-13)
            raise NotImplementedError
        else:
            masked_vector = vector.masked_fill(mask.to(dtype=torch.uint8), mask_fill_value)
            result = nn.functional.softmax(masked_vector, dim=dim)
            result = result * (1 - mask)
    return result


class DiffTopK(torch.autograd.Function):

    @staticmethod
    def forward(ctx, x, k):
        """
        x: tensor of shape (batch_size, n_node)
        k: int
        returns: tensor of shape (batch_size, n_node)
        """
        bs, _ = x.size()
        _, topk_indexes = x.topk(k, 1)  # (batch_size, k)
        output = x.new_zeros(x.size())
        ri = torch.arange(bs).unsqueeze(1).expand(bs, k).contiguous().view(-1)
        output[ri, topk_indexes.view(-1)] = 1
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.clone(), None


class SimilarityFunction(nn.Module):
    """
    A ``SimilarityFunction`` takes a pair of tensors with the same shape, and computes a similarity
    function on the vectors in the last dimension.  For example, the tensors might both have shape
    `(batch_size, sentence_length, embedding_dim)`, and we will compute some function of the two
    vectors of length `embedding_dim` for each position `(batch_size, sentence_length)`, returning a
    tensor of shape `(batch_size, sentence_length)`.
    The similarity function could be as simple as a dot product, or it could be a more complex,
    parameterized function.
    """
    default_implementation = 'dot_product'

    def forward(self, tensor_1: torch.Tensor, tensor_2: torch.Tensor) -> torch.Tensor:
        """
        Takes two tensors of the same shape, such as ``(batch_size, length_1, length_2,
        embedding_dim)``.  Computes a (possibly parameterized) similarity on the final dimension
        and returns a tensor with one less dimension, such as ``(batch_size, length_1, length_2)``.
        """
        raise NotImplementedError


class DotProductSimilarity(SimilarityFunction):
    """
    This similarity function simply computes the dot product between each pair of vectors, with an
    optional scaling to reduce the variance of the output elements.
    Parameters
    ----------
    scale_output : ``bool``, optional
        If ``True``, we will scale the output by ``math.sqrt(tensor.size(-1))``, to reduce the
        variance in the result.
    """

    def __init__(self, scale_output: bool = False) -> None:
        super(DotProductSimilarity, self).__init__()
        self._scale_output = scale_output

    def forward(self, tensor_1: torch.Tensor, tensor_2: torch.Tensor) -> torch.Tensor:
        result = (tensor_1 * tensor_2).sum(dim=-1)
        if self._scale_output:
            result *= math.sqrt(tensor_1.size(-1))
        return result


class MatrixAttention(nn.Module):
    def __init__(self, similarity_function: SimilarityFunction = None) -> None:
        super().__init__()
        self._similarity_function = similarity_function or DotProductSimilarity()

    def forward(self, matrix_1: torch.Tensor, matrix_2: torch.Tensor) -> torch.Tensor:
        tiled_matrix_1 = matrix_1.unsqueeze(2).expand(matrix_1.size()[0],
                                                      matrix_1.size()[1],
                                                      matrix_2.size()[1],
                                                      matrix_1.size()[2])
        tiled_matrix_2 = matrix_2.unsqueeze(1).expand(matrix_2.size()[0],
                                                      matrix_1.size()[1],
                                                      matrix_2.size()[1],
                                                      matrix_2.size()[2])

        return self._similarity_function(tiled_matrix_1, tiled_matrix_2)


class CustomizedEmbedding(nn.Module):
    def __init__(self, concept_num, concept_in_dim, concept_out_dim, use_contextualized=False,
                 pretrained_concept_emb=None, freeze_ent_emb=True, scale=1.0, init_range=0.02):
        super().__init__()
        self.scale = scale
        self.use_contextualized = use_contextualized
        if not use_contextualized:
            self.emb = nn.Embedding(concept_num, concept_in_dim)
            if pretrained_concept_emb is not None:
                self.emb.weight.data.copy_(pretrained_concept_emb)
            else:
                self.emb.weight.data.normal_(mean=0.0, std=init_range)
            if freeze_ent_emb:
                freeze_net(self.emb)

        if concept_in_dim != concept_out_dim:
            self.cpt_transform = nn.Linear(concept_in_dim, concept_out_dim)
            self.activation = GELU()

    def forward(self, index, contextualized_emb=None):
        """
        index: size (bz, a)
        contextualized_emb: size (bz, b, emb_size) (optional)
        """
        if contextualized_emb is not None:
            assert index.size(0) == contextualized_emb.size(0)
            if hasattr(self, 'cpt_transform'):
                contextualized_emb = self.activation(self.cpt_transform(contextualized_emb * self.scale))
            else:
                contextualized_emb = contextualized_emb * self.scale
            emb_dim = contextualized_emb.size(-1)
            return contextualized_emb.gather(1, index.unsqueeze(-1).expand(-1, -1, emb_dim))
        else:
            if hasattr(self, 'cpt_transform'):
                return self.activation(self.cpt_transform(self.emb(index) * self.scale))
            else:
                return self.emb(index) * self.scale


def run_test():
    print('testing BilinearAttentionLayer...')
    att = BilinearAttentionLayer(100, 20)
    mask = (torch.randn(70, 30) > 0).float()
    mask.requires_grad_()
    v = torch.randn(70, 30, 20)
    q = torch.randn(70, 100)
    o, _ = att(q, v, mask)
    o.sum().backward()
    print(mask.grad)

    print('testing DiffTopK...')
    x = torch.randn(5, 3)
    x.requires_grad_()
    k = 2
    r = DiffTopK.apply(x, k)
    loss = (r ** 2).sum()
    loss.backward()
    assert (x.grad == r * 2).all()
    print('pass')

    a = TripleEncoder()

    triple_input = torch.tensor([[1, 2, 3], [4, 5, 6]])
    res = a(triple_input)
    print(res.size())

    b = LSTMEncoder(pooling=False)
    lstm_inputs = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]])
    lengths = torch.tensor([3, 2])
    res = b(lstm_inputs, lengths)
    print(res.size())


####from utils.data_utils import get_gpt_token_num

In [6]:
import pickle
import os
import numpy as np
import torch
from transformers import (OpenAIGPTTokenizer, BertTokenizer, XLNetTokenizer, RobertaTokenizer, AutoTokenizer)
try:
    from transformers import AlbertTokenizer
except:
    pass

import json
from tqdm import tqdm

GPT_SPECIAL_TOKENS = ['_start_', '_delimiter_', '_classify_']


class MultiGPUSparseAdjDataBatchGenerator(object):
    def __init__(self, args, mode, device0, device1, batch_size, indexes, qids, labels,
                 tensors0=[], lists0=[], tensors1=[], lists1=[], adj_data=None):
        self.args = args
        self.mode = mode
        self.device0 = device0
        self.device1 = device1
        self.batch_size = batch_size
        self.indexes = indexes
        self.qids = qids
        self.labels = labels
        self.tensors0 = tensors0
        self.lists0 = lists0
        self.tensors1 = tensors1
        self.lists1 = lists1
        # self.adj_empty = adj_empty.to(self.device1)
        self.adj_data = adj_data

    def __len__(self):
        return (self.indexes.size(0) - 1) // self.batch_size + 1

    def __iter__(self):
        bs = self.batch_size
        n = self.indexes.size(0)
        if self.mode=='train' and self.args["drop_partial_batch"]:
            print ('dropping partial batch')
            n = (n//bs) *bs
        elif self.mode=='train' and self.args["fill_partial_batch"]:
            print ('filling partial batch')
            remain = n % bs
            if remain > 0:
                extra = np.random.choice(self.indexes[:-remain], size=(bs-remain), replace=False)
                self.indexes = torch.cat([self.indexes, torch.tensor(extra)])
                n = self.indexes.size(0)
                assert n % bs == 0

        for a in range(0, n, bs):
            b = min(n, a + bs)
            batch_indexes = self.indexes[a:b]
            batch_qids = [self.qids[idx] for idx in batch_indexes]
            batch_labels = self._to_device(self.labels[batch_indexes], self.device1)
            batch_tensors0 = [self._to_device(x[batch_indexes], self.device0) for x in self.tensors0]
            batch_tensors1 = [self._to_device(x[batch_indexes], self.device1) for x in self.tensors1]
            batch_lists0 = [self._to_device([x[i] for i in batch_indexes], self.device0) for x in self.lists0]
            batch_lists1 = [self._to_device([x[i] for i in batch_indexes], self.device1) for x in self.lists1]


            edge_index_all, edge_type_all = self.adj_data
            #edge_index_all: nested list of shape (n_samples, num_choice), where each entry is tensor[2, E]
            #edge_type_all:  nested list of shape (n_samples, num_choice), where each entry is tensor[E, ]
            edge_index = self._to_device([edge_index_all[i] for i in batch_indexes], self.device1)
            edge_type  = self._to_device([edge_type_all[i] for i in batch_indexes], self.device1)

            yield tuple([batch_qids, batch_labels, *batch_tensors0, *batch_lists0, *batch_tensors1, *batch_lists1, edge_index, edge_type])

    def _to_device(self, obj, device):
        if isinstance(obj, (tuple, list)):
            return [self._to_device(item, device) for item in obj]
        else:
            return obj.to(device)


def load_sparse_adj_data_with_contextnode(adj_pk_path, max_node_num, num_choice, args):
    cache_path = adj_pk_path +'.loaded_cache'
    use_cache = True

    if use_cache and not os.path.exists(cache_path):
        use_cache = False

    if use_cache:
        with open(cache_path, 'rb') as f:
            adj_lengths_ori, concept_ids, node_type_ids, node_scores, adj_lengths, edge_index, edge_type, half_n_rel = pickle.load(f)
    else:
        with open(adj_pk_path, 'rb') as fin:
            adj_concept_pairs = pickle.load(fin)

        n_samples = len(adj_concept_pairs) #this is actually n_questions x n_choices
        edge_index, edge_type = [], []
        adj_lengths = torch.zeros((n_samples,), dtype=torch.long)
        concept_ids = torch.full((n_samples, max_node_num), 1, dtype=torch.long)
        node_type_ids = torch.full((n_samples, max_node_num), 2, dtype=torch.long) #default 2: "other node"
        node_scores = torch.zeros((n_samples, max_node_num, 1), dtype=torch.float)

        adj_lengths_ori = adj_lengths.clone()
        for idx, _data in tqdm(enumerate(adj_concept_pairs), total=n_samples, desc='loading adj matrices'):
            adj, concepts, qm, am, cid2score = _data['adj'], _data['concepts'], _data['qmask'], _data['amask'], _data['cid2score']
            #adj: e.g. <4233x249 (n_nodes*half_n_rels x n_nodes) sparse matrix of type '<class 'numpy.bool'>' with 2905 stored elements in COOrdinate format>
            #concepts: np.array(num_nodes, ), where entry is concept id
            #qm: np.array(num_nodes, ), where entry is True/False
            #am: np.array(num_nodes, ), where entry is True/False
            assert len(concepts) == len(set(concepts))
            qam = qm | am
            #sanity check: should be T,..,T,F,F,..F
            assert qam[0] == True
            F_start = False
            for TF in qam:
                if TF == False:
                    F_start = True
                else:
                    assert F_start == False
            num_concept = min(len(concepts), max_node_num-1) + 1 #this is the final number of nodes including contextnode but excluding PAD
            adj_lengths_ori[idx] = len(concepts)
            adj_lengths[idx] = num_concept

            #Prepare nodes
            concepts = concepts[:num_concept-1]
            concept_ids[idx, 1:num_concept] = torch.tensor(concepts +1)  #To accomodate contextnode, original concept_ids incremented by 1
            concept_ids[idx, 0] = 0 #this is the "concept_id" for contextnode

            #Prepare node scores
            if (cid2score is not None):
                for _j_ in range(num_concept):
                    _cid = int(concept_ids[idx, _j_]) - 1
                    assert _cid in cid2score
                    node_scores[idx, _j_, 0] = torch.tensor(cid2score[_cid])

            #Prepare node types
            node_type_ids[idx, 0] = 3 #contextnode
            node_type_ids[idx, 1:num_concept][torch.tensor(qm, dtype=torch.bool)[:num_concept-1]] = 0
            node_type_ids[idx, 1:num_concept][torch.tensor(am, dtype=torch.bool)[:num_concept-1]] = 1

            #Load adj
            ij = torch.tensor(adj.row, dtype=torch.int64) #(num_matrix_entries, ), where each entry is coordinate
            k = torch.tensor(adj.col, dtype=torch.int64)  #(num_matrix_entries, ), where each entry is coordinate
            n_node = adj.shape[1]
            half_n_rel = adj.shape[0] // n_node
            i, j = ij // n_node, ij % n_node

            #Prepare edges
            i += 2; j += 1; k += 1  # **** increment coordinate by 1, rel_id by 2 ****
            extra_i, extra_j, extra_k = [], [], []
            for _coord, q_tf in enumerate(qm):
                _new_coord = _coord + 1
                if _new_coord > num_concept:
                    break
                if q_tf:
                    extra_i.append(0) #rel from contextnode to question concept
                    extra_j.append(0) #contextnode coordinate
                    extra_k.append(_new_coord) #question concept coordinate
            for _coord, a_tf in enumerate(am):
                _new_coord = _coord + 1
                if _new_coord > num_concept:
                    break
                if a_tf:
                    extra_i.append(1) #rel from contextnode to answer concept
                    extra_j.append(0) #contextnode coordinate
                    extra_k.append(_new_coord) #answer concept coordinate

            half_n_rel += 2 #should be 19 now
            if len(extra_i) > 0:
                i = torch.cat([i, torch.tensor(extra_i)], dim=0)
                j = torch.cat([j, torch.tensor(extra_j)], dim=0)
                k = torch.cat([k, torch.tensor(extra_k)], dim=0)
            ########################

            mask = (j < max_node_num) & (k < max_node_num)
            i, j, k = i[mask], j[mask], k[mask]
            i, j, k = torch.cat((i, i + half_n_rel), 0), torch.cat((j, k), 0), torch.cat((k, j), 0)  # add inverse relations
            edge_index.append(torch.stack([j,k], dim=0)) #each entry is [2, E]
            edge_type.append(i) #each entry is [E, ]

        with open(cache_path, 'wb') as f:
            pickle.dump([adj_lengths_ori, concept_ids, node_type_ids, node_scores, adj_lengths, edge_index, edge_type, half_n_rel], f)


    ori_adj_mean  = adj_lengths_ori.float().mean().item()
    ori_adj_sigma = np.sqrt(((adj_lengths_ori.float() - ori_adj_mean)**2).mean().item())
    print('| ori_adj_len: mu {:.2f} sigma {:.2f} | adj_len: {:.2f} |'.format(ori_adj_mean, ori_adj_sigma, adj_lengths.float().mean().item()) +
          ' prune_rate： {:.2f} |'.format((adj_lengths_ori > adj_lengths).float().mean().item()) +
          ' qc_num: {:.2f} | ac_num: {:.2f} |'.format((node_type_ids == 0).float().sum(1).mean().item(),
                                                      (node_type_ids == 1).float().sum(1).mean().item()))

    edge_index = list(map(list, zip(*(iter(edge_index),) * num_choice))) #list of size (n_questions, n_choices), where each entry is tensor[2, E] #this operation corresponds to .view(n_questions, n_choices)
    edge_type = list(map(list, zip(*(iter(edge_type),) * num_choice))) #list of size (n_questions, n_choices), where each entry is tensor[E, ]

    concept_ids, node_type_ids, node_scores, adj_lengths = [x.view(-1, num_choice, *x.size()[1:]) for x in (concept_ids, node_type_ids, node_scores, adj_lengths)]
    #concept_ids: (n_questions, num_choice, max_node_num)
    #node_type_ids: (n_questions, num_choice, max_node_num)
    #node_scores: (n_questions, num_choice, max_node_num)
    #adj_lengths: (n_questions,　num_choice)
    return concept_ids, node_type_ids, node_scores, adj_lengths, (edge_index, edge_type) #, half_n_rel * 2 + 1





def load_gpt_input_tensors(statement_jsonl_path, max_seq_length):
    def _truncate_seq_pair(tokens_a, tokens_b, max_length):
        """Truncates a sequence pair in place to the maximum length."""
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

    def load_qa_dataset(dataset_path):
        """ Output a list of tuples(story, 1st continuation, 2nd continuation, label) """
        with open(dataset_path, "r", encoding="utf-8") as fin:
            output = []
            for line in fin:
                input_json = json.loads(line)
                label = ord(input_json.get("answerKey", "A")) - ord("A")
                output.append((input_json['id'], input_json["question"]["stem"], *[ending["text"] for ending in input_json["question"]["choices"]], label))
        return output

    def pre_process_datasets(encoded_datasets, num_choices, max_seq_length, start_token, delimiter_token, clf_token):
        """ Pre-process datasets containing lists of tuples(story, 1st continuation, 2nd continuation, label)

            To Transformer inputs of shape (n_batch, n_alternative, length) comprising for each batch, continuation:
            input_ids[batch, alternative, :] = [start_token] + story[:cap_length] + [delimiter_token] + cont1[:cap_length] + [clf_token]
        """
        tensor_datasets = []
        for dataset in encoded_datasets:
            n_batch = len(dataset)
            input_ids = np.zeros((n_batch, num_choices, max_seq_length), dtype=np.int64)
            mc_token_ids = np.zeros((n_batch, num_choices), dtype=np.int64)
            lm_labels = np.full((n_batch, num_choices, max_seq_length), fill_value=-1, dtype=np.int64)
            mc_labels = np.zeros((n_batch,), dtype=np.int64)
            for i, data, in enumerate(dataset):
                q, mc_label = data[0], data[-1]
                choices = data[1:-1]
                for j in range(len(choices)):
                    _truncate_seq_pair(q, choices[j], max_seq_length - 3)
                    qa = [start_token] + q + [delimiter_token] + choices[j] + [clf_token]
                    input_ids[i, j, :len(qa)] = qa
                    mc_token_ids[i, j] = len(qa) - 1
                    lm_labels[i, j, :len(qa) - 1] = qa[1:]
                mc_labels[i] = mc_label
            all_inputs = (input_ids, mc_token_ids, lm_labels, mc_labels)
            tensor_datasets.append(tuple(torch.tensor(t) for t in all_inputs))
        return tensor_datasets

    def tokenize_and_encode(tokenizer, obj):
        """ Tokenize and encode a nested object """
        if isinstance(obj, str):
            return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(obj))
        elif isinstance(obj, int):
            return obj
        else:
            return list(tokenize_and_encode(tokenizer, o) for o in obj)

    tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
    tokenizer.add_tokens(GPT_SPECIAL_TOKENS)
    special_tokens_ids = tokenizer.convert_tokens_to_ids(GPT_SPECIAL_TOKENS)

    dataset = load_qa_dataset(statement_jsonl_path)
    examples_ids = [data[0] for data in dataset]
    dataset = [data[1:] for data in dataset]  # discard example ids
    num_choices = len(dataset[0]) - 2

    encoded_dataset = tokenize_and_encode(tokenizer, dataset)

    (input_ids, mc_token_ids, lm_labels, mc_labels), = pre_process_datasets([encoded_dataset], num_choices, max_seq_length, *special_tokens_ids)
    return examples_ids, mc_labels, input_ids, mc_token_ids, lm_labels


def get_gpt_token_num():
    tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
    tokenizer.add_tokens(GPT_SPECIAL_TOKENS)
    return len(tokenizer)



def load_bert_xlnet_roberta_input_tensors(statement_jsonl_path, model_type, model_name, max_seq_length):
    class InputExample(object):

        def __init__(self, example_id, question, contexts, endings, label=None):
            self.example_id = example_id
            self.question = question
            self.contexts = contexts
            self.endings = endings
            self.label = label

    class InputFeatures(object):

        def __init__(self, example_id, choices_features, label):
            self.example_id = example_id
            self.choices_features = [
                {
                    'input_ids': input_ids,
                    'input_mask': input_mask,
                    'segment_ids': segment_ids,
                    'output_mask': output_mask,
                }
                for _, input_ids, input_mask, segment_ids, output_mask in choices_features
            ]
            self.label = label

    def read_examples(input_file):
        with open(input_file, "r", encoding="utf-8") as f:
            examples = []
            for line in f.readlines():
                json_dic = json.loads(line)
                label = ord(json_dic["answerKey"]) - ord("A") if 'answerKey' in json_dic else 0
                contexts = json_dic["question"]["stem"]
                if "para" in json_dic:
                    contexts = json_dic["para"] + " " + contexts
                if "fact1" in json_dic:
                    contexts = json_dic["fact1"] + " " + contexts
                examples.append(
                    InputExample(
                        example_id=json_dic["id"],
                        contexts=[contexts] * len(json_dic["question"]["choices"]),
                        question="",
                        endings=[ending["text"] for ending in json_dic["question"]["choices"]],
                        label=label
                    ))
        return examples

    def convert_examples_to_features(examples, label_list, max_seq_length,
                                     tokenizer,
                                     cls_token_at_end=False,
                                     cls_token='[CLS]',
                                     cls_token_segment_id=1,
                                     sep_token='[SEP]',
                                     sequence_a_segment_id=0,
                                     sequence_b_segment_id=1,
                                     sep_token_extra=False,
                                     pad_token_segment_id=0,
                                     pad_on_left=False,
                                     pad_token=0,
                                     mask_padding_with_zero=True):
        """ Loads a data file into a list of `InputBatch`s
            `cls_token_at_end` define the location of the CLS token:
                - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
                - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
            `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
        """
        label_map = {label: i for i, label in enumerate(label_list)}

        features = []
        for ex_index, example in enumerate(tqdm(examples)):
            choices_features = []
            for ending_idx, (context, ending) in enumerate(zip(example.contexts, example.endings)):
                tokens_a = tokenizer.tokenize(context)
                tokens_b = tokenizer.tokenize(example.question + " " + ending)

                special_tokens_count = 4 if sep_token_extra else 3
                _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - special_tokens_count)

                # The convention in BERT is:
                # (a) For sequence pairs:
                #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
                #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
                # (b) For single sequences:
                #  tokens:   [CLS] the dog is hairy . [SEP]
                #  type_ids:   0   0   0   0  0     0   0
                #
                # Where "type_ids" are used to indicate whether this is the first
                # sequence or the second sequence. The embedding vectors for `type=0` and
                # `type=1` were learned during pre-training and are added to the wordpiece
                # embedding vector (and position vector). This is not *strictly* necessary
                # since the [SEP] token unambiguously separates the sequences, but it makes
                # it easier for the model to learn the concept of sequences.
                #
                # For classification tasks, the first vector (corresponding to [CLS]) is
                # used as as the "sentence vector". Note that this only makes sense because
                # the entire model is fine-tuned.
                tokens = tokens_a + [sep_token]
                if sep_token_extra:
                    # roberta uses an extra separator b/w pairs of sentences
                    tokens += [sep_token]

                segment_ids = [sequence_a_segment_id] * len(tokens)

                if tokens_b:
                    tokens += tokens_b + [sep_token]
                    segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

                if cls_token_at_end:
                    tokens = tokens + [cls_token]
                    segment_ids = segment_ids + [cls_token_segment_id]
                else:
                    tokens = [cls_token] + tokens
                    segment_ids = [cls_token_segment_id] + segment_ids

                input_ids = tokenizer.convert_tokens_to_ids(tokens)

                # The mask has 1 for real tokens and 0 for padding tokens. Only real
                # tokens are attended to.

                input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)
                special_token_id = tokenizer.convert_tokens_to_ids([cls_token, sep_token])
                output_mask = [1 if id in special_token_id else 0 for id in input_ids]  # 1 for mask

                # Zero-pad up to the sequence length.
                padding_length = max_seq_length - len(input_ids)
                if pad_on_left:
                    input_ids = ([pad_token] * padding_length) + input_ids
                    input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                    output_mask = ([1] * padding_length) + output_mask

                    segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
                else:
                    input_ids = input_ids + ([pad_token] * padding_length)
                    input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                    output_mask = output_mask + ([1] * padding_length)
                    segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

                assert len(input_ids) == max_seq_length
                assert len(output_mask) == max_seq_length
                assert len(input_mask) == max_seq_length
                assert len(segment_ids) == max_seq_length
                choices_features.append((tokens, input_ids, input_mask, segment_ids, output_mask))
            label = label_map[example.label]
            features.append(InputFeatures(example_id=example.example_id, choices_features=choices_features, label=label))

        return features

    def _truncate_seq_pair(tokens_a, tokens_b, max_length):
        """Truncates a sequence pair in place to the maximum length."""

        # This is a simple heuristic which will always truncate the longer sequence
        # one token at a time. This makes more sense than truncating an equal percent
        # of tokens from each, since if one sequence is very short then each token
        # that's truncated likely contains more information than a longer sequence.
        while True:
            total_length = len(tokens_a) + len(tokens_b)
            if total_length <= max_length:
                break
            if len(tokens_a) > len(tokens_b):
                tokens_a.pop()
            else:
                tokens_b.pop()

    def select_field(features, field):
        return [[choice[field] for choice in feature.choices_features] for feature in features]

    def convert_features_to_tensors(features):
        all_input_ids = torch.tensor(select_field(features, 'input_ids'), dtype=torch.long)
        all_input_mask = torch.tensor(select_field(features, 'input_mask'), dtype=torch.long)
        all_segment_ids = torch.tensor(select_field(features, 'segment_ids'), dtype=torch.long)
        all_output_mask = torch.tensor(select_field(features, 'output_mask'), dtype=torch.bool)
        all_label = torch.tensor([f.label for f in features], dtype=torch.long)
        return all_input_ids, all_input_mask, all_segment_ids, all_output_mask, all_label

    # try:
    #     tokenizer_class = {'bert': BertTokenizer, 'xlnet': XLNetTokenizer, 'roberta': RobertaTokenizer, 'albert': AlbertTokenizer}.get(model_type)
    # except:
    #     tokenizer_class = {'bert': BertTokenizer, 'xlnet': XLNetTokenizer, 'roberta': RobertaTokenizer}.get(model_type)
    tokenizer_class = AutoTokenizer
    tokenizer = tokenizer_class.from_pretrained(model_name)
    examples = read_examples(statement_jsonl_path)
    features = convert_examples_to_features(examples, list(range(len(examples[0].endings))), max_seq_length, tokenizer,
                                            cls_token_at_end=bool(model_type in ['xlnet']),  # xlnet has a cls token at the end
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            sep_token_extra=bool(model_type in ['roberta', 'albert']),
                                            cls_token_segment_id=2 if model_type in ['xlnet'] else 0,
                                            pad_on_left=bool(model_type in ['xlnet']),  # pad on the left for xlnet
                                            pad_token_segment_id=4 if model_type in ['xlnet'] else 0,
                                            sequence_b_segment_id=0 if model_type in ['roberta', 'albert'] else 1)
    example_ids = [f.example_id for f in features]
    *data_tensors, all_label = convert_features_to_tensors(features)
    return (example_ids, all_label, *data_tensors)



def load_input_tensors(input_jsonl_path, model_type, model_name, max_seq_length):
    if model_type in ('lstm',):
        raise NotImplementedError
    elif model_type in ('gpt',):
        return load_gpt_input_tensors(input_jsonl_path, max_seq_length)
    elif model_type in ('bert', 'xlnet', 'roberta', 'albert'):
        return load_bert_xlnet_roberta_input_tensors(input_jsonl_path, model_type, model_name, max_seq_length)


def load_info(statement_path: str):
    n = sum(1 for _ in open(statement_path, "r"))
    num_choice = None
    with open(statement_path, "r", encoding="utf-8") as fin:
        ids = []
        labels = []
        for line in fin:
            input_json = json.loads(line)
            labels.append(ord(input_json.get("answerKey", "A")) - ord("A"))
            ids.append(input_json['id'])
            if num_choice is None:
                num_choice = len(input_json["question"]["choices"])
        labels = torch.tensor(labels, dtype=torch.long)

    return ids, labels, num_choice


def load_statement_dict(statement_path):
    all_dict = {}
    with open(statement_path, 'r', encoding='utf-8') as fin:
        for line in fin:
            instance_dict = json.loads(line)
            qid = instance_dict['id']
            all_dict[qid] = {
                'question': instance_dict['question']['stem'],
                'answers': [dic['text'] for dic in instance_dict['question']['choices']]
            }
    return all_dict


####from modeling.modeling_encoder import TextEncoder, MODEL_NAME_TO_CLASS

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import (OPENAI_GPT_PRETRAINED_CONFIG_ARCHIVE_MAP, BERT_PRETRAINED_CONFIG_ARCHIVE_MAP,
                          XLNET_PRETRAINED_CONFIG_ARCHIVE_MAP, ROBERTA_PRETRAINED_CONFIG_ARCHIVE_MAP)
try:
    from transformers import ALBERT_PRETRAINED_CONFIG_ARCHIVE_MAP
except:
    pass
from transformers import AutoModel, BertModel, BertConfig
# from transformers.modeling_bert import BERT_PRETRAINED_MODEL_ARCHIVE_MAP
# from utils.layers import *
# from utils.data_utils import get_gpt_token_num

MODEL_CLASS_TO_NAME = {
    'gpt': list(OPENAI_GPT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()),
    'bert': list(BERT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()),
    'xlnet': list(XLNET_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()),
    'roberta': list(ROBERTA_PRETRAINED_CONFIG_ARCHIVE_MAP.keys()),
    'lstm': ['lstm'],
}
try:
    MODEL_CLASS_TO_NAME['albert'] =  list(ALBERT_PRETRAINED_CONFIG_ARCHIVE_MAP.keys())
except:
    pass

MODEL_NAME_TO_CLASS = {model_name: model_class for model_class, model_name_list in MODEL_CLASS_TO_NAME.items() for model_name in model_name_list}

# #Add SapBERT configuration
# model_name = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'
# MODEL_NAME_TO_CLASS[model_name] = 'bert'


# class LSTMTextEncoder(nn.Module):
#     pool_layer_classes = {'mean': MeanPoolLayer, 'max': MaxPoolLayer}

#     def __init__(self, vocab_size=1, emb_size=300, hidden_size=300, output_size=300, num_layers=2, bidirectional=True,
#                  emb_p=0.0, input_p=0.0, hidden_p=0.0, pretrained_emb_or_path=None, freeze_emb=True,
#                  pool_function='max', output_hidden_states=False):
#         super().__init__()
#         self.output_size = output_size
#         self.num_layers = num_layers
#         self.output_hidden_states = output_hidden_states
#         assert not bidirectional or hidden_size % 2 == 0

#         if pretrained_emb_or_path is not None:
#             if isinstance(pretrained_emb_or_path, str):  # load pretrained embedding from a .npy file
#                 pretrained_emb_or_path = torch.tensor(np.load(pretrained_emb_or_path), dtype=torch.float)
#             emb = nn.Embedding.from_pretrained(pretrained_emb_or_path, freeze=freeze_emb)
#             emb_size = emb.weight.size(1)
#         else:
#             emb = nn.Embedding(vocab_size, emb_size)
#         self.emb = EmbeddingDropout(emb, emb_p)
#         self.rnns = nn.ModuleList([nn.LSTM(emb_size if l == 0 else hidden_size,
#                                            (hidden_size if l != num_layers else output_size) // (2 if bidirectional else 1),
#                                            1, bidirectional=bidirectional, batch_first=True) for l in range(num_layers)])
#         self.pooler = self.pool_layer_classes[pool_function]()

#         self.input_dropout = nn.Dropout(input_p)
#         self.hidden_dropout = nn.ModuleList([RNNDropout(hidden_p) for _ in range(num_layers)])

#     def forward(self, inputs, lengths):
#         """
#         inputs: tensor of shape (batch_size, seq_len)
#         lengths: tensor of shape (batch_size)

#         returns: tensor of shape (batch_size, hidden_size)
#         """
#         assert (lengths > 0).all()
#         batch_size, seq_len = inputs.size()
#         hidden_states = self.input_dropout(self.emb(inputs))
#         all_hidden_states = [hidden_states]
#         for l, (rnn, hid_dp) in enumerate(zip(self.rnns, self.hidden_dropout)):
#             hidden_states = pack_padded_sequence(hidden_states, lengths, batch_first=True, enforce_sorted=False)
#             hidden_states, _ = rnn(hidden_states)
#             hidden_states, _ = pad_packed_sequence(hidden_states, batch_first=True, total_length=seq_len)
#             all_hidden_states.append(hidden_states)
#             if l != self.num_layers - 1:
#                 hidden_states = hid_dp(hidden_states)
#         pooled = self.pooler(all_hidden_states[-1], lengths)
#         assert len(all_hidden_states) == self.num_layers + 1
#         outputs = (all_hidden_states[-1], pooled)
#         if self.output_hidden_states:
#             outputs = outputs + (all_hidden_states,)
#         return outputs


class TextEncoder(nn.Module):
    valid_model_types = set(MODEL_CLASS_TO_NAME.keys())

    def __init__(self, model_name, output_token_states=False, from_checkpoint=None, **kwargs):
        super().__init__()
        self.model_type = MODEL_NAME_TO_CLASS[model_name]
        self.output_token_states = output_token_states
        assert not self.output_token_states or self.model_type in ('bert', 'roberta', 'albert')

        if self.model_type in ('lstm',):
            self.module = LSTMTextEncoder(**kwargs, output_hidden_states=True)
            self.sent_dim = self.module.output_size
        else:
            model_class = AutoModel
            self.module = model_class.from_pretrained(model_name, output_hidden_states=True)
            if from_checkpoint is not None:
                self.module = self.module.from_pretrained(from_checkpoint, output_hidden_states=True)
            if self.model_type in ('gpt',):
                self.module.resize_token_embeddings(get_gpt_token_num())
            self.sent_dim = self.module.config.n_embd if self.model_type in ('gpt',) else self.module.config.hidden_size

    def forward(self, *inputs, layer_id=-1):
        '''
        layer_id: only works for non-LSTM encoders
        output_token_states: if True, return hidden states of specific layer and attention masks
        '''

        if self.model_type in ('lstm',):  # lstm
            input_ids, lengths = inputs
            outputs = self.module(input_ids, lengths)
        elif self.model_type in ('gpt',):  # gpt
            input_ids, cls_token_ids, lm_labels = inputs  # lm_labels is not used
            outputs = self.module(input_ids)
        else:  # bert / xlnet / roberta
            input_ids, attention_mask, token_type_ids, output_mask = inputs
            outputs = self.module(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        all_hidden_states = outputs[-1]
        hidden_states = all_hidden_states[layer_id]

        if self.model_type in ('lstm',):
            sent_vecs = outputs[1]
        elif self.model_type in ('gpt',):
            cls_token_ids = cls_token_ids.view(-1).unsqueeze(-1).unsqueeze(-1).expand(-1, 1, hidden_states.size(-1))
            sent_vecs = hidden_states.gather(1, cls_token_ids).squeeze(1)
        elif self.model_type in ('xlnet',):
            sent_vecs = hidden_states[:, -1]
        elif self.model_type in ('albert',):
            if self.output_token_states:
                return hidden_states, output_mask
            sent_vecs = hidden_states[:, 0]
        else:  # bert / roberta
            if self.output_token_states:
                return hidden_states, output_mask
            sent_vecs = self.module.pooler(hidden_states)
        return sent_vecs, all_hidden_states


# def run_test():
#     encoder = TextEncoder('lstm', vocab_size=100, emb_size=100, hidden_size=200, num_layers=4)
#     input_ids = torch.randint(0, 100, (30, 70))
#     lenghts = torch.randint(1, 70, (30,))
#     outputs = encoder(input_ids, lenghts)
#     assert outputs[0].size() == (30, 200)
#     assert len(outputs[1]) == 4 + 1
#     assert all([x.size() == (30, 70, 100 if l == 0 else 200) for l, x in enumerate(outputs[1])])
#     print('all tests are passed')


####Main model classes

In [8]:
# from modeling.modeling_encoder import TextEncoder, MODEL_NAME_TO_CLASS
# from utils.data_utils import *
# from utils.layers import *
import torch.nn.functional as F


class QAGNN_Message_Passing(nn.Module):
    def __init__(self, args, k, n_ntype, n_etype, input_size, hidden_size, output_size, dropout=0.1):
        super().__init__()
        assert input_size == output_size
        self.args = args
        self.n_ntype = n_ntype
        self.n_etype = n_etype

        assert input_size == hidden_size
        self.hidden_size = hidden_size

        self.emb_node_type = nn.Linear(self.n_ntype, hidden_size//2)

        self.basis_f = 'sin' #['id', 'linact', 'sin', 'none']
        if self.basis_f in ['id']:
            self.emb_score = nn.Linear(1, hidden_size//2)
        elif self.basis_f in ['linact']:
            self.B_lin = nn.Linear(1, hidden_size//2)
            self.emb_score = nn.Linear(hidden_size//2, hidden_size//2)
        elif self.basis_f in ['sin']:
            self.emb_score = nn.Linear(hidden_size//2, hidden_size//2)

        self.edge_encoder = torch.nn.Sequential(torch.nn.Linear(n_etype +1 + n_ntype *2, hidden_size), torch.nn.BatchNorm1d(hidden_size),
                                                torch.nn.ReLU(), torch.nn.Linear(hidden_size, hidden_size))


        self.k = k
        self.gnn_layers = nn.ModuleList([GATConvE(args, hidden_size, n_ntype, n_etype, self.edge_encoder) for _ in range(k)])


        self.Vh = nn.Linear(input_size, output_size)
        self.Vx = nn.Linear(hidden_size, output_size)

        self.activation = GELU()
        self.dropout = nn.Dropout(dropout)
        self.dropout_rate = dropout


    def mp_helper(self, _X, edge_index, edge_type, _node_type, _node_feature_extra):
        for _ in range(self.k):
            _X = self.gnn_layers[_](_X, edge_index, edge_type, _node_type, _node_feature_extra)
            _X = self.activation(_X)
            _X = F.dropout(_X, self.dropout_rate, training = self.training)
        return _X


    def forward(self, H, A, node_type, node_score, cache_output=False):
        """
        H: tensor of shape (batch_size, n_node, d_node)
            node features from the previous layer
        A: (edge_index, edge_type)
        node_type: long tensor of shape (batch_size, n_node)
            0 == question entity; 1 == answer choice entity; 2 == other node; 3 == context node
        node_score: tensor of shape (batch_size, n_node, 1)
        """
        _batch_size, _n_nodes = node_type.size()

        #Embed type
        T = make_one_hot(node_type.view(-1).contiguous(), self.n_ntype).view(_batch_size, _n_nodes, self.n_ntype)
        node_type_emb = self.activation(self.emb_node_type(T)) #[batch_size, n_node, dim/2]

        #Embed score
        if self.basis_f == 'sin':
            js = torch.arange(self.hidden_size//2).unsqueeze(0).unsqueeze(0).float().to(node_type.device) #[1,1,dim/2]
            js = torch.pow(1.1, js) #[1,1,dim/2]
            B = torch.sin(js * node_score) #[batch_size, n_node, dim/2]
            node_score_emb = self.activation(self.emb_score(B)) #[batch_size, n_node, dim/2]
        elif self.basis_f == 'id':
            B = node_score
            node_score_emb = self.activation(self.emb_score(B)) #[batch_size, n_node, dim/2]
        elif self.basis_f == 'linact':
            B = self.activation(self.B_lin(node_score)) #[batch_size, n_node, dim/2]
            node_score_emb = self.activation(self.emb_score(B)) #[batch_size, n_node, dim/2]


        X = H
        edge_index, edge_type = A #edge_index: [2, total_E]   edge_type: [total_E, ]  where total_E is for the batched graph
        _X = X.view(-1, X.size(2)).contiguous() #[`total_n_nodes`, d_node] where `total_n_nodes` = b_size * n_node
        _node_type = node_type.view(-1).contiguous() #[`total_n_nodes`, ]
        _node_feature_extra = torch.cat([node_type_emb, node_score_emb], dim=2).view(_node_type.size(0), -1).contiguous() #[`total_n_nodes`, dim]

        _X = self.mp_helper(_X, edge_index, edge_type, _node_type, _node_feature_extra)

        X = _X.view(node_type.size(0), node_type.size(1), -1) #[batch_size, n_node, dim]

        output = self.activation(self.Vh(H) + self.Vx(X))
        output = self.dropout(output)

        return output



class QAGNN(nn.Module):
    def __init__(self, args, k, n_ntype, n_etype, sent_dim,
                 n_concept, concept_dim, concept_in_dim, n_attention_head,
                 fc_dim, n_fc_layer, p_emb, p_gnn, p_fc,
                 pretrained_concept_emb=None, freeze_ent_emb=True,
                 init_range=0.02):
        super().__init__()
        self.init_range = init_range

        self.concept_emb = CustomizedEmbedding(concept_num=n_concept, concept_out_dim=concept_dim,
                                               use_contextualized=False, concept_in_dim=concept_in_dim,
                                               pretrained_concept_emb=pretrained_concept_emb, freeze_ent_emb=freeze_ent_emb)
        self.svec2nvec = nn.Linear(sent_dim, concept_dim)

        self.concept_dim = concept_dim

        self.activation = GELU()

        self.gnn = QAGNN_Message_Passing(args, k=k, n_ntype=n_ntype, n_etype=n_etype,
                                        input_size=concept_dim, hidden_size=concept_dim, output_size=concept_dim, dropout=p_gnn)

        self.pooler = MultiheadAttPoolLayer(n_attention_head, sent_dim, concept_dim)

        self.fc = MLP(concept_dim + sent_dim + concept_dim, fc_dim, 1, n_fc_layer, p_fc, layer_norm=True)

        self.dropout_e = nn.Dropout(p_emb)
        self.dropout_fc = nn.Dropout(p_fc)

        if init_range > 0:
            self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=self.init_range)
            if hasattr(module, 'bias') and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)


    def forward(self, sent_vecs, concept_ids, node_type_ids, node_scores, adj_lengths, adj, emb_data=None, cache_output=False):
        """
        sent_vecs: (batch_size, dim_sent)
        concept_ids: (batch_size, n_node)
        adj: edge_index, edge_type
        adj_lengths: (batch_size,)
        node_type_ids: (batch_size, n_node)
            0 == question entity; 1 == answer choice entity; 2 == other node; 3 == context node
        node_scores: (batch_size, n_node, 1)

        returns: (batch_size, 1)
        """
        gnn_input0 = self.activation(self.svec2nvec(sent_vecs)).unsqueeze(1) #(batch_size, 1, dim_node)
        gnn_input1 = self.concept_emb(concept_ids[:, 1:]-1, emb_data) #(batch_size, n_node-1, dim_node)
        gnn_input1 = gnn_input1.to(node_type_ids.device)
        gnn_input = self.dropout_e(torch.cat([gnn_input0, gnn_input1], dim=1)) #(batch_size, n_node, dim_node)


        #Normalize node score (use norm from Z)
        _mask = (torch.arange(node_scores.size(1), device=node_scores.device) < adj_lengths.unsqueeze(1)).float() #0 means masked out #[batch_size, n_node]
        node_scores = -node_scores
        node_scores = node_scores - node_scores[:, 0:1, :] #[batch_size, n_node, 1]
        node_scores = node_scores.squeeze(2) #[batch_size, n_node]
        node_scores = node_scores * _mask
        mean_norm  = (torch.abs(node_scores)).sum(dim=1) / adj_lengths  #[batch_size, ]
        node_scores = node_scores / (mean_norm.unsqueeze(1) + 1e-05) #[batch_size, n_node]
        node_scores = node_scores.unsqueeze(2) #[batch_size, n_node, 1]


        gnn_output = self.gnn(gnn_input, adj, node_type_ids, node_scores)

        Z_vecs = gnn_output[:,0]   #(batch_size, dim_node)

        mask = torch.arange(node_type_ids.size(1), device=node_type_ids.device) >= adj_lengths.unsqueeze(1) #1 means masked out

        mask = mask | (node_type_ids == 3) #pool over all KG nodes
        mask[mask.all(1), 0] = 0  # a temporary solution to avoid zero node

        sent_vecs_for_pooler = sent_vecs
        graph_vecs, pool_attn = self.pooler(sent_vecs_for_pooler, gnn_output, mask)

        if cache_output:
            self.concept_ids = concept_ids
            self.adj = adj
            self.pool_attn = pool_attn

        concat = self.dropout_fc(torch.cat((graph_vecs, sent_vecs, Z_vecs), 1))
        logits = self.fc(concat)
        return logits, pool_attn


class LM_QAGNN(nn.Module):
    def __init__(self, args, model_name, k, n_ntype, n_etype,
                 n_concept, concept_dim, concept_in_dim, n_attention_head,
                 fc_dim, n_fc_layer, p_emb, p_gnn, p_fc,
                 pretrained_concept_emb=None, freeze_ent_emb=True,
                 init_range=0.0, encoder_config={}):
        super().__init__()
        self.encoder = TextEncoder(model_name, **encoder_config)
        self.decoder = QAGNN(args, k, n_ntype, n_etype, self.encoder.sent_dim,
                                        n_concept, concept_dim, concept_in_dim, n_attention_head,
                                        fc_dim, n_fc_layer, p_emb, p_gnn, p_fc,
                                        pretrained_concept_emb=pretrained_concept_emb, freeze_ent_emb=freeze_ent_emb,
                                        init_range=init_range)


    def forward(self, *inputs, layer_id=-1, cache_output=False, detail=False):
        """
        sent_vecs: (batch_size, num_choice, d_sent)    -> (batch_size * num_choice, d_sent)
        concept_ids: (batch_size, num_choice, n_node)  -> (batch_size * num_choice, n_node)
        node_type_ids: (batch_size, num_choice, n_node) -> (batch_size * num_choice, n_node)
        adj_lengths: (batch_size, num_choice)          -> (batch_size * num_choice, )
        adj -> edge_index, edge_type
            edge_index: list of (batch_size, num_choice) -> list of (batch_size * num_choice, ); each entry is torch.tensor(2, E(variable))
                                                         -> (2, total E)
            edge_type:  list of (batch_size, num_choice) -> list of (batch_size * num_choice, ); each entry is torch.tensor(E(variable), )
                                                         -> (total E, )
        returns: (batch_size, 1)
        """
        bs, nc = inputs[0].size(0), inputs[0].size(1)

        #Here, merge the batch dimension and the num_choice dimension
        edge_index_orig, edge_type_orig = inputs[-2:]
        _inputs = [x.view(x.size(0) * x.size(1), *x.size()[2:]) for x in inputs[:-6]] + [x.view(x.size(0) * x.size(1), *x.size()[2:]) for x in inputs[-6:-2]] + [sum(x,[]) for x in inputs[-2:]]

        *lm_inputs, concept_ids, node_type_ids, node_scores, adj_lengths, edge_index, edge_type = _inputs
        edge_index, edge_type = self.batch_graph(edge_index, edge_type, concept_ids.size(1))
        adj = (edge_index.to(node_type_ids.device), edge_type.to(node_type_ids.device)) #edge_index: [2, total_E]   edge_type: [total_E, ]

        sent_vecs, all_hidden_states = self.encoder(*lm_inputs, layer_id=layer_id)
        logits, attn = self.decoder(sent_vecs.to(node_type_ids.device),
                                    concept_ids,
                                    node_type_ids, node_scores, adj_lengths, adj,
                                    emb_data=None, cache_output=cache_output)
        logits = logits.view(bs, nc)
        if not detail:
            return logits, attn
        else:
            return logits, attn, concept_ids.view(bs, nc, -1), node_type_ids.view(bs, nc, -1), edge_index_orig, edge_type_orig
            #edge_index_orig: list of (batch_size, num_choice). each entry is torch.tensor(2, E)
            #edge_type_orig: list of (batch_size, num_choice). each entry is torch.tensor(E, )


    def batch_graph(self, edge_index_init, edge_type_init, n_nodes):
        #edge_index_init: list of (n_examples, ). each entry is torch.tensor(2, E)
        #edge_type_init:  list of (n_examples, ). each entry is torch.tensor(E, )
        n_examples = len(edge_index_init)
        edge_index = [edge_index_init[_i_] + _i_ * n_nodes for _i_ in range(n_examples)]
        edge_index = torch.cat(edge_index, dim=1) #[2, total_E]
        edge_type = torch.cat(edge_type_init, dim=0) #[total_E, ]
        return edge_index, edge_type



class LM_QAGNN_DataLoader(object):
    def __init__(self, args, train_statement_path, train_adj_path,
                 dev_statement_path, dev_adj_path,
                 test_statement_path, test_adj_path,
                 batch_size, eval_batch_size, device, model_name, max_node_num=200, max_seq_length=128,
                 is_inhouse=False, inhouse_train_qids_path=None,
                 subsample=1.0, use_cache=True):
        super().__init__()
        self.args = args
        self.batch_size = batch_size
        self.eval_batch_size = eval_batch_size
        self.device0, self.device1 = device
        self.is_inhouse = is_inhouse

        model_type = MODEL_NAME_TO_CLASS[model_name]
        print('train_statement_path', train_statement_path)

        self.train_qids, self.train_labels, *self.train_encoder_data = load_input_tensors(train_statement_path, model_type, model_name, max_seq_length)
        self.dev_qids, self.dev_labels, *self.dev_encoder_data = load_input_tensors(dev_statement_path, model_type, model_name, max_seq_length)

        num_choice = self.train_encoder_data[0].size(1)
        self.num_choice = num_choice
        print ('num_choice', num_choice)

        *self.train_decoder_data, self.train_adj_data = load_sparse_adj_data_with_contextnode(train_adj_path, max_node_num, num_choice, args)
        *self.dev_decoder_data, self.dev_adj_data = load_sparse_adj_data_with_contextnode(dev_adj_path, max_node_num, num_choice, args)

        assert all(len(self.train_qids) == len(self.train_adj_data[0]) == x.size(0) for x in [self.train_labels] + self.train_encoder_data + self.train_decoder_data)
        assert all(len(self.dev_qids) == len(self.dev_adj_data[0]) == x.size(0) for x in [self.dev_labels] + self.dev_encoder_data + self.dev_decoder_data)

        if test_statement_path is not None:
            self.test_qids, self.test_labels, *self.test_encoder_data = load_input_tensors(test_statement_path, model_type, model_name, max_seq_length)
            *self.test_decoder_data, self.test_adj_data = load_sparse_adj_data_with_contextnode(test_adj_path, max_node_num, num_choice, args)
            assert all(len(self.test_qids) == len(self.test_adj_data[0]) == x.size(0) for x in [self.test_labels] + self.test_encoder_data + self.test_decoder_data)


        if self.is_inhouse:
            with open(inhouse_train_qids_path, 'r') as fin:
                inhouse_qids = set(line.strip() for line in fin)
            self.inhouse_train_indexes = torch.tensor([i for i, qid in enumerate(self.train_qids) if qid in inhouse_qids])
            self.inhouse_test_indexes = torch.tensor([i for i, qid in enumerate(self.train_qids) if qid not in inhouse_qids])

        assert 0. < subsample <= 1.
        if subsample < 1.:
            n_train = int(self.train_size() * subsample)
            assert n_train > 0
            if self.is_inhouse:
                self.inhouse_train_indexes = self.inhouse_train_indexes[:n_train]
            else:
                self.train_qids = self.train_qids[:n_train]
                self.train_labels = self.train_labels[:n_train]
                self.train_encoder_data = [x[:n_train] for x in self.train_encoder_data]
                self.train_decoder_data = [x[:n_train] for x in self.train_decoder_data]
                self.train_adj_data = self.train_adj_data[:n_train]
                assert all(len(self.train_qids) == len(self.train_adj_data[0]) == x.size(0) for x in [self.train_labels] + self.train_encoder_data + self.train_decoder_data)
            assert self.train_size() == n_train

    def train_size(self):
        return self.inhouse_train_indexes.size(0) if self.is_inhouse else len(self.train_qids)

    def dev_size(self):
        return len(self.dev_qids)

    def test_size(self):
        if self.is_inhouse:
            return self.inhouse_test_indexes.size(0)
        else:
            return len(self.test_qids) if hasattr(self, 'test_qids') else 0

    def train(self):
        if self.is_inhouse:
            n_train = self.inhouse_train_indexes.size(0)
            train_indexes = self.inhouse_train_indexes[torch.randperm(n_train)]
        else:
            train_indexes = torch.randperm(len(self.train_qids))
        return MultiGPUSparseAdjDataBatchGenerator(self.args, 'train', self.device0, self.device1, self.batch_size, train_indexes, self.train_qids, self.train_labels, tensors0=self.train_encoder_data, tensors1=self.train_decoder_data, adj_data=self.train_adj_data)

    def train_eval(self):
        return MultiGPUSparseAdjDataBatchGenerator(self.args, 'eval', self.device0, self.device1, self.eval_batch_size, torch.arange(len(self.train_qids)), self.train_qids, self.train_labels, tensors0=self.train_encoder_data, tensors1=self.train_decoder_data, adj_data=self.train_adj_data)

    def dev(self):
        return MultiGPUSparseAdjDataBatchGenerator(self.args, 'eval', self.device0, self.device1, self.eval_batch_size, torch.arange(len(self.dev_qids)), self.dev_qids, self.dev_labels, tensors0=self.dev_encoder_data, tensors1=self.dev_decoder_data, adj_data=self.dev_adj_data)

    def test(self):
        if self.is_inhouse:
            return MultiGPUSparseAdjDataBatchGenerator(self.args, 'eval', self.device0, self.device1, self.eval_batch_size, self.inhouse_test_indexes, self.train_qids, self.train_labels, tensors0=self.train_encoder_data, tensors1=self.train_decoder_data, adj_data=self.train_adj_data)
        else:
            return MultiGPUSparseAdjDataBatchGenerator(self.args, 'eval', self.device0, self.device1, self.eval_batch_size, torch.arange(len(self.test_qids)), self.test_qids, self.test_labels, tensors0=self.test_encoder_data, tensors1=self.test_decoder_data, adj_data=self.test_adj_data)





###############################################################################
############################### GNN architecture ##############################
###############################################################################

from torch.autograd import Variable
def make_one_hot(labels, C):
    '''
    Converts an integer label torch.autograd.Variable to a one-hot Variable.
    labels : torch.autograd.Variable of torch.cuda.LongTensor
        (N, ), where N is batch size.
        Each value is an integer representing correct classification.
    C : integer.
        number of classes in labels.
    Returns : torch.autograd.Variable of torch.cuda.FloatTensor
        N x C, where C is class number. One-hot encoded.
    '''
    labels = labels.unsqueeze(1)
    one_hot = torch.FloatTensor(labels.size(0), C).zero_().to(labels.device)
    target = one_hot.scatter_(1, labels.data, 1)
    target = Variable(target)
    return target



from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree, softmax
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
import torch.nn.functional as F
from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import glorot, zeros



class GATConvE(MessagePassing):
    """
    Args:
        emb_dim (int): dimensionality of GNN hidden states
        n_ntype (int): number of node types (e.g. 4)
        n_etype (int): number of edge relation types (e.g. 38)
    """
    def __init__(self, args, emb_dim, n_ntype, n_etype, edge_encoder, head_count=4, aggr="add"):
        super(GATConvE, self).__init__(aggr=aggr)
        self.args = args

        assert emb_dim % 2 == 0
        self.emb_dim = emb_dim

        self.n_ntype = n_ntype; self.n_etype = n_etype
        self.edge_encoder = edge_encoder

        #For attention
        self.head_count = head_count
        assert emb_dim % head_count == 0
        self.dim_per_head = emb_dim // head_count
        self.linear_key = nn.Linear(3*emb_dim, head_count * self.dim_per_head)
        self.linear_msg = nn.Linear(3*emb_dim, head_count * self.dim_per_head)
        self.linear_query = nn.Linear(2*emb_dim, head_count * self.dim_per_head)

        self._alpha = None

        #For final MLP
        self.mlp = torch.nn.Sequential(torch.nn.Linear(emb_dim, emb_dim), torch.nn.BatchNorm1d(emb_dim), torch.nn.ReLU(), torch.nn.Linear(emb_dim, emb_dim))


    def forward(self, x, edge_index, edge_type, node_type, node_feature_extra, return_attention_weights=False):
        # x: [N, emb_dim]
        # edge_index: [2, E]
        # edge_type [E,] -> edge_attr: [E, 39] / self_edge_attr: [N, 39]
        # node_type [N,] -> headtail_attr [E, 8(=4+4)] / self_headtail_attr: [N, 8]
        # node_feature_extra [N, dim]

        #Prepare edge feature
        edge_vec = make_one_hot(edge_type, self.n_etype +1) #[E, 39]
        self_edge_vec = torch.zeros(x.size(0), self.n_etype +1).to(edge_vec.device)
        self_edge_vec[:,self.n_etype] = 1

        head_type = node_type[edge_index[0]] #[E,] #head=src
        tail_type = node_type[edge_index[1]] #[E,] #tail=tgt
        head_vec = make_one_hot(head_type, self.n_ntype) #[E,4]
        tail_vec = make_one_hot(tail_type, self.n_ntype) #[E,4]
        headtail_vec = torch.cat([head_vec, tail_vec], dim=1) #[E,8]
        self_head_vec = make_one_hot(node_type, self.n_ntype) #[N,4]
        self_headtail_vec = torch.cat([self_head_vec, self_head_vec], dim=1) #[N,8]

        edge_vec = torch.cat([edge_vec, self_edge_vec], dim=0) #[E+N, ?]
        headtail_vec = torch.cat([headtail_vec, self_headtail_vec], dim=0) #[E+N, ?]
        edge_embeddings = self.edge_encoder(torch.cat([edge_vec, headtail_vec], dim=1)) #[E+N, emb_dim]

        #Add self loops to edge_index
        loop_index = torch.arange(0, x.size(0), dtype=torch.long, device=edge_index.device)
        loop_index = loop_index.unsqueeze(0).repeat(2, 1)
        edge_index = torch.cat([edge_index, loop_index], dim=1)  #[2, E+N]

        x = torch.cat([x, node_feature_extra], dim=1)
        x = (x, x)
        aggr_out = self.propagate(edge_index, x=x, edge_attr=edge_embeddings) #[N, emb_dim]
        out = self.mlp(aggr_out)

        alpha = self._alpha
        self._alpha = None

        if return_attention_weights:
            assert alpha is not None
            return out, (edge_index, alpha)
        else:
            return out


    def message(self, edge_index, x_i, x_j, edge_attr): #i: tgt, j:src
        # print ("edge_attr.size()", edge_attr.size()) #[E, emb_dim]
        # print ("x_j.size()", x_j.size()) #[E, emb_dim]
        # print ("x_i.size()", x_i.size()) #[E, emb_dim]
        assert len(edge_attr.size()) == 2
        assert edge_attr.size(1) == self.emb_dim
        assert x_i.size(1) == x_j.size(1) == 2*self.emb_dim
        assert x_i.size(0) == x_j.size(0) == edge_attr.size(0) == edge_index.size(1)

        key   = self.linear_key(torch.cat([x_i, edge_attr], dim=1)).view(-1, self.head_count, self.dim_per_head) #[E, heads, _dim]
        msg = self.linear_msg(torch.cat([x_j, edge_attr], dim=1)).view(-1, self.head_count, self.dim_per_head) #[E, heads, _dim]
        query = self.linear_query(x_j).view(-1, self.head_count, self.dim_per_head) #[E, heads, _dim]


        query = query / math.sqrt(self.dim_per_head)
        scores = (query * key).sum(dim=2) #[E, heads]
        src_node_index = edge_index[0] #[E,]
        alpha = softmax(scores, src_node_index) #[E, heads] #group by src side node
        self._alpha = alpha

        #adjust by outgoing degree of src
        E = edge_index.size(1)            #n_edges
        N = int(src_node_index.max()) + 1 #n_nodes
        ones = torch.full((E,), 1.0, dtype=torch.float).to(edge_index.device)
        src_node_edge_count = scatter(ones, src_node_index, dim=0, dim_size=N, reduce='sum')[src_node_index] #[E,]
        assert len(src_node_edge_count.size()) == 1 and len(src_node_edge_count) == E
        alpha = alpha * src_node_edge_count.unsqueeze(1) #[E, heads]

        out = msg * alpha.view(-1, self.head_count, 1) #[E, heads, _dim]
        return out.view(-1, self.head_count * self.dim_per_head)  #[E, emb_dim]


####from utils.optimization_utils import OPTIMIZER_CLASSES


In [9]:
import math
import torch
from transformers import AdamW
from torch.optim import SGD, Adam
from torch.optim.optimizer import Optimizer


class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))

        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss


OPTIMIZER_CLASSES = {
    'sgd': SGD,
    'adam': Adam,
    'adamw': AdamW,
    'radam': RAdam,
}


# def run_test():
#     import torch.nn as nn
#     model = nn.Sequential(*[nn.Linear(100, 10), nn.ReLU(), nn.Linear(10, 2)])
#     x = torch.randn(10, 100).repeat(100, 1)
#     y = torch.randint(0, 2, (10,)).repeat(100)
#     crit = nn.CrossEntropyLoss()
#     optim = RAdam(model.parameters(), lr=1e-2, weight_decay=0.01)
#     model.train()
#     for a in range(0, 1000, 10):
#         b = a + 10
#         loss = crit(model(x[a:b]), y[a:b])
#         loss.backward()
#         optim.step()
#         print('| loss: {:.4f} |'.format(loss.item()))


# if __name__ == '__main__':
#     run_test()


####from utils.parser_utils import *

In [10]:
import argparse
# from utils.utils import *
# from modeling.modeling_encoder import MODEL_NAME_TO_CLASS

ENCODER_DEFAULT_LR = {
    'default': 1e-3,
    'csqa': {
        'lstm': 3e-4,
        'openai-gpt': 1e-4,
        'bert-base-uncased': 3e-5,
        'bert-large-uncased': 2e-5,
        'roberta-large': 1e-5,
    },
    'obqa': {
        'lstm': 3e-4,
        'openai-gpt': 3e-5,
        'bert-base-cased': 1e-4,
        'bert-large-cased': 1e-4,
        'roberta-large': 1e-5,
    },
    'medqa_usmle': {
        'cambridgeltl/SapBERT-from-PubMedBERT-fulltext': 5e-5,
    },
}

DATASET_LIST = ['csqa', 'obqa', 'socialiqa', 'medqa_usmle']

DATASET_SETTING = {
    'csqa': 'inhouse',
    'obqa': 'official',
    'socialiqa': 'official',
    'medqa_usmle': 'official',
}

DATASET_NO_TEST = ['socialiqa']

EMB_PATHS = {
    'transe': 'data/transe/glove.transe.sgd.ent.npy',
    'lm': 'data/transe/glove.transe.sgd.ent.npy',
    'numberbatch': 'data/transe/concept.nb.npy',
    'tzw': 'data/cpnet/tzw.ent.npy',
    'ddb': 'data/ddb/ent_emb.npy',
}


# def add_data_arguments(parser):
#     # arguments that all datasets share
#     parser.add_argument('--ent_emb', default=['tzw'], nargs='+', help='sources for entity embeddings')
#     # dataset specific
#     parser.add_argument('-ds', '--dataset', default='csqa', choices=DATASET_LIST, help='dataset name')
#     parser.add_argument('-ih', '--inhouse', type=bool_flag, nargs='?', const=True, help='run in-house setting')
#     parser.add_argument('--inhouse_train_qids', default='data/{dataset}/inhouse_split_qids.txt', help='qids of the in-house training set')
#     # statements
#     parser.add_argument('--train_statements', default='data/{dataset}/statement/train.statement.jsonl')
#     parser.add_argument('--dev_statements', default='data/{dataset}/statement/dev.statement.jsonl')
#     parser.add_argument('--test_statements', default='data/{dataset}/statement/test.statement.jsonl')
#     # preprocessing options
#     parser.add_argument('-sl', '--max_seq_len', default=100, type=int)
#     # set dataset defaults
#     args, _ = parser.parse_known_args()
#     parser.set_defaults(ent_emb_paths=[EMB_PATHS.get(s) for s in args["ent_emb"]],
#                         inhouse=(DATASET_SETTING[args["dataset"]] == 'inhouse'),
#                         inhouse_train_qids=args["inhouse_train_qids"].format(dataset=args["dataset"]))
#     data_splits = ('train', 'dev') if args["dataset"] in DATASET_NO_TEST else ('train', 'dev', 'test')
#     for split in data_splits:
#         for attribute in ('statements',):
#             attr_name = f'{split}_{attribute}'
#             parser.set_defaults(**{attr_name: getattr(args, attr_name).format(dataset=args["dataset"])})
#     if 'test' not in data_splits:
#         parser.set_defaults(test_statements=None)


# def add_encoder_arguments(parser):
#     parser.add_argument('-enc', '--encoder', default='bert-large-uncased', help='encoder type')
#     parser.add_argument('--encoder_layer', default=-1, type=int, help='encoder layer ID to use as features (used only by non-LSTM encoders)')
#     parser.add_argument('-elr', '--encoder_lr', default=2e-5, type=float, help='learning rate')
#     args, _ = parser.parse_known_args()
#     parser.set_defaults(encoder_lr=ENCODER_DEFAULT_LR[args["dataset"]].get(args["encoder"], ENCODER_DEFAULT_LR['default']))


# def add_optimization_arguments(parser):
#     parser.add_argument('--loss', default='cross_entropy', choices=['margin_rank', 'cross_entropy'], help='model type')
#     parser.add_argument('--optim', default='radam', choices=['sgd', 'adam', 'adamw', 'radam'], help='learning rate scheduler')
#     parser.add_argument('--lr_schedule', default='fixed', choices=['fixed', 'warmup_linear', 'warmup_constant'], help='learning rate scheduler')
#     parser.add_argument('-bs', '--batch_size', default=32, type=int)
#     parser.add_argument('--warmup_steps', type=float, default=150)
#     parser.add_argument('--max_grad_norm', default=1.0, type=float, help='max grad norm (0 to disable)')
#     parser.add_argument('--weight_decay', default=1e-2, type=float, help='l2 weight decay strength')
#     parser.add_argument('--n_epochs', default=100, type=int, help='total number of training epochs to perform.')
#     parser.add_argument('-me', '--max_epochs_before_stop', default=10, type=int, help='stop training if dev does not increase for N epochs')


# def add_additional_arguments(parser):
#     parser.add_argument('--log_interval', default=10, type=int)
#     parser.add_argument('--cuda', default=True, type=bool_flag, nargs='?', const=True, help='use GPU')
#     parser.add_argument('--seed', default=0, type=int, help='random seed')
#     parser.add_argument('--debug', default=False, type=bool_flag, nargs='?', const=True, help='run in debug mode')
#     args, _ = parser.parse_known_args()
#     if args["debug"]:
#         parser.set_defaults(batch_size=1, log_interval=1, eval_interval=5)


# def get_parser():
#     """A helper function that handles the arguments that all models share"""
#     parser = argparse.ArgumentParser(add_help=False)
#     add_data_arguments(parser)
#     add_encoder_arguments(parser)
#     add_optimization_arguments(parser)
#     add_additional_arguments(parser)
#     return parser


def get_lstm_config_from_args(args):
    lstm_config = {
        'hidden_size': args["encoder_dim"],
        'output_size': args["encoder_dim"],
        'num_layers': args["encoder_layer_num"],
        'bidirectional': args["encoder_bidir"],
        'emb_p': args["encoder_dropoute"],
        'input_p': args["encoder_dropouti"],
        'hidden_p': args["encoder_dropouth"],
        'pretrained_emb_or_path': args["encoder_pretrained_emb"],
        'freeze_emb': args["encoder_freeze_emb"],
        'pool_function': args["encoder_pooler"],
    }
    return lstm_config


###main func.

In [11]:
import random

try:
    from transformers import (ConstantLRSchedule, WarmupLinearSchedule, WarmupConstantSchedule)
except:
    from transformers import get_constant_schedule, get_constant_schedule_with_warmup,  get_linear_schedule_with_warmup

# from modeling.modeling_qagnn import *
# from utils.optimization_utils import OPTIMIZER_CLASSES
# from utils.parser_utils import *


DECODER_DEFAULT_LR = {
    'csqa': 1e-3,
    'brain_teaser_ds': 1e-3,
    'obqa': 3e-4,
    'medqa_usmle': 1e-3,
}

from collections import defaultdict, OrderedDict
import numpy as np
import socket, os, subprocess, datetime

print(socket.gethostname())
print ("pid:", os.getpid())
# print ("conda env:", os.environ['CONDA_DEFAULT_ENV'])
# print ("screen: %s" % subprocess.check_output('echo $STY', shell=True).decode('utf'))
# print ("gpu: %s" % subprocess.check_output('echo $CUDA_VISIBLE_DEVICES', shell=True).decode('utf'))


def evaluate_accuracy(eval_set, model):
    n_samples, n_correct = 0, 0
    model.eval()
    with torch.no_grad():
        for qids, labels, *input_data in tqdm(eval_set):
            logits, _ = model(*input_data)
            n_correct += (logits.argmax(1) == labels).sum().item()
            n_samples += labels.size(0)
    return n_correct / n_samples



def train(args, global_step, best_dev_epoch, is_finetuning_on_new_ds):
    print(args)

    best_dev_acc=None
    lower_epoch=None

    random.seed(args["seed"])
    np.random.seed(args["seed"])
    torch.manual_seed(args["seed"])
    if torch.cuda.is_available() and args["cuda"]:
        torch.cuda.manual_seed(args["seed"])

    config_path = os.path.join(args["log_save_dir"], 'config.json')
    log_path = os.path.join(args["log_save_dir"], 'log.csv')
    best_model_path = os.path.join(args["best_model_save_dir"], 'model.pt')
    checkpoint_path = os.path.join(args["checkpoint_save_dir"], 'model.pt')
    export_config(args, config_path)
    check_path(best_model_path)
    check_path(checkpoint_path)
    check_path(log_path)
    with open(log_path, 'w') as fout:
        fout.write('step,dev_acc,test_acc\n')

    ###################################################################################################
    #   Load data                                                                                     #
    ###################################################################################################
    cp_emb = [np.load(path) for path in args["ent_emb_paths"]]
    cp_emb = torch.tensor(np.concatenate(cp_emb, 1), dtype=torch.float)

    concept_num, concept_dim = cp_emb.size(0), cp_emb.size(1)
    print('| num_concepts: {} |'.format(concept_num))
    print('| concept_dim: {} |'.format(concept_dim))

    # try:
    if True:
        if torch.cuda.device_count() >= 2 and args["cuda"]:
            device0 = torch.device("cuda:0")
            device1 = torch.device("cuda:1")
        elif torch.cuda.device_count() == 1 and args["cuda"]:
            device0 = torch.device("cuda:0")
            device1 = torch.device("cuda:0")
        else:
            device0 = torch.device("cpu")
            device1 = torch.device("cpu")
        dataset = LM_QAGNN_DataLoader(args, args["train_statements"], args["train_adj"],
                                               args["dev_statements"], args["dev_adj"],
                                               args["test_statements"], args["test_adj"],
                                               batch_size=args["batch_size"], eval_batch_size=args["eval_batch_size"],
                                               device=(device0, device1),
                                               model_name=args["encoder"],
                                               max_node_num=args["max_node_num"], max_seq_length=args["max_seq_len"],
                                               is_inhouse=args["inhouse"], inhouse_train_qids_path=args["inhouse_train_qids"],
                                               subsample=args["subsample"], use_cache=args["use_cache"])

        #!
        # return dataset

        ###################################################################################################
        #   Build model                                                                                   #
        ###################################################################################################
        print('args.num_relation', args["num_relation"])
        model = LM_QAGNN(args, args["encoder"], k=args["k"], n_ntype=4, n_etype=args["num_relation"], n_concept=concept_num,
                                   concept_dim=args["gnn_dim"],
                                   concept_in_dim=concept_dim,
                                   n_attention_head=args["att_head_num"], fc_dim=args["fc_dim"], n_fc_layer=args["fc_layer_num"],
                                   p_emb=args["dropouti"], p_gnn=args["dropoutg"], p_fc=args["dropoutf"],
                                   pretrained_concept_emb=cp_emb, freeze_ent_emb=args["freeze_ent_emb"],
                                   init_range=args["init_range"],
                                   encoder_config={})

        if args["load_model_path"]:
            print(f'loading and initializing model from {args["load_model_path"]}')
            # model_state_dict, old_args = torch.load(args["load_model_path"], map_location=torch.device('cpu'))
            # model.load_state_dict(model_state_dict)
            checkpoint = torch.load(args["load_model_path"], map_location=torch.device('cpu'))
            model.load_state_dict(checkpoint['model_state_dict'])
            best_dev_acc=checkpoint['best_dev_acc']
            lower_epoch = checkpoint['epoch']+1

            if is_finetuning_on_new_ds:
              best_dev_acc=None
              lower_epoch=None
            print(f"previous best_dev_acc: {best_dev_acc}")
            print(f"current training epoch start from: {lower_epoch}")

        model.encoder.to(device0)
        model.decoder.to(device1)


    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

    grouped_parameters = [
        {'params': [p for n, p in model.encoder.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args["weight_decay"], 'lr': args["encoder_lr"]},
        {'params': [p for n, p in model.encoder.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr': args["encoder_lr"]},
        {'params': [p for n, p in model.decoder.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args["weight_decay"], 'lr': args["decoder_lr"]},
        {'params': [p for n, p in model.decoder.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0, 'lr': args["decoder_lr"]},
    ]

    optimizer = OPTIMIZER_CLASSES[args["optim"]](grouped_parameters)

    if args["lr_schedule"] == 'fixed':
        try:
            scheduler = ConstantLRSchedule(optimizer)
        except:
            scheduler = get_constant_schedule(optimizer)
    elif args["lr_schedule"] == 'warmup_constant':
        try:
            scheduler = WarmupConstantSchedule(optimizer, warmup_steps=args["warmup_steps"])
        except:
            scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=args["warmup_steps"])
    elif args["lr_schedule"] == 'warmup_linear':
        max_steps = int(args["n_epochs"] * (dataset.train_size() / args["batch_size"]))
        try:
            scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args["warmup_steps"], t_total=max_steps)
        except:
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args["warmup_steps"], num_training_steps=max_steps)

    print('parameters:')
    for name, param in model.decoder.named_parameters():
        if param.requires_grad:
            print('\t{:45}\ttrainable\t{}\tdevice:{}'.format(name, param.size(), param.device))
        else:
            print('\t{:45}\tfixed\t{}\tdevice:{}'.format(name, param.size(), param.device))
    num_params = sum(p.numel() for p in model.decoder.parameters() if p.requires_grad)
    print('\ttotal:', num_params)

    if args["loss"] == 'margin_rank':
        loss_func = nn.MarginRankingLoss(margin=0.1, reduction='mean')
    elif args["loss"] == 'cross_entropy':
        loss_func = nn.CrossEntropyLoss(reduction='mean')

    def compute_loss(logits, labels):
        if args["loss"] == 'margin_rank':
            num_choice = logits.size(1)
            flat_logits = logits.view(-1)
            correct_mask = F.one_hot(labels, num_classes=num_choice).view(-1)  # of length batch_size*num_choice
            correct_logits = flat_logits[correct_mask == 1].contiguous().view(-1, 1).expand(-1, num_choice - 1).contiguous().view(-1)  # of length batch_size*(num_choice-1)
            wrong_logits = flat_logits[correct_mask == 0]
            y = wrong_logits.new_ones((wrong_logits.size(0),))
            loss = loss_func(correct_logits, wrong_logits, y)  # margin ranking loss
        elif args["loss"] == 'cross_entropy':
            loss = loss_func(logits, labels)
        return loss

    ###################################################################################################
    #   Training                                                                                      #
    ###################################################################################################

    print()
    print('-' * 71)
    if args["fp16"]:
        print ('Using fp16 training')
        scaler = torch.cuda.amp.GradScaler()

    # global_step, best_dev_epoch = 0, 0

    # best_dev_acc, final_test_acc, total_loss = 0.0, 0.0, 0.0

    final_test_acc, total_loss = 0.0, 0.0

    if best_dev_acc is None:
      best_dev_acc=0.0

    if lower_epoch==None:
      lower_epoch=0
    # if args['best_dev_acc']!=None:
    #   best_dev_acc=args['best_dev_acc']
    # else:
    #   best_dev_acc=0.0

    start_time = time.time()
    model.train()
    freeze_net(model.encoder)
    if True:
    # try:
        for epoch_id in range(lower_epoch, args["n_epochs"]):
            print(f"epoch: {epoch_id+1}/{args['n_epochs']}")
            if epoch_id == args["unfreeze_epoch"]:
                unfreeze_net(model.encoder)
            if epoch_id == args["refreeze_epoch"]:
                freeze_net(model.encoder)
            model.train()
            for qids, labels, *input_data in dataset.train():
                optimizer.zero_grad()
                bs = labels.size(0)
                for a in range(0, bs, args["mini_batch_size"]):
                    b = min(a + args["mini_batch_size"], bs)
                    if args["fp16"]:
                        with torch.cuda.amp.autocast():
                            logits, _ = model(*[x[a:b] for x in input_data], layer_id=args["encoder_layer"])
                            loss = compute_loss(logits, labels[a:b])
                    else:
                        logits, _ = model(*[x[a:b] for x in input_data], layer_id=args["encoder_layer"])
                        loss = compute_loss(logits, labels[a:b])
                    loss = loss * (b - a) / bs
                    if args["fp16"]:
                        scaler.scale(loss).backward()
                    else:
                        loss.backward()
                    total_loss += loss.item()
                if args["max_grad_norm"] > 0:
                    if args["fp16"]:
                        scaler.unscale_(optimizer)
                        nn.utils.clip_grad_norm_(model.parameters(), args["max_grad_norm"])
                    else:
                        nn.utils.clip_grad_norm_(model.parameters(), args["max_grad_norm"])
                scheduler.step()
                if args["fp16"]:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()

                if (global_step + 1) % args["log_interval"] == 0:
                    total_loss /= args["log_interval"]
                    ms_per_batch = 1000 * (time.time() - start_time) / args["log_interval"]
                    print('| step {:5} |  lr: {:9.7f} | loss {:7.4f} | ms/batch {:7.2f} |'.format(global_step, scheduler.get_lr()[0], total_loss, ms_per_batch))
                    total_loss = 0
                    start_time = time.time()
                global_step += 1


            """ Evaluation """
            model.eval()
            dev_acc = evaluate_accuracy(dataset.dev(), model)
            # save_test_preds = args["save_model"]
            save_test_preds = args["save_test_preds"]
            if not save_test_preds:
                test_acc = evaluate_accuracy(dataset.test(), model) if args["test_statements"] else 0.0
            else:
                eval_set = dataset.test()
                total_acc = []
                count = 0
                preds_path = os.path.join(args["log_save_dir"], 'test_e{}_preds.csv'.format(epoch_id))
                with open(preds_path, 'w') as f_preds:
                    with torch.no_grad():
                        for qids, labels, *input_data in tqdm(eval_set):
                            count += 1
                            logits, _, concept_ids, node_type_ids, edge_index, edge_type = model(*input_data, detail=True)
                            predictions = logits.argmax(1) #[bsize, ]
                            preds_ranked = (-logits).argsort(1) #[bsize, n_choices]
                            for i, (qid, label, pred, _preds_ranked, cids, ntype, edges, etype) in enumerate(zip(qids, labels, predictions, preds_ranked, concept_ids, node_type_ids, edge_index, edge_type)):
                                acc = int(pred.item()==label.item())
                                print ('{},{}'.format(qid, chr(ord('A') + pred.item())), file=f_preds)
                                f_preds.flush()
                                total_acc.append(acc)
                test_acc = float(sum(total_acc))/len(total_acc)

            print('-' * 71)
            print('| epoch {:3} | step {:5} | dev_acc {:7.4f} | test_acc {:7.4f} |'.format(epoch_id, global_step, dev_acc, test_acc))
            print('-' * 71)
            with open(log_path, 'a') as fout:
                fout.write('{},{},{}\n'.format(global_step, dev_acc, test_acc))
            if dev_acc >= best_dev_acc:
                best_dev_acc = dev_acc
                final_test_acc = test_acc
                best_dev_epoch = epoch_id
                if args["save_best_model"]:
                    # torch.save([model.state_dict(), args], f"{model_path}.{epoch_id}.best_model")
                    torch.save({
                          'epoch': epoch_id,
                          'model_state_dict': model.state_dict(),
                          'args':args,
                          'best_dev_acc':best_dev_acc,
                          'total_loss': total_loss,
                          'global_step':global_step,
                          }, f"{best_model_path}.{epoch_id}.best_model")
                    # with open(model_path +".{}.log.txt".format(epoch_id), 'w') as f:
                    #     for p in model.named_parameters():
                    #         print (p, file=f)
                    print(f'model saved to {best_model_path}.{epoch_id}.best_model')
            else:
                if args["save_checkpoint"]:
                    # torch.save([model.state_dict(), args], f"{model_path}.{epoch_id}.temp_checkpoint")
                    torch.save({
                          'epoch': epoch_id,
                          'model_state_dict': model.state_dict(),
                          'args':args,
                          'best_dev_acc':best_dev_acc,
                          'total_loss': total_loss,
                          'global_step':global_step,
                          }, f"{checkpoint_path}.{epoch_id}.temp_checkpoint")

                    # with open(model_path +".{}.log.txt".format(epoch_id), 'w') as f:
                    #     for p in model.named_parameters():
                    #         print (p, file=f)


                    print(f'model saved to {checkpoint_path}.{epoch_id}.temp_checkpoint')
            model.train()
            start_time = time.time()
            if epoch_id > args["unfreeze_epoch"] and epoch_id - best_dev_epoch >= args["max_epochs_before_stop"]:
                break
    # except (KeyboardInterrupt, RuntimeError) as e:
    #     print(e)



def eval_detail(args):
    assert args["load_model_path"] is not None
    model_path = args["load_model_path"]

    cp_emb = [np.load(path) for path in args["ent_emb_paths"]]
    cp_emb = torch.tensor(np.concatenate(cp_emb, 1), dtype=torch.float)
    concept_num, concept_dim = cp_emb.size(0), cp_emb.size(1)
    print('| num_concepts: {} |'.format(concept_num))

    model_state_dict, old_args = torch.load(model_path, map_location=torch.device('cpu'))
    model = LM_QAGNN(old_args, old_args["encoder"], k=old_args["k"], n_ntype=4, n_etype=old_args["num_relation"], n_concept=concept_num,
                               concept_dim=old_args["gnn_dim"],
                               concept_in_dim=concept_dim,
                               n_attention_head=old_args["att_head_num"], fc_dim=old_args["fc_dim"], n_fc_layer=old_args["fc_layer_num"],
                               p_emb=old_args["dropouti"], p_gnn=old_args["dropoutg"], p_fc=old_args["dropoutf"],
                               pretrained_concept_emb=cp_emb, freeze_ent_emb=old_args["freeze_ent_emb"],
                               init_range=old_args["init_range"],
                               encoder_config={})
    # model.load_state_dict(model_state_dict)
    checkpoint = torch.load(args["load_model_path"], map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'])

    if torch.cuda.device_count() >= 2 and args["cuda"]:
        device0 = torch.device("cuda:0")
        device1 = torch.device("cuda:1")
    elif torch.cuda.device_count() == 1 and args["cuda"]:
        device0 = torch.device("cuda:0")
        device1 = torch.device("cuda:0")
    else:
        device0 = torch.device("cpu")
        device1 = torch.device("cpu")
    model.encoder.to(device0)
    model.decoder.to(device1)
    model.eval()

    statement_dic = {}
    for statement_path in (args["train_statements"], args["dev_statements"], args["test_statements"]):
        statement_dic.update(load_statement_dict(statement_path))

    # use_contextualized = 'lm' in old_args["ent_emb"]
    use_contextualized = 'lm' in args["ent_emb"]

    print ('inhouse?', args["inhouse"])

    print ('args.train_statements', args["train_statements"])
    print ('args.dev_statements', args["dev_statements"])
    print ('args.test_statements', args["test_statements"])
    print ('args.train_adj', args["train_adj"])
    print ('args.dev_adj', args["dev_adj"])
    print ('args.test_adj', args["test_adj"])

    dataset = LM_QAGNN_DataLoader(args, args["train_statements"], args["train_adj"],
                                           args["dev_statements"], args["dev_adj"],
                                           args["test_statements"], args["test_adj"],
                                           batch_size=args["batch_size"], eval_batch_size=args["eval_batch_size"],
                                           device=(device0, device1),
                                           model_name=old_args["encoder"],
                                           max_node_num=old_args["max_node_num"], max_seq_length=old_args["max_seq_len"],
                                           is_inhouse=args["inhouse"], inhouse_train_qids_path=args["inhouse_train_qids"],
                                           subsample=args["subsample"], use_cache=args["use_cache"])

    save_test_preds = args["save_test_preds"]
    dev_acc = evaluate_accuracy(dataset.dev(), model)
    print('dev_acc {:7.4f}'.format(dev_acc))
    if not save_test_preds:
        test_acc = evaluate_accuracy(dataset.test(), model) if args["test_statements"] else 0.0
    else:
        eval_set = dataset.test()
        total_acc = []
        count = 0
        dt = datetime.datetime.today().strftime('%Y%m%d%H%M%S')
        preds_path = os.path.join(args["log_save_dir"], 'test_preds_{}.csv'.format(dt))
        with open(preds_path, 'w') as f_preds:
            with torch.no_grad():
                for qids, labels, *input_data in tqdm(eval_set):
                    count += 1
                    logits, _, concept_ids, node_type_ids, edge_index, edge_type = model(*input_data, detail=True)
                    predictions = logits.argmax(1) #[bsize, ]
                    preds_ranked = (-logits).argsort(1) #[bsize, n_choices]
                    for i, (qid, label, pred, _preds_ranked, cids, ntype, edges, etype) in enumerate(zip(qids, labels, predictions, preds_ranked, concept_ids, node_type_ids, edge_index, edge_type)):
                        acc = int(pred.item()==label.item())
                        print ('{},{}'.format(qid, chr(ord('A') + pred.item())), file=f_preds)
                        f_preds.flush()
                        total_acc.append(acc)
        test_acc = float(sum(total_acc))/len(total_acc)

        print('-' * 71)
        print('test_acc {:7.4f}'.format(test_acc))
        print('-' * 71)



def predict(args):
  assert args["load_model_path"] is not None
  model_path = args["load_model_path"]

  cp_emb = [np.load(path) for path in args["ent_emb_paths"]]
  cp_emb = torch.tensor(np.concatenate(cp_emb, 1), dtype=torch.float)
  concept_num, concept_dim = cp_emb.size(0), cp_emb.size(1)
  print('| num_concepts: {} |'.format(concept_num))

  args["use_cache"]=True        #use cached data to accelerate data loading

  # model architecture
  args["att_head_num"]=2    # type=int, help='number of attention heads'
  args["fc_dim"]=200    # type=int, help='number of FC hidden units'
  args["fc_layer_num"]=1   # help='number of FC layers'
  args["freeze_ent_emb"]=True     # help='freeze entity embedding layer'

  args["max_node_num"]=200
  args["simple"]=False    # type=bool_flag, nargs='?', const=True
  args["subsample"]=1.0
  args["init_range"]=0.02     #type=float, help='stddev when initializing with normal distribution'


  # regularization
  args["dropouti"]=0.2   # help='dropout for embedding layer'
  args["dropoutg"]=0.2    #help='dropout for GNN layers'
  args["dropoutf"]=0.2    #help='dropout for fully-connected layers'

  # optimization
  args["mbs"]=1
  args["ebs"]=2
  args["eval_batch_size"]=2
  args["unfreeze_epoch"]=4
  args["refreeze_epoch"]=10000
  args["drop_partial_batch"]=False
  args["fill_partial_batch"]=False

  model = LM_QAGNN(args, args["encoder"], k=args["k"], n_ntype=4, n_etype=args["num_relation"], n_concept=concept_num,
                                    concept_dim=args["gnn_dim"],
                                    concept_in_dim=concept_dim,
                                    n_attention_head=args["att_head_num"], fc_dim=args["fc_dim"], n_fc_layer=args["fc_layer_num"],
                                    p_emb=args["dropouti"], p_gnn=args["dropoutg"], p_fc=args["dropoutf"],
                                    pretrained_concept_emb=cp_emb, freeze_ent_emb=args["freeze_ent_emb"],
                                    init_range=args["init_range"],
                                    encoder_config={})
  checkpoint = torch.load(args["load_model_path"], map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['model_state_dict'])

  if torch.cuda.device_count() >= 2 and args["cuda"]:
      device0 = torch.device("cuda:0")
      device1 = torch.device("cuda:1")
  elif torch.cuda.device_count() == 1 and args["cuda"]:
      device0 = torch.device("cuda:0")
      device1 = torch.device("cuda:0")
  else:
      device0 = torch.device("cpu")
      device1 = torch.device("cpu")
  model.encoder.to(device0)
  model.decoder.to(device1)
  model.eval()

  print ('inhouse?', args["inhouse"])

  print ('args.train_statements', args["train_statements"])
  print ('args.dev_statements', args["dev_statements"])
  print ('args.test_statements', args["test_statements"])
  print ('args.train_adj', args["train_adj"])
  print ('args.dev_adj', args["dev_adj"])
  print ('args.test_adj', args["test_adj"])


  dataset = LM_QAGNN_DataLoader(args, args["train_statements"], args["train_adj"],
                                          args["dev_statements"], args["dev_adj"],
                                          args["test_statements"], args["test_adj"],
                                          batch_size=args["batch_size"], eval_batch_size=args["eval_batch_size"],
                                          device=(device0, device1),
                                          model_name=args["encoder"],
                                          max_node_num=args["max_node_num"], max_seq_length=args["max_seq_len"],
                                          is_inhouse=args["inhouse"], inhouse_train_qids_path=args["inhouse_train_qids"],
                                          subsample=args["subsample"], use_cache=args["use_cache"])

  save_test_preds = args["save_test_preds"]
  dev_acc = evaluate_accuracy(dataset.dev(), model)
  print('dev_acc {:7.4f}'.format(dev_acc))
  test_preds=[]
  if not save_test_preds:
      test_acc = evaluate_accuracy(dataset.test(), model) if args["test_statements"] else 0.0
  else:
      eval_set = dataset.test()
      count = 0
      dt = datetime.datetime.today().strftime('%Y%m%d%H%M%S')
      preds_path = os.path.join(args["log_save_dir"], 'test_preds_{}.csv'.format(dt))
      with open(preds_path, 'w') as f_preds:
          with torch.no_grad():
              for qids, labels, *input_data in tqdm(eval_set):
                  count += 1
                  logits, _, concept_ids, node_type_ids, edge_index, edge_type = model(*input_data, detail=True)
                  predictions = logits.argmax(1) #[bsize, ]
                  preds_ranked = (-logits).argsort(1) #[bsize, n_choices]
                  for i, (qid, label, pred, _preds_ranked, cids, ntype, edges, etype) in enumerate(zip(qids, labels, predictions, preds_ranked, concept_ids, node_type_ids, edge_index, edge_type)):
                      print ('{},{}'.format(qid, pred.item()), file=f_preds)
                      test_preds.append(pred.item())
                      f_preds.flush()

      print()
      print('-' * 71)
  return test_preds



def main_5(args, global_step, best_dev_epoch, is_finetuning_on_new_ds):

    # args["save_dir"]=f'/content/drive/Shareddrives/Gdrive/My_Thesis/QA_GNN/saved_models'   #model output directory
    # args["load_model_path"]='/content/drive/Shareddrives/Gdrive/My_Thesis/QA_GNN/saved_models/csqa/enc-roberta-large__k5__gnndim100__bs8__seed0/model.pt.2'

    args["use_cache"]=True        #use cached data to accelerate data loading

    # model architecture
    args["att_head_num"]=2    # type=int, help='number of attention heads'
    args["fc_dim"]=200    # type=int, help='number of FC hidden units'
    args["fc_layer_num"]=1   # help='number of FC layers'
    args["freeze_ent_emb"]=True     # help='freeze entity embedding layer'

    args["max_node_num"]=200
    args["simple"]=False    # type=bool_flag, nargs='?', const=True
    args["subsample"]=1.0
    args["init_range"]=0.02     #type=float, help='stddev when initializing with normal distribution'


    # regularization
    args["dropouti"]=0.2   # help='dropout for embedding layer'
    args["dropoutg"]=0.2    #help='dropout for GNN layers'
    args["dropoutf"]=0.2    #help='dropout for fully-connected layers'

    # optimization
    args["mbs"]=1
    args["ebs"]=2
    args["eval_batch_size"]=2
    args["unfreeze_epoch"]=4
    args["refreeze_epoch"]=10000
    args["drop_partial_batch"]=False
    args["fill_partial_batch"]=False


    if args["simple"]:
        args["k"]=1

    args["fp16"] = args["fp16"] and (torch.__version__ >= '1.6.0')

    if args["mode"] == 'train':
        train(args, global_step, best_dev_epoch, is_finetuning_on_new_ds)
    elif args["mode"] == 'eval_detail':
        # raise NotImplementedError
        eval_detail(args)
    else:
        raise ValueError('Invalid mode')



96c4552eff2c
pid: 2005


##<font color=greeen>Train brain_teaser</font>

In [12]:
dataset="brain_teaser_ds"
model='roberta-large'
# model='bert-large-uncased'

# elr=1e-5 #encoder learning rate
elr=2e-4 #encoder learning rate
dlr=1e-3  #decoder learing rate
batch_size=128 #128 default
mbs=2   #mini-batch-size
n_epochs=100   #def 100
num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges
seed=0

k=5 #num of gnn layers
gnndim=200    #dimension of the GNN layers def 200

!echo "***** hyperparameters *****"
!echo "dataset: $dataset"
!echo "enc_name: $model"
!echo "batch_size: $batch_size"
!echo "learning_rate: elr $elr dlr $dlr"
!echo "gnn: dim $gnndim layer $k"
!echo "Epochs: $n_epochs"
!echo "******************************"

main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"

if not os.path.exists(save_dir_pref):
  %mkdir -p $save_dir_pref
%mkdir -p logs

args=dict()

args['enc']=model
args['encoder']=model

# args["best_model_save_dir"]=f"{save_dir_pref}/new_SP_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
# # args["checkpoint_save_dir"]=f"{save_dir_pref}/new_SP_checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
# args["checkpoint_save_dir"]=f"/content/new_SP_checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
# args["log_save_dir"]=f"{save_dir_pref}/new_SP_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"


args["best_model_save_dir"]=f"{save_dir_pref}/new_WP_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
# args["checkpoint_save_dir"]=f"{save_dir_pref}/new_WP_checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["checkpoint_save_dir"]=f"/content/new_WP_checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/new_WP_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"


args["save_best_model"]=True
args['save_test_preds']=False
args["save_checkpoint"]=True

args['loss']='cross_entropy'
args['lr_schedule']='fixed'
args['bs']=batch_size
args['batch_size']=batch_size

args['warmup_steps']=0

args['max_grad_norm']=1.0
args['me']=10
args['log_interval']=5
args['debug']=False
args["optim"]="radam"

args["mode"]="train"    #choices=['train', 'eval_detail'], help='run training or evaluation
args["dataset"]=dataset
args["encoder"]=model
args['enc']=model
args["k"]=k   #perform k-layer message passing
args["gnn_dim"]=gnndim
args["encoder_lr"]=elr
args["decoder_lr"]=dlr
args["batch_size"]=batch_size
args["mini_batch_size"]=mbs
args["fp16"]=False   #use fp16 training. this requires torch>=1.6.0
args["seed"]=seed
args["num_relation"]=num_relation
args["n_epochs"]=n_epochs
args["max_epochs_before_stop"]=10
# args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/SP_train.graph.adj.pk"
# args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/SP_dev.graph.adj.pk"
args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/WP_train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/WP_dev.graph.adj.pk"
args["test_adj"]=None
# args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/SP_test.graph.adj.pk"

# args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/SP_train.statement.jsonl"
# args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/SP_dev.statement.jsonl"
args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/WP_train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/WP_dev.statement.jsonl"

args["test_statements"]=None
# args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/SP_test.statement.jsonl"


args["cuda"]=True
# args["ent_emb_paths"]=['data/cpnet/tzw.ent.npy']
args["ent_emb_paths"]=['/content/drive/MyDrive/brain_teaser/datasets/cpnet/tzw.ent.npy']

args["max_seq_len"]=128
args["inhouse"]=False
# args["inhouse_train_qids"]=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/inhouse_split_qids.txt"
args["inhouse_train_qids"]=f"/content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/WP_inhouse_split_qids.txt"

args["weight_decay"]=1e-2

args['encoder_layer']=-1
# args['elr']=2e-4
# args["encoder_lr"]=2e-4

# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.16.best_model'

args["load_model_path"]=None
# args["load_model_path"]="/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.26.best_model"
# args["load_model_path"]="/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.best_model"
# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-bert-large-uncased_k5__gnndim200__bs128__seed0/model.pt.7.best_model'
# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_checkpoint_brain_teaser_ds__enc-bert-large-uncased_k5__gnndim200__bs128__seed0/model.pt.8.temp_checkpoint'

***** hyperparameters *****
dataset: brain_teaser_ds
enc_name: roberta-large
batch_size: 128
learning_rate: elr 0.0002 dlr 0.001
gnn: dim 200 layer 5
Epochs: 100
******************************


###predict on test data

In [13]:
import numpy as np
import pandas as pd

####SP

In [14]:
args["save_test_preds"]=True

k=5 #num of gnn layers

task_type="SP"
# args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.best_model"
args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.33.best_model"


main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"
dataset="brain_teaser_ds"
model='roberta-large'

args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_dev.graph.adj.pk"
args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_test.graph.adj.pk"

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_dev.statement.jsonl"
args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_test.statement.jsonl"

args["best_model_save_dir"]=f"{save_dir_pref}/new_{task_type}_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/new_{task_type}_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"


In [15]:
args["best_model_save_dir"]

'/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_SP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0'

In [16]:
sp_test_pred=predict(args)

| num_concepts: 799273 |


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

| ori_adj_len: mu 369.61 sigma 278.58 | adj_len: 172.92 | prune_rate： 0.61 | qc_num: 17.12 | ac_num: 4.92 |


100%|██████████| 26/26 [00:06<00:00,  3.76it/s]


dev_acc  0.9020


100%|██████████| 60/60 [00:13<00:00,  4.53it/s]



-----------------------------------------------------------------------


In [17]:
len(sp_test_pred)

120

In [18]:
# SP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
SP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP_new_test.npy', allow_pickle=True,encoding='bytes')
print(len(SP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in SP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

SP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
SP_val_df.head(2)

120


,question,choice_list
0,"In a small village, two farmers are working in...","[The lazy farmer is his mother., The lazy farm..."
1,Romeo and Juliet are discovered dead on the be...,[They were sleeping and scared by the sound of...


In [19]:
ids=[]
questions=[]
choice_lists=[]
predicted_labels=[]

for i in tqdm(range(len(SP_val_df))):
  sample=SP_val_df.iloc[i]
  pred=sp_test_pred[i]
  ids.append(i)
  questions.append(sample['question'].strip())
  predicted_labels.append(pred)
  choice_lists.append(sample['choice_list'])

temp_dict={'id':ids,
'question':questions,
'choice_list':choice_lists,
'label':predicted_labels}

sp_new_df=pd.DataFrame(data=temp_dict)
sp_new_df

100%|██████████| 120/120 [00:00<00:00, 19801.58it/s]


,id,question,choice_list,label
0,0,"In a small village, two farmers are working in...","[The lazy farmer is his mother., The lazy farm...",0
1,1,Romeo and Juliet are discovered dead on the be...,[They were sleeping and scared by the sound of...,2
2,2,How many years in your life it happens that be...,"[In ech leap year., In the first year of gradu...",2
3,3,Who would serve as the team's captain if a cru...,"[The first officer., The captain., The second ...",1
4,4,"In one city, 5% of the population has an unlis...","[One hundred people., Ninty-five people., Five...",1
...,...,...,...,...
115,115,A professional fisherman caught 30 fish during...,"[Two., One., Three., None of above.]",3
116,116,Bob was working on a project when suddenly int...,"[He needed a membership to search in google., ...",2
117,117,"He has wed numerous women, but never himself. ...","[A teacher., A preacher., A laywer., None of a...",3
118,118,You walk into a room and see a bed and lie on ...,"[Thirty-six, as there are eighteen animals., S...",0


In [20]:
import datetime
dt = datetime.datetime.today().strftime('%Y_%m_%d')
dt

'2024_01_21'

In [21]:
sp_new_df.to_csv(f"/content/drive/MyDrive/brain_teaser/SP_qagnn_predictions_test_{dt}.csv", index=False)

In [22]:
res=[str(x) for x in sp_new_df['label'].values]
with open('answer_sen.txt', 'w') as f:
    f.write('\n'.join(res))

####WP

In [23]:
args["save_test_preds"]=True

k=5 #num of gnn layers
task_type="WP"
# args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.26.best_model"
args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.28.best_model"


main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"
dataset="brain_teaser_ds"
model='roberta-large'

args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_dev.graph.adj.pk"
args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_test.graph.adj.pk"

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_dev.statement.jsonl"
args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_test.statement.jsonl"

args["best_model_save_dir"]=f"{save_dir_pref}/new_{task_type}_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/new_{task_type}_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"

In [24]:
args["best_model_save_dir"]

'/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0'

In [25]:
wp_test_pred=predict(args)

| num_concepts: 799273 |


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


inhouse? False
args.train_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl
args.dev_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_dev.statement.jsonl
args.test_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_test.statement.jsonl
args.train_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_train.graph.adj.pk
args.dev_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_dev.graph.adj.pk
args.test_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_test.graph.adj.pk
train_statement_path /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl


100%|██████████| 40/40 [00:00<00:00, 1186.53it/s]


num_choice 4
| ori_adj_len: mu 96.10 sigma 96.83 | adj_len: 85.07 | prune_rate： 0.10 | qc_num: 6.03 | ac_num: 2.34 |
| ori_adj_len: mu 125.84 sigma 140.18 | adj_len: 94.26 | prune_rate： 0.20 | qc_num: 6.69 | ac_num: 2.72 |


...

| ori_adj_len: mu 83.97 sigma 74.87 | adj_len: 78.50 | prune_rate： 0.09 | qc_num: 5.00 | ac_num: 2.15 |


100%|██████████| 20/20 [00:04<00:00,  4.60it/s]


dev_acc  0.8500


100%|██████████| 48/48 [00:10<00:00,  4.69it/s]



-----------------------------------------------------------------------


In [26]:
len(wp_test_pred)

96

In [27]:
# WP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
WP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP_new_test.npy', allow_pickle=True,encoding='bytes')

print(len(WP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in WP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

WP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
WP_val_df.head(2)

96


,question,choice_list
0,What kind of stock doesn't have shares?,"[Small-cap stock., Livestock., Growth stock., ..."
1,What do you call a toothless bear?,"[A brown bear., A polar bear., A gummy bear., ..."


In [28]:
ids=[]
questions=[]
choice_lists=[]
predicted_labels=[]

for i in tqdm(range(len(WP_val_df))):
  sample=WP_val_df.iloc[i]
  pred=wp_test_pred[i]
  ids.append(i)
  questions.append(sample['question'].strip())
  predicted_labels.append(pred)
  choice_lists.append(sample['choice_list'])

temp_dict={'id':ids,
'question':questions,
'choice_list':choice_lists,
'label':predicted_labels}

WP_new_df=pd.DataFrame(data=temp_dict)
WP_new_df

100%|██████████| 96/96 [00:00<00:00, 20972.61it/s]


,id,question,choice_list,label
0,0,What kind of stock doesn't have shares?,"[Small-cap stock., Livestock., Growth stock., ...",1
1,1,What do you call a toothless bear?,"[A brown bear., A polar bear., A gummy bear., ...",2
2,2,Where will a computer technician keep all his ...,"[A mouse., A monitor., A keyboard., None of ab...",0
3,3,What do you call a bear with no teeth?,"[A brown bear., A gummy bear., A polar bear., ...",1
4,4,What was the result of the tiny lobster's math...,"[Very-bad., Sea-plus., Very-Good., None of abo...",1
...,...,...,...,...
91,91,What can you find at the end of a rainbow?,"[The letter R., The letter W., The letter I., ...",1
92,92,"Without using any R sounds, how do you say, ""P...","[Bought a dog., Purchased A Rotweiler., Uchase...",2
93,93,What kind of phone is most colorful?,"[A cellphone., A smartphone., Chromophone., No...",0
94,94,What's the coolest letter in the alphabet?,"[ 'Z', because it's the end of alphabet., 'A'...",2


In [29]:
dt = datetime.datetime.today().strftime('%Y_%m_%d')
dt

'2024_01_21'

In [30]:
WP_new_df.to_csv(f"/content/drive/MyDrive/brain_teaser/WP_qagnn_predictions_test_{dt}.csv", index=False)

In [31]:
res=[str(x) for x in WP_new_df['label'].values]
with open('answer_word.txt', 'w') as f:
    f.write('\n'.join(res))

###training log

WP

In [ ]:
main_5(args, global_step=0, best_dev_epoch=0, is_finetuning_on_new_ds=True)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/new_WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_WP_log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/My

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch  36 | step   111 | dev_acc  0.8000 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.36.temp_checkpoint
epoch: 38/100


100%|██████████| 20/20 [00:04<00:00,  4.17it/s]


-----------------------------------------------------------------------
| epoch  37 | step   114 | dev_acc  0.7250 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.37.temp_checkpoint
epoch: 39/100
| step   114 |  lr: 0.0002000 | loss  0.2458 | ms/batch 7551.11 |


100%|██████████| 20/20 [00:04<00:00,  4.29it/s]


-----------------------------------------------------------------------
| epoch  38 | step   117 | dev_acc  0.7750 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.38.temp_checkpoint


SP

In [ ]:
main_5(args, global_step=0, best_dev_epoch=0, is_finetuning_on_new_ds=True)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_SP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/new_SP_log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/My

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch  41 | step   168 | dev_acc  0.8235 | test_acc  0.0000 |
-----------------------------------------------------------------------
-----------------------------------------------------------------------
| epoch  41 | step   168 | dev_acc  0.8235 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.41.temp_checkpoint
epoch: 43/100
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.41.temp_checkpoint
epoch: 43/100
| step   169 |  lr: 0.0002000 | loss  0.0324 | ms/batch 16897.30 |
| step   169 |  lr: 0.0002000 | loss  0.0324 | ms/batch 16897.30 |


100%|██████████| 26/26 [00:06<00:00,  4.23it/s]



-----------------------------------------------------------------------
| epoch  42 | step   172 | dev_acc  0.8039 | test_acc  0.0000 |
-----------------------------------------------------------------------
-----------------------------------------------------------------------
| epoch  42 | step   172 | dev_acc  0.8039 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.42.temp_checkpoint
epoch: 44/100
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.42.temp_checkpoint
epoch: 44/100
| step   174 |  lr: 0.0002000 | loss  0.0210 | ms/batch 25295.92 |
| step   174 |  lr: 0.0002000 | loss  0.0210 | ms/batch 25295.92 |


100%|██████████| 26/26 [00:06<00:00,  4.23it/s]



-----------------------------------------------------------------------
| epoch  43 | step   176 | dev_acc  0.8627 | test_acc  0.0000 |
-----------------------------------------------------------------------
-----------------------------------------------------------------------
| epoch  43 | step   176 | dev_acc  0.8627 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.43.temp_checkpoint
model saved to /content/new_SP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.43.temp_checkpoint


###predict

In [ ]:
import numpy as np
import pandas as pd

SP_pred

In [ ]:
args["save_test_preds"]=True

k=5 #num of gnn layers

task_type="SP"
args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.best_model"
main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"
dataset="brain_teaser_ds"
model='roberta-large'

args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_dev.graph.adj.pk"
args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_test.graph.adj.pk"

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_dev.statement.jsonl"
args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_test.statement.jsonl"

args["best_model_save_dir"]=f"{save_dir_pref}/{task_type}_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/{task_type}_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"


In [ ]:
sp_test_pred=predict(args)

| num_concepts: 799273 |


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


inhouse? False
args.train_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_train.statement.jsonl
args.dev_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_dev.statement.jsonl
args.test_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_test.statement.jsonl
args.train_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_train.graph.adj.pk
args.dev_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_dev.graph.adj.pk
args.test_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/SP_test.graph.adj.pk
train_statement_path /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/SP_train.statement.jsonl


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

| ori_adj_len: mu 448.84 sigma 297.23 | adj_len: 183.54 | prune_rate： 0.78 | qc_num: 21.48 | ac_num: 5.64 |


100%|██████████| 26/26 [00:06<00:00,  3.81it/s]


dev_acc  0.9412


100%|██████████| 60/60 [00:13<00:00,  4.38it/s]



-----------------------------------------------------------------------


In [ ]:
len(sp_test_pred)

120

In [ ]:
SP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/SP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
print(len(SP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in SP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

SP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
SP_val_df.head(2)

120


,question,choice_list
0,"Everyone called him ""Batman,"" but he knew noth...","[He tries to be friendly., He is afraid others..."
1,"All of Mrs. Smith pets are dogs except one, an...",[Mrs.Smith has one additional pet that is neit...


In [ ]:
ids=[]
questions=[]
choice_lists=[]
predicted_labels=[]

for i in tqdm(range(len(SP_val_df))):
  sample=SP_val_df.iloc[i]
  pred=sp_test_pred[i]
  ids.append(i)
  questions.append(sample['question'].strip())
  predicted_labels.append(pred)
  choice_lists.append(sample['choice_list'])

temp_dict={'id':ids,
'question':questions,
'choice_list':choice_lists,
'label':predicted_labels}

sp_new_df=pd.DataFrame(data=temp_dict)
sp_new_df

100%|██████████| 120/120 [00:00<00:00, 17489.63it/s]


,id,question,choice_list,label
0,0,"Everyone called him ""Batman,"" but he knew noth...","[He tries to be friendly., He is afraid others...",2
1,1,"All of Mrs. Smith pets are dogs except one, an...",[Mrs.Smith has one additional pet that is neit...,2
2,2,Three doors are present. Every door has a way ...,"[The third door, the alligator can be tierd an...",1
3,3,Tom and his younger sister were fighting. Thei...,"[Tom's mother slid a newspaper under a door, e...",1
4,4,A woman sells a bottle of perfume to a man for...,"[Fifteen dollars., Fifteen dollars plus the pe...",1
...,...,...,...,...
115,115,How was it possible for a cowboy to arrive in ...,"[His horse is named Wednesday., Friday and Sat...",0
116,116,Natives of the Arctic will never eat a penguin...,"[Penguins only live in Antarctica., Penguins' ...",0
117,117,A full water glass was in the hands of the mag...,[Surface tension prevents water from spilling....,2
118,118,Which could see the best in complete darkness?...,"[Bat., Tiger., Owl., None of above.]",3


In [ ]:
import datetime
dt = datetime.datetime.today().strftime('%Y_%m_%d')
dt

'2023_12_13'

In [ ]:
sp_new_df.to_csv(f"/content/drive/MyDrive/brain_teaser/SP_qagnn_predictions_{dt}.csv", index=False)

In [ ]:
res=[str(x) for x in sp_new_df['label'].values]
with open('answer_sen.txt', 'w') as f:
    f.write('\n'.join(res))

WP_pred

In [ ]:
args["save_test_preds"]=True

k=5 #num of gnn layers
task_type="WP"
args["load_model_path"]=f"/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/{task_type}_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.26.best_model"

main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"
dataset="brain_teaser_ds"
model='roberta-large'

args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_dev.graph.adj.pk"
args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/{task_type}_test.graph.adj.pk"

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_dev.statement.jsonl"
args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/{task_type}_test.statement.jsonl"

args["best_model_save_dir"]=f"{save_dir_pref}/{task_type}_best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/{task_type}_log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"


In [ ]:
args["log_save_dir"]

'/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_log_brain_teaser_ds__enc-roberta-large_kchoice_list__gnndim200__bs128__seed0'

In [ ]:
wp_test_pred=predict(args)

| num_concepts: 799273 |


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


inhouse? False
args.train_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl
args.dev_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_dev.statement.jsonl
args.test_statements /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_test.statement.jsonl
args.train_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_train.graph.adj.pk
args.dev_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_dev.graph.adj.pk
args.test_adj /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/graph/WP_test.graph.adj.pk
train_statement_path /content/drive/MyDrive/brain_teaser/datasets/brain_teaser_ds/statement/WP_train.statement.jsonl


100%|██████████| 40/40 [00:00<00:00, 1627.70it/s]


num_choice 4
| ori_adj_len: mu 98.33 sigma 98.79 | adj_len: 86.43 | prune_rate： 0.11 | qc_num: 6.04 | ac_num: 2.41 |
| ori_adj_len: mu 82.68 sigma 82.56 | adj_len: 76.02 | prune_rate： 0.05 | qc_num: 5.01 | ac_num: 2.23 |


...

| ori_adj_len: mu 112.62 sigma 128.16 | adj_len: 91.44 | prune_rate： 0.14 | qc_num: 6.42 | ac_num: 3.11 |


100%|██████████| 20/20 [00:04<00:00,  4.20it/s]


dev_acc  0.8500


100%|██████████| 60/60 [00:14<00:00,  4.05it/s]



-----------------------------------------------------------------------


In [ ]:
len(wp_test_pred)

120

In [ ]:
WP_dataset_val = np.load('/content/drive/MyDrive/brain_teaser/datasets/WP_eval_data_for_practice.npy', allow_pickle=True,encoding='bytes')
print(len(WP_dataset_val))

data_dict = {'question':[],
             'choice_list':[]}
for i in WP_dataset_val:
  for k,v in i.items():
    data_dict[k].append(v)

WP_val_df = pd.DataFrame(data_dict, columns=['question', 'choice_list'])
WP_val_df.head(2)

120


,question,choice_list
0,What kind of nut has no shell?,"[A peanut., A Doughnut., A walnut., None of ab..."
1,Which nut doesn't have a shell?,"[A Doughnut., A walnut., A peanut., None of ab..."


In [ ]:
ids=[]
questions=[]
choice_lists=[]
predicted_labels=[]

for i in tqdm(range(len(WP_val_df))):
  sample=WP_val_df.iloc[i]
  pred=wp_test_pred[i]
  ids.append(i)
  questions.append(sample['question'].strip())
  predicted_labels.append(pred)
  choice_lists.append(sample['choice_list'])

temp_dict={'id':ids,
'question':questions,
'choice_list':choice_lists,
'label':predicted_labels}

WP_new_df=pd.DataFrame(data=temp_dict)
WP_new_df

100%|██████████| 120/120 [00:00<00:00, 17291.94it/s]


,id,question,choice_list,label
0,0,What kind of nut has no shell?,"[A peanut., A Doughnut., A walnut., None of ab...",1
1,1,Which nut doesn't have a shell?,"[A Doughnut., A walnut., A peanut., None of ab...",0
2,2,Which type of bell doesn't make a sound?,"[A fire bell., A cow bell., A Bluebell., None ...",2
3,3,What does a stone become when in the water?,"[A whetstone., A limestone, A sandstone., None...",0
4,4,What changes a stone makes when submerged in w...,"[A whetstone., A sandstone., A limestone, None...",0
...,...,...,...,...
115,115,hijklmno' stands for what liquid?,"[Milk. ""H L M N O"" is similar as the nursery r...",2
116,116,acdfgnijkmn' stands for what award?,"[Nobel. ""Acdfgnijkmn"" is 'no B E L"" in the alp...",0
117,117,You are in a place called Wall's World and the...,[Each word in Wall's World must contain less t...,2
118,118,"You are in Wall's World, where there is only o...",[Each word in Wall's World must contain more t...,1


In [ ]:
dt = datetime.datetime.today().strftime('%Y_%m_%d')
dt

'2023_12_13'

In [ ]:
WP_new_df.to_csv(f"/content/drive/MyDrive/brain_teaser/WP_qagnn_predictions_{dt}.csv", index=False)

In [ ]:
res=[str(x) for x in WP_new_df['label'].values]
with open('answer_word.txt', 'w') as f:
    f.write('\n'.join(res))

###training logs

12_11_2023

WP continue training, loading from best saved model (with new concept extraction code)

In [ ]:
elr=1e-5 #encoder learning rate
args["encoder_lr"]=elr

args['log_interval']=2
main_5(args, global_step=81, best_dev_epoch=26, is_finetuning_on_new_ds=False)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 2, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 1e-05, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_t

100%|██████████| 40/40 [00:00<00:00, 1465.81it/s]


num_choice 4
| ori_adj_len: mu 98.33 sigma 98.79 | adj_len: 86.43 | prune_rate： 0.11 | qc_num: 6.04 | ac_num: 2.41 |
| ori_adj_len: mu 82.68 sigma 82.56 | adj_len: 76.02 | prune_rate： 0.05 | qc_num: 5.01 | ac_num: 2.23 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loading and initializing model from /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.26.best_model
previous best_dev_acc: 0.85
current training epoch start from: 27
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:

...

-----------------------------------------------------------------------
| epoch  34 | step   105 | dev_acc  0.7750 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.34.temp_checkpoint
epoch: 36/100
| step   105 |  lr: 0.0000100 | loss  0.3237 | ms/batch 8376.35 |
| step   107 |  lr: 0.0000100 | loss  0.3535 | ms/batch 15004.57 |


100%|██████████| 20/20 [00:04<00:00,  4.64it/s]


-----------------------------------------------------------------------
| epoch  35 | step   108 | dev_acc  0.7500 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.35.temp_checkpoint
epoch: 37/100
| step   109 |  lr: 0.0000100 | loss  0.3179 | ms/batch 16780.31 |


100%|██████████| 20/20 [00:04<00:00,  4.64it/s]


-----------------------------------------------------------------------
| epoch  36 | step   111 | dev_acc  0.7250 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.36.temp_checkpoint


In [ ]:
elr=0.0002
args["encoder_lr"]=elr

args['log_interval']=2
main_5(args, global_step=81, best_dev_epoch=26, is_finetuning_on_new_ds=False)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 2, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch  34 | step   105 | dev_acc  0.7500 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.34.temp_checkpoint
epoch: 36/100
| step   105 |  lr: 0.0002000 | loss  0.3235 | ms/batch 8382.24 |
| step   107 |  lr: 0.0002000 | loss  0.3534 | ms/batch 14971.52 |


100%|██████████| 20/20 [00:04<00:00,  4.65it/s]


-----------------------------------------------------------------------
| epoch  35 | step   108 | dev_acc  0.7250 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.35.temp_checkpoint
epoch: 37/100
| step   109 |  lr: 0.0002000 | loss  0.3182 | ms/batch 16841.73 |


100%|██████████| 20/20 [00:04<00:00,  4.65it/s]


-----------------------------------------------------------------------
| epoch  36 | step   111 | dev_acc  0.7250 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.36.temp_checkpoint


WP

In [ ]:
args['log_interval']=2
main_5(args, global_step=0, best_dev_epoch=0, is_finetuning_on_new_ds=False)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/WP_log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 2, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_

100%|██████████| 40/40 [00:00<00:00, 1424.83it/s]


num_choice 4
| ori_adj_len: mu 98.31 sigma 98.81 | adj_len: 86.41 | prune_rate： 0.11 | qc_num: 6.04 | ac_num: 2.41 |
| ori_adj_len: mu 82.68 sigma 82.56 | adj_len: 76.02 | prune_rate： 0.05 | qc_num: 5.01 | ac_num: 2.23 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:cuda:0
	gnn.emb_score.bias                           	trainable	torch.Size([100])	device:cuda:0
	gnn.edge_encoder.0.weight                    	trainable	torch.Size([200, 47])	device:cuda:0
	gnn.edge_encoder.0.bias                      	trainable	torch.Size([

...

-----------------------------------------------------------------------
| epoch  34 | step   105 | dev_acc  0.7500 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.34.temp_checkpoint
epoch: 36/100
| step   105 |  lr: 0.0002000 | loss  0.2194 | ms/batch 19476.78 |
| step   107 |  lr: 0.0002000 | loss  0.1683 | ms/batch 33957.90 |


100%|██████████| 20/20 [00:04<00:00,  4.42it/s]


-----------------------------------------------------------------------
| epoch  35 | step   108 | dev_acc  0.7000 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.35.temp_checkpoint
epoch: 37/100
| step   109 |  lr: 0.0002000 | loss  0.1767 | ms/batch 38653.55 |


100%|██████████| 20/20 [00:04<00:00,  4.46it/s]


-----------------------------------------------------------------------
| epoch  36 | step   111 | dev_acc  0.7000 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/WP_checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.36.temp_checkpoint


12-8-2023

SP

args["encoder_lr"]=1e-5

In [ ]:
main_5(args, global_step=88, best_dev_epoch=21, is_finetuning_on_new_ds=False)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 1e-05, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train

100%|██████████| 51/51 [00:00<00:00, 812.44it/s]


num_choice 4
| ori_adj_len: mu 394.59 sigma 290.55 | adj_len: 177.47 | prune_rate： 0.72 | qc_num: 18.76 | ac_num: 5.36 |
| ori_adj_len: mu 386.00 sigma 252.60 | adj_len: 180.20 | prune_rate： 0.80 | qc_num: 17.96 | ac_num: 6.38 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loading and initializing model from /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.best_model
previous best_dev_acc: 0.9411764705882353
current training epoch start from: 22
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 10

...

-----------------------------------------------------------------------
| epoch  29 | step   120 | dev_acc  0.9216 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.29.temp_checkpoint
epoch: 31/100


100%|██████████| 26/26 [00:06<00:00,  4.02it/s]


-----------------------------------------------------------------------
| epoch  30 | step   124 | dev_acc  0.9216 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.30.temp_checkpoint
epoch: 32/100
| step   124 |  lr: 0.0000100 | loss  0.0004 | ms/batch 4005.60 |


100%|██████████| 26/26 [00:06<00:00,  4.04it/s]


-----------------------------------------------------------------------
| epoch  31 | step   128 | dev_acc  0.9216 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.31.temp_checkpoint


args["encoder_lr"]=2e-4

In [ ]:
main_5(args, global_step=0, best_dev_epoch=0, is_finetuning_on_new_ds=True)

{'enc': 'roberta-large', 'encoder': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'brain_teaser_ds', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'trai

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

100%|██████████| 26/26 [00:06<00:00,  4.07it/s]


-----------------------------------------------------------------------
| epoch  21 | step    88 | dev_acc  0.9412 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.best_model
epoch: 23/100
| step    89 |  lr: 0.0002000 | loss  0.0008 | ms/batch 17157.95 |


100%|██████████| 26/26 [00:06<00:00,  4.05it/s]


-----------------------------------------------------------------------
| epoch  22 | step    92 | dev_acc  0.8627 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_brain_teaser_ds__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.22.temp_checkpoint
epoch: 24/100


KeyboardInterrupt: ignored

##<font color=pink>Train Riddle_sense</font>

In [ ]:
dataset="riddle_sense"
model='roberta-large'

# elr=1e-5 #encoder learning rate
elr=2e-4 #encoder learning rate
dlr=1e-3  #decoder learing rate
batch_size=128 #128 default
mbs=2   #mini-batch-size
n_epochs=100 #def 100
num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges
seed=0

k=5 #num of gnn layers
gnndim=200    #dimension of the GNN layers def 200

!echo "***** hyperparameters *****"
!echo "dataset: $dataset"
!echo "enc_name: $model"
!echo "batch_size: $batch_size"
!echo "learning_rate: elr $elr dlr $dlr"
!echo "gnn: dim $gnndim layer $k"
!echo "Epochs: $n_epochs"
!echo "******************************"

main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"

if not os.path.exists(save_dir_pref):
  %mkdir -p $save_dir_pref
%mkdir -p logs

args=dict()

args['warmup_steps']=0

args["encoder"]=model
args['enc']=model

# args["save_dir"]=f"{save_dir_pref}/enc-{model}_dataset_{dataset}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["best_model_save_dir"]=f"{save_dir_pref}/best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["checkpoint_save_dir"]=f"{save_dir_pref}/checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"

args["save_best_model"]=True
args['save_test_preds']=False
args["save_checkpoint"]=True

args['loss']='cross_entropy'
args['lr_schedule']='fixed'
args['bs']=batch_size
args['batch_size']=batch_size


args['max_grad_norm']=1.0
args['me']=10
args['log_interval']=5
args['debug']=False
args["optim"]="radam"

args["mode"]="train"    #choices=['train', 'eval_detail'], help='run training or evaluation
args["dataset"]=dataset

args["k"]=k   #perform k-layer message passing
args["gnn_dim"]=gnndim
args["encoder_lr"]=elr
args["decoder_lr"]=dlr
args["batch_size"]=batch_size
args["mini_batch_size"]=mbs
args["fp16"]=False   #use fp16 training. this requires torch>=1.6.0
args["seed"]=seed
args["num_relation"]=num_relation
args["n_epochs"]=n_epochs
args["max_epochs_before_stop"]=10
args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/dev.graph.adj.pk"
args["test_adj"]=None

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/dev.statement.jsonl"
args["test_statements"]=None


args["cuda"]=True
# args["ent_emb_paths"]=['data/cpnet/tzw.ent.npy']
args["ent_emb_paths"]=['/content/drive/MyDrive/brain_teaser/datasets/cpnet/tzw.ent.npy']
args["max_seq_len"]=128
args["inhouse"]=False
args["inhouse_train_qids"]=f'{main_dir}/datasets/{dataset}/inhouse_split_qids.txt'
args["weight_decay"]=1e-2

args['encoder_layer']=-1
# args['elr']=2e-4
# args["encoder_lr"]=2e-4


# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.7.best_model'

# args["load_model_path"]=None
args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.12.best_model'
# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.15.temp_checkpoint'

***** hyperparameters *****
dataset: riddle_sense
enc_name: roberta-large
batch_size: 128
learning_rate: elr 1e-05 dlr 0.001
gnn: dim 200 layer 5
Epochs: 100
******************************


12_8_2023

In [ ]:
main_5(args, global_step=364, best_dev_epoch=12, is_finetuning_on_new_ds=False)

{'warmup_steps': 0, 'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/co

100%|██████████| 1021/1021 [00:00<00:00, 1027.97it/s]


num_choice 5
| ori_adj_len: mu 167.61 sigma 172.13 | adj_len: 115.70 | prune_rate： 0.30 | qc_num: 9.78 | ac_num: 1.64 |
| ori_adj_len: mu 194.18 sigma 175.85 | adj_len: 131.31 | prune_rate： 0.36 | qc_num: 10.75 | ac_num: 1.49 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loading and initializing model from /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.12.best_model
previous best_dev_acc: 0.5974534769833496
current training epoch start from: 13
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])

...

100%|██████████| 511/511 [02:15<00:00,  3.78it/s]


-----------------------------------------------------------------------
| epoch  21 | step   616 | dev_acc  0.6102 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.21.temp_checkpoint
epoch: 23/100
| step   619 |  lr: 0.0002000 | loss  0.0507 | ms/batch 17180.14 |
| step   624 |  lr: 0.0002000 | loss  0.0621 | ms/batch 21202.27 |
| step   629 |  lr: 0.0002000 | loss  0.0678 | ms/batch 21286.56 |
| step   634 |  lr: 0.0002000 | loss  0.0718 | ms/batch 21373.93 |
| step   639 |  lr: 0.0002000 | loss  0.0562 | ms/batch 21306.40 |


100%|██████████| 511/511 [02:15<00:00,  3.78it/s]


-----------------------------------------------------------------------
| epoch  22 | step   644 | dev_acc  0.6092 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.22.temp_checkpoint
epoch: 24/100
| step   644 |  lr: 0.0002000 | loss  0.0746 | ms/batch 4136.28 |
| step   649 |  lr: 0.0002000 | loss  0.0528 | ms/batch 21733.76 |
| step   654 |  lr: 0.0002000 | loss  0.0423 | ms/batch 21178.99 |


KeyboardInterrupt: ignored

12_7_2023_  Loading from csqa

In [ ]:
main_5(args, global_step=0, best_dev_epoch=0, is_finetuning_on_new_ds=False)

{'warmup_steps': 0, 'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/co

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch  13 | step   392 | dev_acc  0.5788 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.13.temp_checkpoint
epoch: 15/100
| step   394 |  lr: 0.0002000 | loss  0.1012 | ms/batch 28446.54 |
| step   399 |  lr: 0.0002000 | loss  0.0627 | ms/batch 47545.48 |
| step   404 |  lr: 0.0002000 | loss  0.0729 | ms/batch 47448.40 |
| step   409 |  lr: 0.0002000 | loss  0.0865 | ms/batch 47490.79 |
| step   414 |  lr: 0.0002000 | loss  0.0901 | ms/batch 47720.88 |
| step   419 |  lr: 0.0002000 | loss  0.0962 | ms/batch 41499.37 |


100%|██████████| 511/511 [02:17<00:00,  3.72it/s]


-----------------------------------------------------------------------
| epoch  14 | step   420 | dev_acc  0.5426 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.14.temp_checkpoint
epoch: 16/100
| step   424 |  lr: 0.0002000 | loss  0.0940 | ms/batch 47975.36 |
| step   429 |  lr: 0.0002000 | loss  0.0392 | ms/batch 47551.85 |
| step   434 |  lr: 0.0002000 | loss  0.0479 | ms/batch 47380.80 |
| step   439 |  lr: 0.0002000 | loss  0.0449 | ms/batch 47580.19 |
| step   444 |  lr: 0.0002000 | loss  0.0420 | ms/batch 47044.22 |


100%|██████████| 511/511 [02:16<00:00,  3.73it/s]


-----------------------------------------------------------------------
| epoch  15 | step   448 | dev_acc  0.5828 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.15.temp_checkpoint
epoch: 17/100
| step   449 |  lr: 0.0002000 | loss  0.0477 | ms/batch 19053.58 |


###12_5_2023

In [ ]:
main_5(args)

{'warmup_steps': 0, 'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/co

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

loading and initializing model from /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.18.temp_checkpoint
previous best_dev_acc: 0.604309500489716
current training epoch start from: 19
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 1

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
<ipython-input-9-267d489110eb>:61: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_schedu

| step     4 |  lr: 0.0002000 | loss  0.1085 | ms/batch 21649.84 |
| step     9 |  lr: 0.0002000 | loss  0.1172 | ms/batch 21968.65 |
| step    14 |  lr: 0.0002000 | loss  0.1028 | ms/batch 22209.29 |
| step    19 |  lr: 0.0002000 | loss  0.1071 | ms/batch 22187.86 |
| step    24 |  lr: 0.0002000 | loss  0.1289 | ms/batch 22110.62 |


100%|██████████| 511/511 [02:20<00:00,  3.65it/s]


-----------------------------------------------------------------------
| epoch  19 | step    28 | dev_acc  0.5877 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.19.temp_checkpoint


####previous training

In [ ]:
main_5(args)

{'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 150, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch  18 | step   308 | dev_acc  0.5935 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.18.temp_checkpoint
epoch: 20/100
| step   309 |  lr: 0.0002000 | loss  0.1408 | ms/batch 8210.76 |
| step   314 |  lr: 0.0002000 | loss  0.1495 | ms/batch 20112.61 |
| step   319 |  lr: 0.0002000 | loss  0.1351 | ms/batch 20318.54 |
| step   324 |  lr: 0.0002000 | loss  0.1608 | ms/batch 20221.87 |
| step   329 |  lr: 0.0002000 | loss  0.1628 | ms/batch 20040.86 |
| step   334 |  lr: 0.0002000 | loss  0.1374 | ms/batch 20301.88 |


100%|██████████| 511/511 [02:09<00:00,  3.96it/s]


-----------------------------------------------------------------------
| epoch  19 | step   336 | dev_acc  0.5759 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.19.temp_checkpoint
epoch: 21/100
| step   339 |  lr: 0.0002000 | loss  0.1400 | ms/batch 16101.61 |
| step   344 |  lr: 0.0002000 | loss  0.1328 | ms/batch 20124.45 |
| step   349 |  lr: 0.0002000 | loss  0.1098 | ms/batch 19988.15 |
| step   354 |  lr: 0.0002000 | loss  0.1333 | ms/batch 20016.34 |
| step   359 |  lr: 0.0002000 | loss  0.0814 | ms/batch 20663.57 |


100%|██████████| 511/511 [02:09<00:00,  3.96it/s]


-----------------------------------------------------------------------
| epoch  20 | step   364 | dev_acc  0.5857 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_riddle_sense__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.20.temp_checkpoint


In [ ]:
main_5(args)

{'save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/enc-roberta-large_dataset_riddle_sense_k5__gnndim200__bs128__seed0', 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 150, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'encoder': 'roberta-large', 'enc': 'roberta-large', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/train.graph.adj.pk', 'dev_adj': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/dev.graph.adj.pk', 'test_adj': None, 'train_statements': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl', 'dev_statements': '/content/drive/MyDrive/brain_teaser/datasets/riddle_se

100%|██████████| 1021/1021 [00:01<00:00, 983.11it/s]


num_choice 5
| ori_adj_len: mu 167.61 sigma 172.13 | adj_len: 115.70 | prune_rate： 0.30 | qc_num: 9.78 | ac_num: 1.64 |
| ori_adj_len: mu 194.18 sigma 175.85 | adj_len: 131.31 | prune_rate： 0.36 | qc_num: 10.75 | ac_num: 1.49 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:cuda:0
	gnn.emb_score.bias                           	trainable	torch.Size([100])	device:cuda:0
	gnn.edge_encoder.0.weight                    	trainable	torch.Size([200, 47])	device:cuda:0
	gnn.edge_encoder.0.bias                      	trainable	torch.Size([

...

-----------------------------------------------------------------------
| epoch   6 | step   196 | dev_acc  0.4192 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/enc-roberta-large_dataset_riddle_sense_k5__gnndim200__bs128__seed0/model.pt.6.best_model
epoch: 8/100
| step   199 |  lr: 0.0002000 | loss  0.6125 | ms/batch 39363.81 |
| step   204 |  lr: 0.0002000 | loss  0.4966 | ms/batch 48888.00 |
| step   209 |  lr: 0.0002000 | loss  0.5219 | ms/batch 48681.96 |
| step   214 |  lr: 0.0002000 | loss  0.5301 | ms/batch 48757.25 |
| step   219 |  lr: 0.0002000 | loss  0.5100 | ms/batch 48969.34 |


100%|██████████| 511/511 [02:22<00:00,  3.59it/s]


-----------------------------------------------------------------------
| epoch   7 | step   224 | dev_acc  0.5886 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/enc-roberta-large_dataset_riddle_sense_k5__gnndim200__bs128__seed0/model.pt.7.best_model
epoch: 9/100
| step   224 |  lr: 0.0002000 | loss  0.4410 | ms/batch 10164.26 |
| step   229 |  lr: 0.0002000 | loss  0.3228 | ms/batch 48993.60 |
| step   234 |  lr: 0.0002000 | loss  0.2961 | ms/batch 48882.93 |
| step   239 |  lr: 0.0002000 | loss  0.3039 | ms/batch 49257.75 |
| step   244 |  lr: 0.0002000 | loss  0.4062 | ms/batch 48602.36 |
| step   249 |  lr: 0.0002000 | loss  0.3589 | ms/batch 48626.07 |


100%|██████████| 511/511 [02:22<00:00,  3.59it/s]


-----------------------------------------------------------------------
| epoch   8 | step   252 | dev_acc  0.5828 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/enc-roberta-large_dataset_riddle_sense_k5__gnndim200__bs128__seed0/model.pt.8.temp_checkpoint
epoch: 10/100
| step   254 |  lr: 0.0002000 | loss  0.2691 | ms/batch 29805.56 |
| step   259 |  lr: 0.0002000 | loss  0.1771 | ms/batch 48632.33 |


####batch_size 64

In [ ]:
main_5(args)

{'save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/enc-roberta-large_dataset_riddle_sense_k5__gnndim200__bs128__seed0', 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 150, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 5, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'riddle_sense', 'encoder': 'bert-large-uncased', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 5, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/train.graph.adj.pk', 'dev_adj': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/graph/dev.graph.adj.pk', 'test_adj': None, 'train_statements': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/train.statement.jsonl', 'dev_statements': '/content/drive/MyDrive/brain_teaser/datasets/riddle_sense/statement/dev.sta

100%|██████████| 1021/1021 [00:01<00:00, 1003.92it/s]


num_choice 5
| ori_adj_len: mu 167.61 sigma 172.13 | adj_len: 115.70 | prune_rate： 0.30 | qc_num: 9.78 | ac_num: 1.64 |
| ori_adj_len: mu 194.18 sigma 175.85 | adj_len: 131.31 | prune_rate： 0.36 | qc_num: 10.75 | ac_num: 1.49 |
args.num_relation 38
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:cuda:0
	g

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
<ipython-input-6-267d489110eb>:61: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1519.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_schedu

| step     4 |  lr: 0.0002000 | loss  1.6268 | ms/batch 21560.42 |
| step     9 |  lr: 0.0002000 | loss  1.5908 | ms/batch 21041.02 |
| step    14 |  lr: 0.0002000 | loss  1.4988 | ms/batch 20808.12 |
| step    19 |  lr: 0.0002000 | loss  1.4786 | ms/batch 20767.71 |
| step    24 |  lr: 0.0002000 | loss  1.4344 | ms/batch 20897.00 |


...

100%|██████████| 511/511 [02:17<00:00,  3.73it/s]


-----------------------------------------------------------------------
| epoch   1 | step    56 | dev_acc  0.2674 | test_acc  0.0000 |
-----------------------------------------------------------------------
epoch: 3/5
| step    59 |  lr: 0.0002000 | loss  1.2648 | ms/batch 16738.92 |
| step    64 |  lr: 0.0002000 | loss  1.2587 | ms/batch 20827.85 |
| step    69 |  lr: 0.0002000 | loss  1.2634 | ms/batch 20788.94 |
| step    74 |  lr: 0.0002000 | loss  1.2352 | ms/batch 21340.64 |
| step    79 |  lr: 0.0002000 | loss  1.2407 | ms/batch 22293.98 |


100%|██████████| 511/511 [02:28<00:00,  3.44it/s]


-----------------------------------------------------------------------
| epoch   2 | step    84 | dev_acc  0.2684 | test_acc  0.0000 |
-----------------------------------------------------------------------
epoch: 4/5


KeyboardInterrupt: ignored

##Train CSQA

In [ ]:
# %cd "/content/data"

# %cp -av /content/data/cpnet   /content/drive/MyDrive/brain_teaser/datasets/

In [ ]:
dataset="csqa"
model='roberta-large'

elr=1e-5 #encoder learning rate
dlr=1e-3  #decoder learing rate
batch_size=64 #128 default
mbs=2   #mini-batch-size
n_epochs=100 #def 100
num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges
seed=0

k=5 #num of gnn layers
gnndim=200    #dimension of the GNN layers def 200

!echo "***** hyperparameters *****"
!echo "dataset: $dataset"
!echo "enc_name: $model"
!echo "batch_size: $batch_size"
!echo "learning_rate: elr $elr dlr $dlr"
!echo "gnn: dim $gnndim layer $k"
!echo "Epochs: $n_epochs"
!echo "******************************"

main_dir='/content/drive/MyDrive/brain_teaser'
save_dir_pref=f"{main_dir}/QA_GNN/saved_models"

if not os.path.exists(save_dir_pref):
  %mkdir -p $save_dir_pref
%mkdir -p logs

args=dict()
args["encoder"]=model
args['enc']=model

# args["save_dir"]=f"{save_dir_pref}/enc-{model}_dataset_{dataset}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["best_model_save_dir"]=f"{save_dir_pref}/best_model_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["checkpoint_save_dir"]=f"{save_dir_pref}/checkpoint_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"
args["log_save_dir"]=f"{save_dir_pref}/log_{dataset}__enc-{model}_k{k}__gnndim{gnndim}__bs{batch_size}__seed{seed}"

args["save_best_model"]=True
args['save_test_preds']=False
args["save_checkpoint"]=True

args['loss']='cross_entropy'
args['lr_schedule']='fixed'
args['bs']=batch_size
args['batch_size']=batch_size
args['warmup_steps']=0
args['max_grad_norm']=1.0
args['me']=10
args['log_interval']=10
args['debug']=False
args["optim"]="radam"

args["mode"]="train"    #choices=['train', 'eval_detail'], help='run training or evaluation
args["dataset"]=dataset

args["k"]=k   #perform k-layer message passing
args["gnn_dim"]=gnndim
args["encoder_lr"]=elr
args["decoder_lr"]=dlr
args["batch_size"]=batch_size
args["mini_batch_size"]=mbs
args["fp16"]=False   #use fp16 training. this requires torch>=1.6.0
args["seed"]=seed
args["num_relation"]=num_relation
args["n_epochs"]=n_epochs
args["max_epochs_before_stop"]=10
args["train_adj"]=f"{main_dir}/datasets/{dataset}/graph/train.graph.adj.pk"
args["dev_adj"]=f"{main_dir}/datasets/{dataset}/graph/dev.graph.adj.pk"
# args["test_adj"]=f"{main_dir}/datasets/{dataset}/graph/test.graph.adj.pk"
args["test_adj"]=None

args["train_statements"]=f"{main_dir}/datasets/{dataset}/statement/train.statement.jsonl"
args["dev_statements"]=f"{main_dir}/datasets/{dataset}/statement/dev.statement.jsonl"
# args["test_statements"]=f"{main_dir}/datasets/{dataset}/statement/test.statement.jsonl"
args["test_statements"]=None


args["cuda"]=True
# args["ent_emb_paths"]=['data/cpnet/tzw.ent.npy']
args["ent_emb_paths"]=['/content/drive/MyDrive/brain_teaser/datasets/cpnet/tzw.ent.npy']

args["max_seq_len"]=128
args["inhouse"]=True
args["inhouse_train_qids"]=f'{main_dir}/datasets/{dataset}/inhouse_split_qids.txt'
args["weight_decay"]=1e-2

args['encoder_layer']=-1
# args['elr']=2e-4
# args["encoder_lr"]=2e-4

# args["load_model_path"]=None
# args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.11.best_model'
args["load_model_path"]='/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.5.temp_checkpoint'

***** hyperparameters *****
dataset: csqa
enc_name: roberta-large
batch_size: 64
learning_rate: elr 1e-05 dlr 0.001
gnn: dim 200 layer 5
Epochs: 100
******************************


12_6_2023

bs 64 lr corrected

In [ ]:
main_5(args)

{'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 64, 'batch_size': 64, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 10, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'csqa', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 1e-05, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_teaser/data

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

-----------------------------------------------------------------------
| epoch   6 | step   133 | dev_acc  0.4447 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.6.temp_checkpoint
epoch: 8/100
| step   139 |  lr: 0.0000100 | loss  1.2570 | ms/batch 7284.30 |
| step   149 |  lr: 0.0000100 | loss  1.2927 | ms/batch 10351.27 |
| step   159 |  lr: 0.0000100 | loss  1.2947 | ms/batch 10409.26 |
| step   169 |  lr: 0.0000100 | loss  1.3034 | ms/batch 10456.52 |
| step   179 |  lr: 0.0000100 | loss  1.2254 | ms/batch 10347.60 |
| step   189 |  lr: 0.0000100 | loss  1.3117 | ms/batch 10295.37 |
| step   199 |  lr: 0.0000100 | loss  1.2715 | ms/batch 10352.63 |
| step   209 |  lr: 0.0000100 | loss  1.2646 | ms/batch 10408.71 |
| step   219 |  lr: 0.0000100 | loss  1.2684 | ms/batch 10560.96 |
| step   229 |  l

100%|██████████| 611/611 [02:34<00:00,  3.95it/s]


-----------------------------------------------------------------------
| epoch   7 | step   266 | dev_acc  0.4636 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.7.best_model
epoch: 9/100
| step   269 |  lr: 0.0000100 | loss  1.2749 | ms/batch 4323.70 |
| step   279 |  lr: 0.0000100 | loss  1.2471 | ms/batch 10343.75 |
| step   289 |  lr: 0.0000100 | loss  1.2502 | ms/batch 10419.99 |
| step   299 |  lr: 0.0000100 | loss  1.2816 | ms/batch 10400.18 |
| step   309 |  lr: 0.0000100 | loss  1.2928 | ms/batch 10375.97 |
| step   319 |  lr: 0.0000100 | loss  1.2789 | ms/batch 10489.82 |
| step   329 |  lr: 0.0000100 | loss  1.2987 | ms/batch 10515.48 |
| step   339 |  lr: 0.0000100 | loss  1.2975 | ms/batch 10308.94 |
| step   349 |  lr: 0.0000100 | loss  1.2464 | ms/batch 10400.14 |
| step   359 |  lr: 0.

100%|██████████| 611/611 [02:34<00:00,  3.95it/s]


-----------------------------------------------------------------------
| epoch   8 | step   399 | dev_acc  0.4505 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.8.temp_checkpoint
epoch: 10/100
| step   399 |  lr: 0.0000100 | loss  1.2229 | ms/batch  975.30 |
| step   409 |  lr: 0.0000100 | loss  1.2597 | ms/batch 10158.79 |


In [ ]:
main_5(args)

{'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 64, 'batch_size': 64, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 10, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'csqa', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 1e-05, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_teaser/data

100%|██████████| 1221/1221 [00:01<00:00, 1211.42it/s]


num_choice 5
| ori_adj_len: mu 121.54 sigma 94.04 | adj_len: 107.96 | prune_rate： 0.17 | qc_num: 7.43 | ac_num: 2.07 |
| ori_adj_len: mu 118.44 sigma 90.55 | adj_len: 106.55 | prune_rate： 0.15 | qc_num: 7.20 | ac_num: 2.05 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:cuda:0
	gnn.emb_score.bias                           	trainable	torch.Size([100])	device:cuda:0
	gnn.edge_encoder.0.weight                    	trainable	torch.Size([200, 47])	device:cuda:0
	gnn.edge_encoder.0.bias                      	trainable	torch.Size([

...

-----------------------------------------------------------------------
| epoch   3 | step   532 | dev_acc  0.4169 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.3.temp_checkpoint
epoch: 5/100
| step   539 |  lr: 0.0000100 | loss  1.3265 | ms/batch 17670.78 |
| step   549 |  lr: 0.0000100 | loss  1.3980 | ms/batch 22070.23 |
| step   559 |  lr: 0.0000100 | loss  1.3349 | ms/batch 22057.27 |
| step   569 |  lr: 0.0000100 | loss  1.3643 | ms/batch 22062.89 |
| step   579 |  lr: 0.0000100 | loss  1.3122 | ms/batch 22159.40 |
| step   589 |  lr: 0.0000100 | loss  1.4029 | ms/batch 22142.90 |
| step   599 |  lr: 0.0000100 | loss  1.3193 | ms/batch 22159.55 |
| step   609 |  lr: 0.0000100 | loss  1.3425 | ms/batch 22208.17 |
| step   619 |  lr: 0.0000100 | loss  1.2995 | ms/batch 22099.31 |
| step   629 |  

100%|██████████| 611/611 [02:28<00:00,  4.12it/s]


-----------------------------------------------------------------------
| epoch   4 | step   665 | dev_acc  0.4636 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.4.best_model
epoch: 6/100
| step   669 |  lr: 0.0000100 | loss  1.3272 | ms/batch 11165.60 |
| step   679 |  lr: 0.0000100 | loss  1.3244 | ms/batch 22093.74 |
| step   689 |  lr: 0.0000100 | loss  1.2879 | ms/batch 22096.13 |
| step   699 |  lr: 0.0000100 | loss  1.3315 | ms/batch 22026.27 |
| step   709 |  lr: 0.0000100 | loss  1.3419 | ms/batch 22021.71 |
| step   719 |  lr: 0.0000100 | loss  1.3272 | ms/batch 22182.72 |
| step   729 |  lr: 0.0000100 | loss  1.3525 | ms/batch 22135.32 |
| step   739 |  lr: 0.0000100 | loss  1.3348 | ms/batch 22041.18 |
| step   749 |  lr: 0.0000100 | loss  1.3403 | ms/batch 22150.84 |
| step   759 |  lr: 0

100%|██████████| 611/611 [02:28<00:00,  4.10it/s]


-----------------------------------------------------------------------
| epoch   5 | step   798 | dev_acc  0.4365 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs64__seed0/model.pt.5.temp_checkpoint
epoch: 7/100
| step   799 |  lr: 0.0000100 | loss  1.3417 | ms/batch 4466.51 |


In [ ]:
main_5(args)

{'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 10, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'csqa', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_tease

100%|██████████| 1221/1221 [00:00<00:00, 1263.61it/s]


num_choice 5
| ori_adj_len: mu 121.54 sigma 94.04 | adj_len: 107.96 | prune_rate： 0.17 | qc_num: 7.43 | ac_num: 2.07 |
| ori_adj_len: mu 118.44 sigma 90.55 | adj_len: 106.55 | prune_rate： 0.15 | qc_num: 7.20 | ac_num: 2.05 |
args.num_relation 38


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


loading and initializing model from /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.4.best_model
previous best_dev_acc: 0.44471744471744473
current training epoch start from: 5
parameters:
	concept_emb.emb.weight                       	fixed	torch.Size([799273, 1024])	device:cuda:0
	concept_emb.cpt_transform.weight             	trainable	torch.Size([200, 1024])	device:cuda:0
	concept_emb.cpt_transform.bias               	trainable	torch.Size([200])	device:cuda:0
	svec2nvec.weight                             	trainable	torch.Size([200, 1024])	device:cuda:0
	svec2nvec.bias                               	trainable	torch.Size([200])	device:cuda:0
	gnn.emb_node_type.weight                     	trainable	torch.Size([100, 4])	device:cuda:0
	gnn.emb_node_type.bias                       	trainable	torch.Size([100])	device:cuda:0
	gnn.emb_score.weight                         	trainable	torch.Size([100, 100])	device:c

...

100%|██████████| 611/611 [02:29<00:00,  4.09it/s]


-----------------------------------------------------------------------
| epoch  11 | step   469 | dev_acc  0.4627 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.11.best_model
epoch: 13/100
| step   469 |  lr: 0.0002000 | loss  1.2714 | ms/batch 1984.12 |
| step   479 |  lr: 0.0002000 | loss  1.2300 | ms/batch 20022.81 |
| step   489 |  lr: 0.0002000 | loss  1.2290 | ms/batch 20021.18 |
| step   499 |  lr: 0.0002000 | loss  1.2455 | ms/batch 19873.35 |
| step   509 |  lr: 0.0002000 | loss  1.2771 | ms/batch 20042.16 |
| step   519 |  lr: 0.0002000 | loss  1.2641 | ms/batch 19948.56 |
| step   529 |  lr: 0.0002000 | loss  1.2370 | ms/batch 19909.10 |


100%|██████████| 611/611 [02:28<00:00,  4.12it/s]


-----------------------------------------------------------------------
| epoch  12 | step   536 | dev_acc  0.4423 | test_acc  0.0000 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.12.temp_checkpoint
epoch: 14/100
| step   539 |  lr: 0.0002000 | loss  1.2387 | ms/batch 7964.93 |
| step   549 |  lr: 0.0002000 | loss  1.3072 | ms/batch 19859.10 |
| step   559 |  lr: 0.0002000 | loss  1.3086 | ms/batch 20040.47 |


KeyboardInterrupt: ignored

In [ ]:
main_5(args)

{'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 10, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'csqa', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_tease

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

100%|██████████| 621/621 [02:28<00:00,  4.18it/s]


-----------------------------------------------------------------------
| epoch   3 | step    67 | dev_acc  0.4275 | test_acc  0.4376 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.3.temp_checkpoint
epoch: 5/100
| step    69 |  lr: 0.0002000 | loss  1.3064 | ms/batch 13105.31 |
| step    79 |  lr: 0.0002000 | loss  1.3283 | ms/batch 44166.02 |
| step    89 |  lr: 0.0002000 | loss  1.2848 | ms/batch 44123.61 |
| step    99 |  lr: 0.0002000 | loss  1.2910 | ms/batch 43936.02 |
| step   109 |  lr: 0.0002000 | loss  1.2889 | ms/batch 44295.09 |
| step   119 |  lr: 0.0002000 | loss  1.2789 | ms/batch 44029.61 |
| step   129 |  lr: 0.0002000 | loss  1.3053 | ms/batch 44153.52 |


100%|██████████| 621/621 [02:30<00:00,  4.13it/s]


-----------------------------------------------------------------------
| epoch   4 | step   134 | dev_acc  0.4447 | test_acc  0.4343 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.4.best_model
epoch: 6/100


KeyboardInterrupt: ignored

12_5_2023

In [ ]:
main_5(args)

{'warmup_steps': 150, 'encoder': 'roberta-large', 'enc': 'roberta-large', 'best_model_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'checkpoint_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/checkpoint_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'log_save_dir': '/content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/log_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0', 'save_best_model': True, 'save_test_preds': False, 'save_checkpoint': True, 'loss': 'cross_entropy', 'lr_schedule': 'fixed', 'bs': 128, 'batch_size': 128, 'max_grad_norm': 1.0, 'me': 10, 'log_interval': 10, 'debug': False, 'optim': 'radam', 'mode': 'train', 'dataset': 'csqa', 'k': 5, 'gnn_dim': 200, 'encoder_lr': 0.0002, 'decoder_lr': 0.001, 'mini_batch_size': 2, 'fp16': False, 'seed': 0, 'num_relation': 38, 'n_epochs': 100, 'max_epochs_before_stop': 10, 'train_adj': '/content/drive/MyDrive/brain_tea

100%|██████████| 1221/1221 [00:01<00:00, 1009.69it/s]


num_choice 5
| ori_adj_len: mu 121.54 sigma 94.04 | adj_len: 107.96 | prune_rate： 0.17 | qc_num: 7.43 | ac_num: 2.07 |
| ori_adj_len: mu 118.44 sigma 90.55 | adj_len: 106.55 | prune_rate： 0.15 | qc_num: 7.20 | ac_num: 2.05 |


100%|██████████| 1140/1140 [00:01<00:00, 1092.86it/s]


| ori_adj_len: mu 119.20 sigma 93.49 | adj_len: 106.22 | prune_rate： 0.16 | qc_num: 7.38 | ac_num: 2.05 |
args.num_relation 38


...

100%|██████████| 621/621 [02:35<00:00,  3.99it/s]


-----------------------------------------------------------------------
| epoch   1 | step   134 | dev_acc  0.4128 | test_acc  0.4037 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.1.best_model
epoch: 3/100
| step   139 |  lr: 0.0002000 | loss  1.4026 | ms/batch 12431.32 |
| step   149 |  lr: 0.0002000 | loss  1.4133 | ms/batch 20723.68 |
| step   159 |  lr: 0.0002000 | loss  1.4051 | ms/batch 20733.92 |
| step   169 |  lr: 0.0002000 | loss  1.3773 | ms/batch 20727.97 |
| step   179 |  lr: 0.0002000 | loss  1.3584 | ms/batch 20597.03 |
| step   189 |  lr: 0.0002000 | loss  1.3836 | ms/batch 20720.01 |
| step   199 |  lr: 0.0002000 | loss  1.3330 | ms/batch 20529.39 |


100%|██████████| 621/621 [02:35<00:00,  3.99it/s]


-----------------------------------------------------------------------
| epoch   2 | step   201 | dev_acc  0.4447 | test_acc  0.4150 |
-----------------------------------------------------------------------
model saved to /content/drive/MyDrive/brain_teaser/QA_GNN/saved_models/best_model_csqa__enc-roberta-large_k5__gnndim200__bs128__seed0/model.pt.2.best_model
epoch: 4/100


KeyboardInterrupt: ignored

###previous trainings

batch size 128

In [ ]:
# main_5(args)

In [ ]:
'| epoch   0 | step  1063 | dev_acc  0.4201 | test_acc  0.4061 |'
'| epoch   1 | step  2126 | dev_acc  0.4242 | test_acc  0.4102 |'
'| epoch   2 | step  3189 | dev_acc  0.4177 | test_acc  0.4198 |'

bartch size 64

In [ ]:
'| epoch   0 | step   266 | dev_acc  0.4341 | test_acc  0.4303 |'
'| epoch   1 | step   532 | dev_acc  0.4267 | test_acc  0.4376 |'
'| epoch   2 | step   798 | dev_acc  0.4365 | test_acc  0.4569 |'
'| epoch   3 | step  1064 | dev_acc  0.4292 | test_acc  0.4319 |'
'| epoch   4 | step  1330 | dev_acc  0.3374 | test_acc  0.3642 |'

In [ ]:
# /content/drive/Shareddrives/Gdrive/My_Thesis/QA_GNN/saved_models/csqa/enc-roberta-large__k5__gnndim100__bs8__seed0/model.pt.2

#Model Evaluation

##CSQA

In [ ]:
dataset="csqa"
model='roberta-large'

elr=1e-5 #encoder learning rate
dlr=1e-3  #decoder learing rate
bs=32
mbs=2   #mini-batch-size
n_epochs=5 #def 100
num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges
seed=0

k=5 #num of gnn layers
gnndim=100    #dimension of the GNN layers def 200

!echo "***** hyperparameters *****"
!echo "dataset: $dataset"
!echo "enc_name: $model"
!echo "batch_size: $bs"
!echo "learning_rate: elr $elr dlr $dlr"
!echo "gnn: dim $gnndim layer $k"
!echo "******************************"

save_dir_pref='/content/drive/Shareddrives/Gdrive/My_Thesis/QA_GNN/saved_models'
%mkdir -p $save_dir_pref
%mkdir -p logs

args=dict()

args['loss']='cross_entropy'
args['lr_schedule']='fixed'
args['bs']=32
args['batch_size']=32
args['warmup_steps']=150
args['max_grad_norm']=1.0
args['me']=10
args['log_interval']=10
args['debug']=False
args["optim"]="radam"

args["mode"]="eval_detail"    #choices=['train', 'eval_detail'], help='run training or evaluation
args["dataset"]=dataset
args["encoder"]=model
args["k"]=k   #perform k-layer message passing
args["gnn_dim"]=gnndim
args["encoder_lr"]=elr
args["decoder_lr"]=dlr
args["batch_size"]=bs
args["mini_batch_size"]=mbs
args["fp16"]=False   #use fp16 training. this requires torch>=1.6.0
args["seed"]=seed
args["num_relation"]=num_relation
args["n_epochs"]=n_epochs
args["max_epochs_before_stop"]=10
args["train_adj"]=f"data/{dataset}/graph/train.graph.adj.pk"
args["dev_adj"]=f"data/{dataset}/graph/dev.graph.adj.pk"
args["test_adj"]=f"data/{dataset}/graph/test.graph.adj.pk"
args["train_statements"]=f"data/{dataset}/statement/train.statement.jsonl"
args["dev_statements"]=f"data/{dataset}/statement/dev.statement.jsonl"
args["test_statements"]=f"data/{dataset}/statement/test.statement.jsonl"
args["save_model"]=True
args["save_dir"]=f"{save_dir_pref}/{dataset}/enc-{model}__k{k}__gnndim{gnndim}__bs{bs}__seed{seed}"
args["cuda"]=True
args["ent_emb_paths"]=['data/cpnet/tzw.ent.npy']
args["max_seq_len"]=128
args["inhouse"]=True
args["inhouse_train_qids"]=f'data/{dataset}/inhouse_split_qids.txt'
args["weight_decay"]=1e-2

args['enc']="bert-large-uncased"
args['encoder']="bert-large-uncased"
args['encoder_layer']=-1
args['elr']=2e-4
args["encoder_lr"]=2e-4
args["ent_emb"]='tzw'
args["load_model_path"]='/content/drive/Shareddrives/Gdrive/My_Thesis/QA_GNN/saved_models/csqa/enc-roberta-large__k5__gnndim100__bs32__seed0/model.pt.2'

main_5(args)

***** hyperparameters *****
dataset: csqa
enc_name: roberta-large
batch_size: 32
learning_rate: elr 1e-05 dlr 0.001
gnn: dim 100 layer 5
******************************
| num_concepts: 799273 |


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


inhouse? True
args.train_statements data/csqa/statement/train.statement.jsonl
args.dev_statements data/csqa/statement/dev.statement.jsonl
args.test_statements data/csqa/statement/test.statement.jsonl
args.train_adj data/csqa/graph/train.graph.adj.pk
args.dev_adj data/csqa/graph/dev.graph.adj.pk
args.test_adj data/csqa/graph/test.graph.adj.pk
train_statement_path data/csqa/statement/train.statement.jsonl


100%|██████████| 1221/1221 [00:00<00:00, 1232.66it/s]


num_choice 5
| ori_adj_len: mu 121.54 sigma 94.04 | adj_len: 107.96 | prune_rate： 0.17 | qc_num: 7.43 | ac_num: 2.07 |
| ori_adj_len: mu 118.44 sigma 90.55 | adj_len: 106.55 | prune_rate： 0.15 | qc_num: 7.20 | ac_num: 2.05 |


...

| ori_adj_len: mu 119.20 sigma 93.49 | adj_len: 106.22 | prune_rate： 0.16 | qc_num: 7.38 | ac_num: 2.05 |


100%|██████████| 611/611 [02:34<00:00,  3.94it/s]


dev_acc  0.4357


100%|██████████| 621/621 [02:44<00:00,  3.78it/s]


-----------------------------------------------------------------------
test_acc  0.4561
-----------------------------------------------------------------------


In [ ]:
# #!/bin/bash

# export CUDA_VISIBLE_DEVICES=0
# dt=`date '+%Y%m%d_%H%M%S'`


# dataset="csqa"
# model='roberta-large'
# shift
# shift
# args=$@


# echo "******************************"
# echo "dataset: $dataset"
# echo "******************************"

# save_dir_pref='saved_models'
# mkdir -p $save_dir_pref

# ###### Eval ######
# python3 -u qagnn.py --dataset $dataset \
#       --train_adj data/${dataset}/graph/train.graph.adj.pk \
#       --dev_adj   data/${dataset}/graph/dev.graph.adj.pk \
#       --test_adj  data/${dataset}/graph/test.graph.adj.pk \
#       --train_statements data/${dataset}/statement/train.statement.jsonl \
#       --dev_statements   data/${dataset}/statement/dev.statement.jsonl \
#       --test_statements  data/${dataset}/statement/test.statement.jsonl \
#       --save_model \
#       --save_dir saved_models \
#       --mode eval_detail \
#       --load_model_path saved_models/csqa_model_hf3.4.0.pt \
#       $args
